## Library Comparison

In this notebook we will look at how MaterialX Elements can be compared with one another.

The example case chosen is to determine what has changed between two versions of the standard
libraries. In this case version 1.38 versus the current version used to test with.

The <a href="./compareVersions.py" target="_blank">compareVersions.py</a> script is an sample utility which uses
the logic in this notebook. The results of a comparison can be seen <a href="../resources/mtlx_138_vs_1389.html" target="_blank">here</a>

The following aspects will be covered:
1. Loading in different versions into separate Documents.
2. Looking at what has been added or removed.
3. Performing a detailed comparison between Elements.

### Library Loading

We will first load in two versions of the standard libraries.
- One is the "current" one found in the 'libraries' folder of the MaterialX
Python distribution. 
- The other is a downloaded version of the same libraries copied into a renamed folder. 

In [81]:
import MaterialX as mx
from mtlxutils.mxfile import *
from IPython.display import display_markdown

# Load the current standard libraries
currLibrary, currlibFiles, status = MtlxFile.createLibraryDocument()
print(status)

# Load in the old standard libraries
otherVersion = ''
libraryFolders = [ 'libraries1.38' ]
otherLibrary = mx.createDocument()
searchPath = mx.FileSearchPath('../resources/')
# Attempt to preserve the original version of the library
options = mx.XmlReadOptions()
options.upgradeVersion = False

libFiles = mx.loadLibraries(libraryFolders, searchPath, otherLibrary)
print('- Loaded %d old library definitions from %d files' % (len(otherLibrary.getNodeDefs()), len(libFiles)))

- Loaded 746 library definitions from 46 files
- Loaded 0 old library definitions from 0 files


It is possible to selectively filter out which files by providing additional arguments to `loadLibraries` or to perform the equivalent traversal explicitly as shown below.

In [82]:
otherLibrary2 = mx.createDocument()
otherVersion2 = ''
loadedLibraries = set()
readOptions = mx.XmlReadOptions()
readOptions.upgradeVersion = False
for libraryName in libraryFolders:
    libraryPath = searchPath.find(libraryName)
    for path in libraryPath.getSubDirectories():
          for filename in path.getFilesInDirectory('mtlx'):
                file = path / filename;
                if file.asString() not in loadedLibraries:
                    mx.loadLibrary(file, otherLibrary2, searchPath, readOptions)                    
                    loadedLibraries.add(file.asString())
                    otherVersion2 = otherLibrary2.getVersionString()  

print('Read in %d definitions from %d files. Version: %s' % 
      (len(otherLibrary2.getNodeDefs()), len(loadedLibraries), otherVersion2))

Read in 0 definitions from 0 files. Version: 


Note that both libraries will return same version when querying using `getVersionString()` on the document.
At the time of writing we are using a patch and thus grab the version from the Python package instead.

It would be useful if there was a way to find patch versions on documents but this is currently
not supported. Integrations could write additional `patch` meta-data atttibutes but this would not
be standard.

In [83]:
print('- Current library / package version: %s' % mx.getVersionString())
print('- Old library version: %s' % otherLibrary.getVersionString())

- Current library / package version: 1.38.9
- Old library version: 1.38


### Testing for Definition Additions and Removals 

The first test is just a check on the number of definitions in each file. The result found is that there are more definitions, and there appear to be no definitions which have been removed.

In [84]:
# Compare number of nodedefs
currNodeDefs = currLibrary.getNodeDefs()
currNodeDefsCount = len(currNodeDefs)
otherNodeDefs = otherLibrary.getNodeDefs()
otherNodeDefsCount = len(otherNodeDefs) # There seems to be no way to get the previous version?
print('There are %d new definitions.' % (currNodeDefsCount - otherNodeDefsCount))
print('- Current library has %d nodedefs' % currNodeDefsCount)
print('- Other library has %d nodedefs' % otherNodeDefsCount)

There are 746 new definitions.
- Current library has 746 nodedefs
- Other library has 0 nodedefs


To setup for comparision testing, we build up a dictionary of node definitions keyed by the name of the definition.

At time of writing the convention is that definitions cannot be defined at any location other that the child of root Document. Thus only the name is used as the key. The path could also be used for long term robustness if this convention ever changes.

In [85]:
# Create dictionary of current node definitions
currNodeDefSet = {}
for nodeDef in currNodeDefs:
    currNodeDefSet[nodeDef.getName()] = nodeDef

# Create dictionary of old node definitions
otherNodeDefSet = {}
for nodeDef in otherNodeDefs:
    otherNodeDefSet[nodeDef.getName()] = nodeDef


The first check is to see if any definitions have been removed. 

If a definition has been remapped then the hope is that the update process performed on load will handle this. For example when material nodes replaced material elements as part of 1.38.7 the update process performed the appropriate remapping.  

If the definition no longer exists then the integration will need to find out how to handle this. In general, this should hopefully never occur.

In [86]:
    
# Find out if any definitions were removed
removedNodeDefs = []
for nodeDef in otherNodeDefs:
    if nodeDef.getName() not in currNodeDefSet:
        removedNodeDefs.append(nodeDef)

text = ''  
if removedNodeDefs:
    removedNodeDefs = sorted(removedNodeDefs, key=lambda x: x.getName())
    text = text + '| Name | Category |\n'
    text = text + '| :--- | :--- |\n'
    for nd in removedNodeDefs:
        text = text + '| %s | %s |\n' % (nd.getName(), nd.getNodeString())

    delta = '%d node definitions removed' % len(removedNodeDefs)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

The next check is to see if any definitions have been added and list them out.

At the same time we cache a list of definitions which exist in both the old and new library versions for later
comparison.

In [87]:
# Find out what new definitions have been added.
# Cache common definitions in `compareDetails` for further comparison.
compareDetails = []
newNodeDefs = []
for nodeDef in currNodeDefs:
    if nodeDef.getName() not in otherNodeDefSet:
        newNodeDefs.append(nodeDef)
    else:
        # Cache these two for more comparison
        otherNodeDef = otherNodeDefSet[nodeDef.getName()]
        compareDetails.append( [otherNodeDef, nodeDef] )

text = ''
if newNodeDefs:
    newNodeDefs = sorted(newNodeDefs, key=lambda x: x.getNodeString())
    text = text + '| Name | Category | NodeGroup |\n'
    text = text + '| :--- | :--- | :---| \n'
    for nd in newNodeDefs:
        text = text + '| <a href="../documents/definitions/%s.html">%s</a> | %s | %s |\n' % (nd.getNodeString(), nd.getName(), nd.getNodeString(), nd.getNodeGroup())

    delta = '%d node definitions added. (Sorted by category)' % len(newNodeDefs)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

<details open><summary>746 node definitions added. (Sorted by category)</summary>

| Name | Category | NodeGroup |
| :--- | :--- | :---| 
| <a href="../documents/definitions/LamaAdd.html">ND_lama_add_bsdf</a> | LamaAdd | pbr |
| <a href="../documents/definitions/LamaAdd.html">ND_lama_add_edf</a> | LamaAdd | pbr |
| <a href="../documents/definitions/LamaConductor.html">ND_lama_conductor</a> | LamaConductor | pbr |
| <a href="../documents/definitions/LamaDielectric.html">ND_lama_dielectric</a> | LamaDielectric | pbr |
| <a href="../documents/definitions/LamaDiffuse.html">ND_lama_diffuse</a> | LamaDiffuse | pbr |
| <a href="../documents/definitions/LamaEmission.html">ND_lama_emission</a> | LamaEmission | pbr |
| <a href="../documents/definitions/LamaLayer.html">ND_lama_layer_bsdf</a> | LamaLayer | pbr |
| <a href="../documents/definitions/LamaMix.html">ND_lama_mix_bsdf</a> | LamaMix | pbr |
| <a href="../documents/definitions/LamaMix.html">ND_lama_mix_edf</a> | LamaMix | pbr |
| <a href="../documents/definitions/LamaSSS.html">ND_lama_sss</a> | LamaSSS | pbr |
| <a href="../documents/definitions/LamaSheen.html">ND_lama_sheen</a> | LamaSheen | pbr |
| <a href="../documents/definitions/LamaTranslucent.html">ND_lama_translucent</a> | LamaTranslucent | pbr |
| <a href="../documents/definitions/UsdPreviewSurface.html">ND_UsdPreviewSurface_surfaceshader</a> | UsdPreviewSurface | pbr |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_integer</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_boolean</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_string</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_float</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector2</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector3</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector4</a> | UsdPrimvarReader | geometric |
| <a href="../documents/definitions/UsdTransform2d.html">ND_UsdTransform2d</a> | UsdTransform2d | math |
| <a href="../documents/definitions/UsdUVTexture.html">ND_UsdUVTexture</a> | UsdUVTexture | texture2d |
| <a href="../documents/definitions/UsdUVTexture.html">ND_UsdUVTexture_23</a> | UsdUVTexture | texture2d |
| <a href="../documents/definitions/absorption_vdf.html">ND_absorption_vdf</a> | absorption_vdf | pbr |
| <a href="../documents/definitions/absval.html">ND_absval_float</a> | absval | math |
| <a href="../documents/definitions/absval.html">ND_absval_color3</a> | absval | math |
| <a href="../documents/definitions/absval.html">ND_absval_color4</a> | absval | math |
| <a href="../documents/definitions/absval.html">ND_absval_vector2</a> | absval | math |
| <a href="../documents/definitions/absval.html">ND_absval_vector3</a> | absval | math |
| <a href="../documents/definitions/absval.html">ND_absval_vector4</a> | absval | math |
| <a href="../documents/definitions/acescg_to_lin_rec709.html">ND_acescg_to_lin_rec709_color3</a> | acescg_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/acescg_to_lin_rec709.html">ND_acescg_to_lin_rec709_color4</a> | acescg_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/acos.html">ND_acos_float</a> | acos | math |
| <a href="../documents/definitions/acos.html">ND_acos_vector2</a> | acos | math |
| <a href="../documents/definitions/acos.html">ND_acos_vector3</a> | acos | math |
| <a href="../documents/definitions/acos.html">ND_acos_vector4</a> | acos | math |
| <a href="../documents/definitions/add.html">ND_add_bsdf</a> | add | pbr |
| <a href="../documents/definitions/add.html">ND_add_edf</a> | add | pbr |
| <a href="../documents/definitions/add.html">ND_add_vdf</a> | add | pbr |
| <a href="../documents/definitions/add.html">ND_add_float</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_color3</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_color4</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector2</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector3</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector4</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_matrix33</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_matrix44</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_color3FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_color4FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector2FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector3FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_vector4FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_matrix33FA</a> | add | math |
| <a href="../documents/definitions/add.html">ND_add_matrix44FA</a> | add | math |
| <a href="../documents/definitions/adobergb_to_lin_rec709.html">ND_adobergb_to_lin_rec709_color3</a> | adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/adobergb_to_lin_rec709.html">ND_adobergb_to_lin_rec709_color4</a> | adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/ambientocclusion.html">ND_ambientocclusion_float</a> | ambientocclusion | global |
| <a href="../documents/definitions/anisotropic_vdf.html">ND_anisotropic_vdf</a> | anisotropic_vdf | pbr |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_integer_integerarray</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_integerarray_integerarray</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_float_floatarray</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_floatarray_floatarray</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_color3_color3array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_color3array_color3array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_color4_color4array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_color4array_color4array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector2_vector2array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector2array_vector2array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector3_vector3array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector3array_vector3array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector4_vector4array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_vector4array_vector4array</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_string_stringarray</a> | arrayappend | math |
| <a href="../documents/definitions/arrayappend.html">ND_arrayappend_stringarray_stringarray</a> | arrayappend | math |
| <a href="../documents/definitions/artistic_ior.html">ND_artistic_ior</a> | artistic_ior | pbr |
| <a href="../documents/definitions/asin.html">ND_asin_float</a> | asin | math |
| <a href="../documents/definitions/asin.html">ND_asin_vector2</a> | asin | math |
| <a href="../documents/definitions/asin.html">ND_asin_vector3</a> | asin | math |
| <a href="../documents/definitions/asin.html">ND_asin_vector4</a> | asin | math |
| <a href="../documents/definitions/atan2.html">ND_atan2_float</a> | atan2 | math |
| <a href="../documents/definitions/atan2.html">ND_atan2_vector2</a> | atan2 | math |
| <a href="../documents/definitions/atan2.html">ND_atan2_vector3</a> | atan2 | math |
| <a href="../documents/definitions/atan2.html">ND_atan2_vector4</a> | atan2 | math |
| <a href="../documents/definitions/bitangent.html">ND_bitangent_vector3</a> | bitangent | geometric |
| <a href="../documents/definitions/blackbody.html">ND_blackbody</a> | blackbody | pbr |
| <a href="../documents/definitions/blur.html">ND_blur_float</a> | blur | convolution2d |
| <a href="../documents/definitions/blur.html">ND_blur_color3</a> | blur | convolution2d |
| <a href="../documents/definitions/blur.html">ND_blur_color4</a> | blur | convolution2d |
| <a href="../documents/definitions/blur.html">ND_blur_vector2</a> | blur | convolution2d |
| <a href="../documents/definitions/blur.html">ND_blur_vector3</a> | blur | convolution2d |
| <a href="../documents/definitions/blur.html">ND_blur_vector4</a> | blur | convolution2d |
| <a href="../documents/definitions/bump.html">ND_bump_vector3</a> | bump | geometric |
| <a href="../documents/definitions/burley_diffuse_bsdf.html">ND_burley_diffuse_bsdf</a> | burley_diffuse_bsdf | pbr |
| <a href="../documents/definitions/burn.html">ND_burn_float</a> | burn | compositing |
| <a href="../documents/definitions/burn.html">ND_burn_color3</a> | burn | compositing |
| <a href="../documents/definitions/burn.html">ND_burn_color4</a> | burn | compositing |
| <a href="../documents/definitions/ceil.html">ND_ceil_float</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_color3</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_color4</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_vector2</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_vector3</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_vector4</a> | ceil | math |
| <a href="../documents/definitions/ceil.html">ND_ceil_integer</a> | ceil | math |
| <a href="../documents/definitions/cellnoise2d.html">ND_cellnoise2d_float</a> | cellnoise2d | procedural2d |
| <a href="../documents/definitions/cellnoise3d.html">ND_cellnoise3d_float</a> | cellnoise3d | procedural3d |
| <a href="../documents/definitions/checkerboard.html">ND_checkerboard_color3</a> | checkerboard | procedural2d |
| <a href="../documents/definitions/circle.html">ND_circle_float</a> | circle | procedural2d |
| <a href="../documents/definitions/clamp.html">ND_clamp_float</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_color3</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_color4</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector2</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector3</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector4</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_color3FA</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_color4FA</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector2FA</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector3FA</a> | clamp | math |
| <a href="../documents/definitions/clamp.html">ND_clamp_vector4FA</a> | clamp | math |
| <a href="../documents/definitions/cloverleaf.html">ND_cloverleaf_float</a> | cloverleaf | procedural2d |
| <a href="../documents/definitions/colorcorrect.html">ND_colorcorrect_color3</a> | colorcorrect | adjustment |
| <a href="../documents/definitions/colorcorrect.html">ND_colorcorrect_color4</a> | colorcorrect | adjustment |
| <a href="../documents/definitions/combine2.html">ND_combine2_vector2</a> | combine2 | channel |
| <a href="../documents/definitions/combine2.html">ND_combine2_color4CF</a> | combine2 | channel |
| <a href="../documents/definitions/combine2.html">ND_combine2_vector4VF</a> | combine2 | channel |
| <a href="../documents/definitions/combine2.html">ND_combine2_vector4VV</a> | combine2 | channel |
| <a href="../documents/definitions/combine3.html">ND_combine3_color3</a> | combine3 | channel |
| <a href="../documents/definitions/combine3.html">ND_combine3_vector3</a> | combine3 | channel |
| <a href="../documents/definitions/combine4.html">ND_combine4_color4</a> | combine4 | channel |
| <a href="../documents/definitions/combine4.html">ND_combine4_vector4</a> | combine4 | channel |
| <a href="../documents/definitions/conductor_bsdf.html">ND_conductor_bsdf</a> | conductor_bsdf | pbr |
| <a href="../documents/definitions/conical_edf.html">ND_conical_edf</a> | conical_edf | pbr |
| <a href="../documents/definitions/constant.html">ND_constant_float</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_color3</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_color4</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_vector2</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_vector3</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_vector4</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_boolean</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_integer</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_matrix33</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_matrix44</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_string</a> | constant | procedural |
| <a href="../documents/definitions/constant.html">ND_constant_filename</a> | constant | procedural |
| <a href="../documents/definitions/contrast.html">ND_contrast_float</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_color3</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_color4</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector2</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector3</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector4</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_color3FA</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_color4FA</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector2FA</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector3FA</a> | contrast | adjustment |
| <a href="../documents/definitions/contrast.html">ND_contrast_vector4FA</a> | contrast | adjustment |
| <a href="../documents/definitions/convert.html">ND_convert_float_color3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_float_color4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_float_vector2</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_float_vector3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_float_vector4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector2_vector3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector3_color3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector3_vector2</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector3_vector4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector4_color4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_vector4_vector3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_color3_vector3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_color4_vector4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_color3_color4</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_color4_color3</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_boolean_float</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_integer_float</a> | convert | channel |
| <a href="../documents/definitions/convert.html">ND_convert_color3_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_color4_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_float_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector2_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector3_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector4_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_integer_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_boolean_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/cos.html">ND_cos_float</a> | cos | math |
| <a href="../documents/definitions/cos.html">ND_cos_vector2</a> | cos | math |
| <a href="../documents/definitions/cos.html">ND_cos_vector3</a> | cos | math |
| <a href="../documents/definitions/cos.html">ND_cos_vector4</a> | cos | math |
| <a href="../documents/definitions/creatematrix.html">ND_creatematrix_vector3_matrix33</a> | creatematrix | math |
| <a href="../documents/definitions/creatematrix.html">ND_creatematrix_vector3_matrix44</a> | creatematrix | math |
| <a href="../documents/definitions/creatematrix.html">ND_creatematrix_vector4_matrix44</a> | creatematrix | math |
| <a href="../documents/definitions/crosshatch.html">ND_crosshatch_color3</a> | crosshatch | procedural2d |
| <a href="../documents/definitions/crossproduct.html">ND_crossproduct_vector3</a> | crossproduct | math |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_float</a> | curveadjust | adjustment |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_color3</a> | curveadjust | adjustment |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_color4</a> | curveadjust | adjustment |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_vector2</a> | curveadjust | adjustment |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_vector3</a> | curveadjust | adjustment |
| <a href="../documents/definitions/curveadjust.html">ND_curveadjust_vector4</a> | curveadjust | adjustment |
| <a href="../documents/definitions/determinant.html">ND_determinant_matrix33</a> | determinant | math |
| <a href="../documents/definitions/determinant.html">ND_determinant_matrix44</a> | determinant | math |
| <a href="../documents/definitions/dielectric_bsdf.html">ND_dielectric_bsdf</a> | dielectric_bsdf | pbr |
| <a href="../documents/definitions/difference.html">ND_difference_float</a> | difference | compositing |
| <a href="../documents/definitions/difference.html">ND_difference_color3</a> | difference | compositing |
| <a href="../documents/definitions/difference.html">ND_difference_color4</a> | difference | compositing |
| <a href="../documents/definitions/directional_light.html">ND_directional_light</a> | directional_light | light |
| <a href="../documents/definitions/disjointover.html">ND_disjointover_color4</a> | disjointover | compositing |
| <a href="../documents/definitions/disney_brdf_2012.html">ND_disney_brdf_2012_surface</a> | disney_brdf_2012 | pbr |
| <a href="../documents/definitions/disney_bsdf_2015.html">ND_disney_bsdf_2015_surface</a> | disney_bsdf_2015 | pbr |
| <a href="../documents/definitions/displacement.html">ND_displacement_float</a> | displacement | pbr |
| <a href="../documents/definitions/displacement.html">ND_displacement_vector3</a> | displacement | pbr |
| <a href="../documents/definitions/distance.html">ND_distance_vector2</a> | distance | math |
| <a href="../documents/definitions/distance.html">ND_distance_vector3</a> | distance | math |
| <a href="../documents/definitions/distance.html">ND_distance_vector4</a> | distance | math |
| <a href="../documents/definitions/divide.html">ND_divide_float</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_color3</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_color4</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector2</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector3</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector4</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_matrix33</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_matrix44</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_color3FA</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_color4FA</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector2FA</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector3FA</a> | divide | math |
| <a href="../documents/definitions/divide.html">ND_divide_vector4FA</a> | divide | math |
| <a href="../documents/definitions/dodge.html">ND_dodge_float</a> | dodge | compositing |
| <a href="../documents/definitions/dodge.html">ND_dodge_color3</a> | dodge | compositing |
| <a href="../documents/definitions/dodge.html">ND_dodge_color4</a> | dodge | compositing |
| <a href="../documents/definitions/dot.html">ND_dot_float</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_color3</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_color4</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_vector2</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_vector3</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_vector4</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_boolean</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_integer</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_matrix33</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_matrix44</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_string</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_filename</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_surfaceshader</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_displacementshader</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_volumeshader</a> | dot | organization |
| <a href="../documents/definitions/dot.html">ND_dot_lightshader</a> | dot | organization |
| <a href="../documents/definitions/dotproduct.html">ND_dotproduct_vector2</a> | dotproduct | math |
| <a href="../documents/definitions/dotproduct.html">ND_dotproduct_vector3</a> | dotproduct | math |
| <a href="../documents/definitions/dotproduct.html">ND_dotproduct_vector4</a> | dotproduct | math |
| <a href="../documents/definitions/exp.html">ND_exp_float</a> | exp | math |
| <a href="../documents/definitions/exp.html">ND_exp_vector2</a> | exp | math |
| <a href="../documents/definitions/exp.html">ND_exp_vector3</a> | exp | math |
| <a href="../documents/definitions/exp.html">ND_exp_vector4</a> | exp | math |
| <a href="../documents/definitions/extract.html">ND_extract_color3</a> | extract | channel |
| <a href="../documents/definitions/extract.html">ND_extract_color4</a> | extract | channel |
| <a href="../documents/definitions/extract.html">ND_extract_vector2</a> | extract | channel |
| <a href="../documents/definitions/extract.html">ND_extract_vector3</a> | extract | channel |
| <a href="../documents/definitions/extract.html">ND_extract_vector4</a> | extract | channel |
| <a href="../documents/definitions/facingratio.html">ND_facingratio_float</a> | facingratio | npr |
| <a href="../documents/definitions/floor.html">ND_floor_float</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_color3</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_color4</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_vector2</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_vector3</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_vector4</a> | floor | math |
| <a href="../documents/definitions/floor.html">ND_floor_integer</a> | floor | math |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_float</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_color3</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_color4</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector2</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector3</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector4</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_color3FA</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_color4FA</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector2FA</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector3FA</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/fractal3d.html">ND_fractal3d_vector4FA</a> | fractal3d | procedural3d |
| <a href="../documents/definitions/frame.html">ND_frame_float</a> | frame | application |
| <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">ND_g18_rec709_to_lin_rec709_color3</a> | g18_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">ND_g18_rec709_to_lin_rec709_color4</a> | g18_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">ND_g22_ap1_to_lin_rec709_color3</a> | g22_ap1_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">ND_g22_ap1_to_lin_rec709_color4</a> | g22_ap1_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">ND_g22_rec709_to_lin_rec709_color3</a> | g22_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">ND_g22_rec709_to_lin_rec709_color4</a> | g22_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/generalized_schlick_bsdf.html">ND_generalized_schlick_bsdf</a> | generalized_schlick_bsdf | pbr |
| <a href="../documents/definitions/generalized_schlick_edf.html">ND_generalized_schlick_edf</a> | generalized_schlick_edf | pbr |
| <a href="../documents/definitions/geomcolor.html">ND_geomcolor_float</a> | geomcolor | geometric |
| <a href="../documents/definitions/geomcolor.html">ND_geomcolor_color3</a> | geomcolor | geometric |
| <a href="../documents/definitions/geomcolor.html">ND_geomcolor_color4</a> | geomcolor | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_integer</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_boolean</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_string</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_float</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_color3</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_color4</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_vector2</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_vector3</a> | geompropvalue | geometric |
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_vector4</a> | geompropvalue | geometric |
| <a href="../documents/definitions/glossiness_anisotropy.html">ND_glossiness_anisotropy</a> | glossiness_anisotropy | pbr |
| <a href="../documents/definitions/gltf_colorimage.html">ND_gltf_colorimage</a> | gltf_colorimage | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_color3_color3_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_color4_color4_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_float_float_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_vector3_vector3_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_iridescence_thickness.html">ND_gltf_iridescence_thickness_float_1_0</a> | gltf_iridescence_thickness | texture2d |
| <a href="../documents/definitions/gltf_normalmap.html">ND_gltf_normalmap_vector3_1_0</a> | gltf_normalmap | texture2d |
| <a href="../documents/definitions/gltf_pbr.html">ND_gltf_pbr_surfaceshader</a> | gltf_pbr | pbr |
| <a href="../documents/definitions/grid.html">ND_grid_color3</a> | grid | procedural2d |
| <a href="../documents/definitions/heighttonormal.html">ND_heighttonormal_vector3</a> | heighttonormal | convolution2d |
| <a href="../documents/definitions/hexagon.html">ND_hexagon_float</a> | hexagon | procedural2d |
| <a href="../documents/definitions/hsvadjust.html">ND_hsvadjust_color3</a> | hsvadjust | adjustment |
| <a href="../documents/definitions/hsvadjust.html">ND_hsvadjust_color4</a> | hsvadjust | adjustment |
| <a href="../documents/definitions/hsvtorgb.html">ND_hsvtorgb_color3</a> | hsvtorgb | adjustment |
| <a href="../documents/definitions/hsvtorgb.html">ND_hsvtorgb_color4</a> | hsvtorgb | adjustment |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_float</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color3</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color4</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector2</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector3</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector4</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_floatI</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color3I</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color4I</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector2I</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector3I</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector4I</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_floatB</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color3B</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_color4B</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector2B</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector3B</a> | ifequal | conditional |
| <a href="../documents/definitions/ifequal.html">ND_ifequal_vector4B</a> | ifequal | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_float</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_color3</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_color4</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector2</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector3</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector4</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_floatI</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_color3I</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_color4I</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector2I</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector3I</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreater.html">ND_ifgreater_vector4I</a> | ifgreater | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_float</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_color3</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_color4</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector2</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector3</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector4</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_floatI</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_color3I</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_color4I</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector2I</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector3I</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/ifgreatereq.html">ND_ifgreatereq_vector4I</a> | ifgreatereq | conditional |
| <a href="../documents/definitions/image.html">ND_image_float</a> | image | texture2d |
| <a href="../documents/definitions/image.html">ND_image_color3</a> | image | texture2d |
| <a href="../documents/definitions/image.html">ND_image_color4</a> | image | texture2d |
| <a href="../documents/definitions/image.html">ND_image_vector2</a> | image | texture2d |
| <a href="../documents/definitions/image.html">ND_image_vector3</a> | image | texture2d |
| <a href="../documents/definitions/image.html">ND_image_vector4</a> | image | texture2d |
| <a href="../documents/definitions/in.html">ND_in_color4</a> | in | compositing |
| <a href="../documents/definitions/inside.html">ND_inside_float</a> | inside | compositing |
| <a href="../documents/definitions/inside.html">ND_inside_color3</a> | inside | compositing |
| <a href="../documents/definitions/inside.html">ND_inside_color4</a> | inside | compositing |
| <a href="../documents/definitions/invert.html">ND_invert_float</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_color3</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_color4</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector2</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector3</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector4</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_color3FA</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_color4FA</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector2FA</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector3FA</a> | invert | math |
| <a href="../documents/definitions/invert.html">ND_invert_vector4FA</a> | invert | math |
| <a href="../documents/definitions/invertmatrix.html">ND_invertmatrix_matrix33</a> | invertmatrix | math |
| <a href="../documents/definitions/invertmatrix.html">ND_invertmatrix_matrix44</a> | invertmatrix | math |
| <a href="../documents/definitions/layer.html">ND_layer_bsdf</a> | layer | pbr |
| <a href="../documents/definitions/layer.html">ND_layer_vdf</a> | layer | pbr |
| <a href="../documents/definitions/light.html">ND_light</a> | light | pbr |
| <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">ND_lin_adobergb_to_lin_rec709_color3</a> | lin_adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">ND_lin_adobergb_to_lin_rec709_color4</a> | lin_adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">ND_lin_displayp3_to_lin_rec709_color3</a> | lin_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">ND_lin_displayp3_to_lin_rec709_color4</a> | lin_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/line.html">ND_line_float</a> | line | procedural2d |
| <a href="../documents/definitions/ln.html">ND_ln_float</a> | ln | math |
| <a href="../documents/definitions/ln.html">ND_ln_vector2</a> | ln | math |
| <a href="../documents/definitions/ln.html">ND_ln_vector3</a> | ln | math |
| <a href="../documents/definitions/ln.html">ND_ln_vector4</a> | ln | math |
| <a href="../documents/definitions/luminance.html">ND_luminance_color3</a> | luminance | adjustment |
| <a href="../documents/definitions/luminance.html">ND_luminance_color4</a> | luminance | adjustment |
| <a href="../documents/definitions/magnitude.html">ND_magnitude_vector2</a> | magnitude | math |
| <a href="../documents/definitions/magnitude.html">ND_magnitude_vector3</a> | magnitude | math |
| <a href="../documents/definitions/magnitude.html">ND_magnitude_vector4</a> | magnitude | math |
| <a href="../documents/definitions/mask.html">ND_mask_color4</a> | mask | compositing |
| <a href="../documents/definitions/matte.html">ND_matte_color4</a> | matte | compositing |
| <a href="../documents/definitions/max.html">ND_max_float</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_color3</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_color4</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector2</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector3</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector4</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_color3FA</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_color4FA</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector2FA</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector3FA</a> | max | math |
| <a href="../documents/definitions/max.html">ND_max_vector4FA</a> | max | math |
| <a href="../documents/definitions/measured_edf.html">ND_measured_edf</a> | measured_edf | pbr |
| <a href="../documents/definitions/min.html">ND_min_float</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_color3</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_color4</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector2</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector3</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector4</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_color3FA</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_color4FA</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector2FA</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector3FA</a> | min | math |
| <a href="../documents/definitions/min.html">ND_min_vector4FA</a> | min | math |
| <a href="../documents/definitions/minus.html">ND_minus_float</a> | minus | compositing |
| <a href="../documents/definitions/minus.html">ND_minus_color3</a> | minus | compositing |
| <a href="../documents/definitions/minus.html">ND_minus_color4</a> | minus | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_bsdf</a> | mix | pbr |
| <a href="../documents/definitions/mix.html">ND_mix_edf</a> | mix | pbr |
| <a href="../documents/definitions/mix.html">ND_mix_vdf</a> | mix | pbr |
| <a href="../documents/definitions/mix.html">ND_mix_float</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_color3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_color3_color3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_color4</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_color4_color4</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector2</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector2_vector2</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector3_vector3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector4</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector4_vector4</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_surfaceshader</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_displacementshader</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_volumeshader</a> | mix | compositing |
| <a href="../documents/definitions/modulo.html">ND_modulo_float</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_color3</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_color4</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector2</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector3</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector4</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_color3FA</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_color4FA</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector2FA</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector3FA</a> | modulo | math |
| <a href="../documents/definitions/modulo.html">ND_modulo_vector4FA</a> | modulo | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_bsdfC</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_bsdfF</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_edfC</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_edfF</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_vdfC</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_vdfF</a> | multiply | pbr |
| <a href="../documents/definitions/multiply.html">ND_multiply_float</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_color3</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_color4</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector2</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector3</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector4</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_matrix33</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_matrix44</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_color3FA</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_color4FA</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector2FA</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector3FA</a> | multiply | math |
| <a href="../documents/definitions/multiply.html">ND_multiply_vector4FA</a> | multiply | math |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_float</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_color3</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_color4</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector2</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector3</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector4</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_color3FA</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_color4FA</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector2FA</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector3FA</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise2d.html">ND_noise2d_vector4FA</a> | noise2d | procedural2d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_float</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_color3</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_color4</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector2</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector3</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector4</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_color3FA</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_color4FA</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector2FA</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector3FA</a> | noise3d | procedural3d |
| <a href="../documents/definitions/noise3d.html">ND_noise3d_vector4FA</a> | noise3d | procedural3d |
| <a href="../documents/definitions/normal.html">ND_normal_vector3</a> | normal | geometric |
| <a href="../documents/definitions/normalize.html">ND_normalize_vector2</a> | normalize | math |
| <a href="../documents/definitions/normalize.html">ND_normalize_vector3</a> | normalize | math |
| <a href="../documents/definitions/normalize.html">ND_normalize_vector4</a> | normalize | math |
| <a href="../documents/definitions/normalmap.html">ND_normalmap</a> | normalmap | math |
| <a href="../documents/definitions/normalmap.html">ND_normalmap_vector2</a> | normalmap | math |
| <a href="../documents/definitions/open_pbr_surface.html">ND_open_pbr_surface_surfaceshader</a> | open_pbr_surface | pbr |
| <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">ND_oren_nayar_diffuse_bsdf</a> | oren_nayar_diffuse_bsdf | pbr |
| <a href="../documents/definitions/out.html">ND_out_color4</a> | out | compositing |
| <a href="../documents/definitions/outside.html">ND_outside_float</a> | outside | compositing |
| <a href="../documents/definitions/outside.html">ND_outside_color3</a> | outside | compositing |
| <a href="../documents/definitions/outside.html">ND_outside_color4</a> | outside | compositing |
| <a href="../documents/definitions/over.html">ND_over_color4</a> | over | compositing |
| <a href="../documents/definitions/overlay.html">ND_overlay_float</a> | overlay | compositing |
| <a href="../documents/definitions/overlay.html">ND_overlay_color3</a> | overlay | compositing |
| <a href="../documents/definitions/overlay.html">ND_overlay_color4</a> | overlay | compositing |
| <a href="../documents/definitions/place2d.html">ND_place2d_vector2</a> | place2d | math |
| <a href="../documents/definitions/plus.html">ND_plus_float</a> | plus | compositing |
| <a href="../documents/definitions/plus.html">ND_plus_color3</a> | plus | compositing |
| <a href="../documents/definitions/plus.html">ND_plus_color4</a> | plus | compositing |
| <a href="../documents/definitions/point_light.html">ND_point_light</a> | point_light | light |
| <a href="../documents/definitions/position.html">ND_position_vector3</a> | position | geometric |
| <a href="../documents/definitions/power.html">ND_power_float</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_color3</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_color4</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector2</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector3</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector4</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_color3FA</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_color4FA</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector2FA</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector3FA</a> | power | math |
| <a href="../documents/definitions/power.html">ND_power_vector4FA</a> | power | math |
| <a href="../documents/definitions/premult.html">ND_premult_color4</a> | premult | compositing |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_float</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_color3</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_color4</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_vector2</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_vector3</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramp4.html">ND_ramp4_vector4</a> | ramp4 | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_float</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_color3</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_color4</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_vector2</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_vector3</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramplr.html">ND_ramplr_vector4</a> | ramplr | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_float</a> | ramptb | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_color3</a> | ramptb | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_color4</a> | ramptb | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_vector2</a> | ramptb | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_vector3</a> | ramptb | procedural2d |
| <a href="../documents/definitions/ramptb.html">ND_ramptb_vector4</a> | ramptb | procedural2d |
| <a href="../documents/definitions/randomcolor.html">ND_randomcolor_float</a> | randomcolor | procedural3d |
| <a href="../documents/definitions/randomcolor.html">ND_randomcolor_integer</a> | randomcolor | procedural3d |
| <a href="../documents/definitions/randomfloat.html">ND_randomfloat_float</a> | randomfloat | procedural |
| <a href="../documents/definitions/randomfloat.html">ND_randomfloat_integer</a> | randomfloat | procedural |
| <a href="../documents/definitions/range.html">ND_range_float</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_color3</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_color4</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector2</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector3</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector4</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_color3FA</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_color4FA</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector2FA</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector3FA</a> | range | adjustment |
| <a href="../documents/definitions/range.html">ND_range_vector4FA</a> | range | adjustment |
| <a href="../documents/definitions/rec709_display_to_lin_rec709.html">ND_rec709_display_to_lin_rec709_color3</a> | rec709_display_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/rec709_display_to_lin_rec709.html">ND_rec709_display_to_lin_rec709_color4</a> | rec709_display_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/remap.html">ND_remap_float</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_color3</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_color4</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector2</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector3</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector4</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_color3FA</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_color4FA</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector2FA</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector3FA</a> | remap | adjustment |
| <a href="../documents/definitions/remap.html">ND_remap_vector4FA</a> | remap | adjustment |
| <a href="../documents/definitions/rgbtohsv.html">ND_rgbtohsv_color3</a> | rgbtohsv | adjustment |
| <a href="../documents/definitions/rgbtohsv.html">ND_rgbtohsv_color4</a> | rgbtohsv | adjustment |
| <a href="../documents/definitions/rotate2d.html">ND_rotate2d_vector2</a> | rotate2d | math |
| <a href="../documents/definitions/rotate3d.html">ND_rotate3d_vector3</a> | rotate3d | math |
| <a href="../documents/definitions/roughness_anisotropy.html">ND_roughness_anisotropy</a> | roughness_anisotropy | pbr |
| <a href="../documents/definitions/roughness_dual.html">ND_roughness_dual</a> | roughness_dual | pbr |
| <a href="../documents/definitions/saturate.html">ND_saturate_color3</a> | saturate | adjustment |
| <a href="../documents/definitions/saturate.html">ND_saturate_color4</a> | saturate | adjustment |
| <a href="../documents/definitions/screen.html">ND_screen_float</a> | screen | compositing |
| <a href="../documents/definitions/screen.html">ND_screen_color3</a> | screen | compositing |
| <a href="../documents/definitions/screen.html">ND_screen_color4</a> | screen | compositing |
| <a href="../documents/definitions/separate2.html">ND_separate2_vector2</a> | separate2 | channel |
| <a href="../documents/definitions/separate3.html">ND_separate3_color3</a> | separate3 | channel |
| <a href="../documents/definitions/separate3.html">ND_separate3_vector3</a> | separate3 | channel |
| <a href="../documents/definitions/separate4.html">ND_separate4_color4</a> | separate4 | channel |
| <a href="../documents/definitions/separate4.html">ND_separate4_vector4</a> | separate4 | channel |
| <a href="../documents/definitions/sheen_bsdf.html">ND_sheen_bsdf</a> | sheen_bsdf | pbr |
| <a href="../documents/definitions/sign.html">ND_sign_float</a> | sign | math |
| <a href="../documents/definitions/sign.html">ND_sign_color3</a> | sign | math |
| <a href="../documents/definitions/sign.html">ND_sign_color4</a> | sign | math |
| <a href="../documents/definitions/sign.html">ND_sign_vector2</a> | sign | math |
| <a href="../documents/definitions/sign.html">ND_sign_vector3</a> | sign | math |
| <a href="../documents/definitions/sign.html">ND_sign_vector4</a> | sign | math |
| <a href="../documents/definitions/sin.html">ND_sin_float</a> | sin | math |
| <a href="../documents/definitions/sin.html">ND_sin_vector2</a> | sin | math |
| <a href="../documents/definitions/sin.html">ND_sin_vector3</a> | sin | math |
| <a href="../documents/definitions/sin.html">ND_sin_vector4</a> | sin | math |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_float</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_color3</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_color4</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector2</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector3</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector4</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_color3FA</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_color4FA</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector2FA</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector3FA</a> | smoothstep | adjustment |
| <a href="../documents/definitions/smoothstep.html">ND_smoothstep_vector4FA</a> | smoothstep | adjustment |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_float</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_color3</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_color4</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_vector2</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_vector3</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splitlr.html">ND_splitlr_vector4</a> | splitlr | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_float</a> | splittb | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_color3</a> | splittb | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_color4</a> | splittb | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_vector2</a> | splittb | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_vector3</a> | splittb | procedural2d |
| <a href="../documents/definitions/splittb.html">ND_splittb_vector4</a> | splittb | procedural2d |
| <a href="../documents/definitions/spot_light.html">ND_spot_light</a> | spot_light | light |
| <a href="../documents/definitions/sqrt.html">ND_sqrt_float</a> | sqrt | math |
| <a href="../documents/definitions/sqrt.html">ND_sqrt_vector2</a> | sqrt | math |
| <a href="../documents/definitions/sqrt.html">ND_sqrt_vector3</a> | sqrt | math |
| <a href="../documents/definitions/sqrt.html">ND_sqrt_vector4</a> | sqrt | math |
| <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">ND_srgb_displayp3_to_lin_rec709_color3</a> | srgb_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">ND_srgb_displayp3_to_lin_rec709_color4</a> | srgb_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">ND_srgb_texture_to_lin_rec709_color3</a> | srgb_texture_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">ND_srgb_texture_to_lin_rec709_color4</a> | srgb_texture_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/standard_surface.html">ND_standard_surface_surfaceshader</a> | standard_surface | pbr |
| <a href="../documents/definitions/standard_surface.html">ND_standard_surface_surfaceshader_100</a> | standard_surface | pbr |
| <a href="../documents/definitions/standard_surface_to_UsdPreviewSurface.html">ND_standard_surface_to_UsdPreviewSurface</a> | standard_surface_to_UsdPreviewSurface | translation |
| <a href="../documents/definitions/standard_surface_to_gltf_pbr.html">ND_standard_surface_to_gltf_pbr</a> | standard_surface_to_gltf_pbr | translation |
| <a href="../documents/definitions/subsurface_bsdf.html">ND_subsurface_bsdf</a> | subsurface_bsdf | pbr |
| <a href="../documents/definitions/subtract.html">ND_subtract_float</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_color3</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_color4</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector2</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector3</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector4</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_matrix33</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_matrix44</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_color3FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_color4FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector2FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector3FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_vector4FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_matrix33FA</a> | subtract | math |
| <a href="../documents/definitions/subtract.html">ND_subtract_matrix44FA</a> | subtract | math |
| <a href="../documents/definitions/surface.html">ND_surface</a> | surface | pbr |
| <a href="../documents/definitions/surface_unlit.html">ND_surface_unlit</a> | surface_unlit | shader |
| <a href="../documents/definitions/surfacematerial.html">ND_surfacematerial</a> | surfacematerial | material |
| <a href="../documents/definitions/switch.html">ND_switch_float</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_color3</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_color4</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector2</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector3</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector4</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_floatI</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_color3I</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_color4I</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector2I</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector3I</a> | switch | conditional |
| <a href="../documents/definitions/switch.html">ND_switch_vector4I</a> | switch | conditional |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_float_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_float_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_float_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_float_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_float_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_float</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color3_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_float</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_color4_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_float</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector2_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_float</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector3_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_float</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_color3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_color4</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_vector2</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_vector3</a> | swizzle | channel |
| <a href="../documents/definitions/swizzle.html">ND_swizzle_vector4_vector4</a> | swizzle | channel |
| <a href="../documents/definitions/tan.html">ND_tan_float</a> | tan | math |
| <a href="../documents/definitions/tan.html">ND_tan_vector2</a> | tan | math |
| <a href="../documents/definitions/tan.html">ND_tan_vector3</a> | tan | math |
| <a href="../documents/definitions/tan.html">ND_tan_vector4</a> | tan | math |
| <a href="../documents/definitions/tangent.html">ND_tangent_vector3</a> | tangent | geometric |
| <a href="../documents/definitions/texcoord.html">ND_texcoord_vector2</a> | texcoord | geometric |
| <a href="../documents/definitions/texcoord.html">ND_texcoord_vector3</a> | texcoord | geometric |
| <a href="../documents/definitions/thin_film_bsdf.html">ND_thin_film_bsdf</a> | thin_film_bsdf | pbr |
| <a href="../documents/definitions/thin_surface.html">ND_thin_surface</a> | thin_surface | pbr |
| <a href="../documents/definitions/tiledcircles.html">ND_tiledcircles_color3</a> | tiledcircles | procedural2d |
| <a href="../documents/definitions/tiledcloverleafs.html">ND_tiledcloverleafs_color3</a> | tiledcloverleafs | procedural2d |
| <a href="../documents/definitions/tiledhexagons.html">ND_tiledhexagons_color3</a> | tiledhexagons | procedural2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_float</a> | tiledimage | texture2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_color3</a> | tiledimage | texture2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_color4</a> | tiledimage | texture2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_vector2</a> | tiledimage | texture2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_vector3</a> | tiledimage | texture2d |
| <a href="../documents/definitions/tiledimage.html">ND_tiledimage_vector4</a> | tiledimage | texture2d |
| <a href="../documents/definitions/time.html">ND_time_float</a> | time | application |
| <a href="../documents/definitions/transformmatrix.html">ND_transformmatrix_vector2M3</a> | transformmatrix | math |
| <a href="../documents/definitions/transformmatrix.html">ND_transformmatrix_vector3</a> | transformmatrix | math |
| <a href="../documents/definitions/transformmatrix.html">ND_transformmatrix_vector3M4</a> | transformmatrix | math |
| <a href="../documents/definitions/transformmatrix.html">ND_transformmatrix_vector4</a> | transformmatrix | math |
| <a href="../documents/definitions/transformnormal.html">ND_transformnormal_vector3</a> | transformnormal | math |
| <a href="../documents/definitions/transformpoint.html">ND_transformpoint_vector3</a> | transformpoint | math |
| <a href="../documents/definitions/transformvector.html">ND_transformvector_vector3</a> | transformvector | math |
| <a href="../documents/definitions/translucent_bsdf.html">ND_translucent_bsdf</a> | translucent_bsdf | pbr |
| <a href="../documents/definitions/transpose.html">ND_transpose_matrix33</a> | transpose | math |
| <a href="../documents/definitions/transpose.html">ND_transpose_matrix44</a> | transpose | math |
| <a href="../documents/definitions/trianglewave.html">ND_trianglewave_float</a> | trianglewave | math |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_float</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_color3</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_color4</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector2</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector3</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector4</a> | triplanarprojection | texture3d |
| <a href="../documents/definitions/unifiednoise2d.html">ND_unifiednoise2d_float</a> | unifiednoise2d | procedural2d |
| <a href="../documents/definitions/unifiednoise3d.html">ND_unifiednoise3d_float</a> | unifiednoise3d | procedural3d |
| <a href="../documents/definitions/uniform_edf.html">ND_uniform_edf</a> | uniform_edf | pbr |
| <a href="../documents/definitions/unpremult.html">ND_unpremult_color4</a> | unpremult | compositing |
| <a href="../documents/definitions/viewdirection.html">ND_viewdirection_vector3</a> | viewdirection | npr |
| <a href="../documents/definitions/volume.html">ND_volume</a> | volume | pbr |
| <a href="../documents/definitions/volumematerial.html">ND_volumematerial</a> | volumematerial | material |
| <a href="../documents/definitions/worleynoise2d.html">ND_worleynoise2d_float</a> | worleynoise2d | procedural2d |
| <a href="../documents/definitions/worleynoise2d.html">ND_worleynoise2d_vector2</a> | worleynoise2d | procedural2d |
| <a href="../documents/definitions/worleynoise2d.html">ND_worleynoise2d_vector3</a> | worleynoise2d | procedural2d |
| <a href="../documents/definitions/worleynoise3d.html">ND_worleynoise3d_float</a> | worleynoise3d | procedural3d |
| <a href="../documents/definitions/worleynoise3d.html">ND_worleynoise3d_vector2</a> | worleynoise3d | procedural3d |
| <a href="../documents/definitions/worleynoise3d.html">ND_worleynoise3d_vector3</a> | worleynoise3d | procedural3d |

</details>


We print out the category of these new definitions. It is possible that new categories of nodes have been added
in addition to adding new variants on existing categories.

As there is no formal list of such categories, a check needs to be made by scanning all definitions. 

In [88]:
oldCategories = set()
for nodeDef in otherNodeDefs:
    oldCategories.add(nodeDef.getNodeString())
# Sort old categories
oldCategories = sorted(oldCategories)

newCategories = {}
for nodeDef in currNodeDefs:
    newCategory = nodeDef.getNodeString()
    if newCategory not in oldCategories:
        newCategories[newCategory] = [ nodeDef.getNodeGroup(), nodeDef.getType() ]        

if newCategories:
    text = '| Node category | Node group | Type |\n'
    text = text + '| --- | --- | --- |\n' 
    sorted_keys = sorted(newCategories.keys())
    sortedNewCategories = {}
    for key in sorted_keys:
        sortedNewCategories[key] = newCategories[key]                                 
    for category in sortedNewCategories:
        text = text + '| [%s](../documents/definitions/%s.html) | %s | %s |\n' % (category, category, newCategories[category][0], newCategories[category][1])

    delta = '%d new node categories found' % len(newCategories)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

<details open><summary>197 new node categories found</summary>

| Node category | Node group | Type |
| --- | --- | --- |
| [LamaAdd](../documents/definitions/LamaAdd.html) | pbr | EDF |
| [LamaConductor](../documents/definitions/LamaConductor.html) | pbr | BSDF |
| [LamaDielectric](../documents/definitions/LamaDielectric.html) | pbr | BSDF |
| [LamaDiffuse](../documents/definitions/LamaDiffuse.html) | pbr | BSDF |
| [LamaEmission](../documents/definitions/LamaEmission.html) | pbr | EDF |
| [LamaLayer](../documents/definitions/LamaLayer.html) | pbr | BSDF |
| [LamaMix](../documents/definitions/LamaMix.html) | pbr | EDF |
| [LamaSSS](../documents/definitions/LamaSSS.html) | pbr | BSDF |
| [LamaSheen](../documents/definitions/LamaSheen.html) | pbr | BSDF |
| [LamaTranslucent](../documents/definitions/LamaTranslucent.html) | pbr | BSDF |
| [UsdPreviewSurface](../documents/definitions/UsdPreviewSurface.html) | pbr | surfaceshader |
| [UsdPrimvarReader](../documents/definitions/UsdPrimvarReader.html) | geometric | vector4 |
| [UsdTransform2d](../documents/definitions/UsdTransform2d.html) | math | vector2 |
| [UsdUVTexture](../documents/definitions/UsdUVTexture.html) | texture2d | multioutput |
| [absorption_vdf](../documents/definitions/absorption_vdf.html) | pbr | VDF |
| [absval](../documents/definitions/absval.html) | math | vector4 |
| [acescg_to_lin_rec709](../documents/definitions/acescg_to_lin_rec709.html) | colortransform | color4 |
| [acos](../documents/definitions/acos.html) | math | vector4 |
| [add](../documents/definitions/add.html) | math | matrix44 |
| [adobergb_to_lin_rec709](../documents/definitions/adobergb_to_lin_rec709.html) | colortransform | color4 |
| [ambientocclusion](../documents/definitions/ambientocclusion.html) | global | float |
| [anisotropic_vdf](../documents/definitions/anisotropic_vdf.html) | pbr | VDF |
| [arrayappend](../documents/definitions/arrayappend.html) | math | stringarray |
| [artistic_ior](../documents/definitions/artistic_ior.html) | pbr | multioutput |
| [asin](../documents/definitions/asin.html) | math | vector4 |
| [atan2](../documents/definitions/atan2.html) | math | vector4 |
| [bitangent](../documents/definitions/bitangent.html) | geometric | vector3 |
| [blackbody](../documents/definitions/blackbody.html) | pbr | color3 |
| [blur](../documents/definitions/blur.html) | convolution2d | vector4 |
| [bump](../documents/definitions/bump.html) | geometric | vector3 |
| [burley_diffuse_bsdf](../documents/definitions/burley_diffuse_bsdf.html) | pbr | BSDF |
| [burn](../documents/definitions/burn.html) | compositing | color4 |
| [ceil](../documents/definitions/ceil.html) | math | integer |
| [cellnoise2d](../documents/definitions/cellnoise2d.html) | procedural2d | float |
| [cellnoise3d](../documents/definitions/cellnoise3d.html) | procedural3d | float |
| [checkerboard](../documents/definitions/checkerboard.html) | procedural2d | color3 |
| [circle](../documents/definitions/circle.html) | procedural2d | float |
| [clamp](../documents/definitions/clamp.html) | math | vector4 |
| [cloverleaf](../documents/definitions/cloverleaf.html) | procedural2d | float |
| [colorcorrect](../documents/definitions/colorcorrect.html) | adjustment | color4 |
| [combine2](../documents/definitions/combine2.html) | channel | vector4 |
| [combine3](../documents/definitions/combine3.html) | channel | vector3 |
| [combine4](../documents/definitions/combine4.html) | channel | vector4 |
| [conductor_bsdf](../documents/definitions/conductor_bsdf.html) | pbr | BSDF |
| [conical_edf](../documents/definitions/conical_edf.html) | pbr | EDF |
| [constant](../documents/definitions/constant.html) | procedural | filename |
| [contrast](../documents/definitions/contrast.html) | adjustment | vector4 |
| [convert](../documents/definitions/convert.html) | shader | surfaceshader |
| [cos](../documents/definitions/cos.html) | math | vector4 |
| [creatematrix](../documents/definitions/creatematrix.html) | math | matrix44 |
| [crosshatch](../documents/definitions/crosshatch.html) | procedural2d | color3 |
| [crossproduct](../documents/definitions/crossproduct.html) | math | vector3 |
| [curveadjust](../documents/definitions/curveadjust.html) | adjustment | vector4 |
| [determinant](../documents/definitions/determinant.html) | math | float |
| [dielectric_bsdf](../documents/definitions/dielectric_bsdf.html) | pbr | BSDF |
| [difference](../documents/definitions/difference.html) | compositing | color4 |
| [directional_light](../documents/definitions/directional_light.html) | light | lightshader |
| [disjointover](../documents/definitions/disjointover.html) | compositing | color4 |
| [disney_brdf_2012](../documents/definitions/disney_brdf_2012.html) | pbr | surfaceshader |
| [disney_bsdf_2015](../documents/definitions/disney_bsdf_2015.html) | pbr | surfaceshader |
| [displacement](../documents/definitions/displacement.html) | pbr | displacementshader |
| [distance](../documents/definitions/distance.html) | math | float |
| [divide](../documents/definitions/divide.html) | math | vector4 |
| [dodge](../documents/definitions/dodge.html) | compositing | color4 |
| [dot](../documents/definitions/dot.html) | organization | lightshader |
| [dotproduct](../documents/definitions/dotproduct.html) | math | float |
| [exp](../documents/definitions/exp.html) | math | vector4 |
| [extract](../documents/definitions/extract.html) | channel | float |
| [facingratio](../documents/definitions/facingratio.html) | npr | float |
| [floor](../documents/definitions/floor.html) | math | integer |
| [fractal3d](../documents/definitions/fractal3d.html) | procedural3d | vector4 |
| [frame](../documents/definitions/frame.html) | application | float |
| [g18_rec709_to_lin_rec709](../documents/definitions/g18_rec709_to_lin_rec709.html) | colortransform | color4 |
| [g22_ap1_to_lin_rec709](../documents/definitions/g22_ap1_to_lin_rec709.html) | colortransform | color4 |
| [g22_rec709_to_lin_rec709](../documents/definitions/g22_rec709_to_lin_rec709.html) | colortransform | color4 |
| [generalized_schlick_bsdf](../documents/definitions/generalized_schlick_bsdf.html) | pbr | BSDF |
| [generalized_schlick_edf](../documents/definitions/generalized_schlick_edf.html) | pbr | EDF |
| [geomcolor](../documents/definitions/geomcolor.html) | geometric | color4 |
| [geompropvalue](../documents/definitions/geompropvalue.html) | geometric | vector4 |
| [glossiness_anisotropy](../documents/definitions/glossiness_anisotropy.html) | pbr | vector2 |
| [gltf_colorimage](../documents/definitions/gltf_colorimage.html) | texture2d | multioutput |
| [gltf_image](../documents/definitions/gltf_image.html) | texture2d | vector3 |
| [gltf_iridescence_thickness](../documents/definitions/gltf_iridescence_thickness.html) | texture2d | float |
| [gltf_normalmap](../documents/definitions/gltf_normalmap.html) | texture2d | vector3 |
| [gltf_pbr](../documents/definitions/gltf_pbr.html) | pbr | surfaceshader |
| [grid](../documents/definitions/grid.html) | procedural2d | color3 |
| [heighttonormal](../documents/definitions/heighttonormal.html) | convolution2d | vector3 |
| [hexagon](../documents/definitions/hexagon.html) | procedural2d | float |
| [hsvadjust](../documents/definitions/hsvadjust.html) | adjustment | color4 |
| [hsvtorgb](../documents/definitions/hsvtorgb.html) | adjustment | color4 |
| [ifequal](../documents/definitions/ifequal.html) | conditional | vector4 |
| [ifgreater](../documents/definitions/ifgreater.html) | conditional | vector4 |
| [ifgreatereq](../documents/definitions/ifgreatereq.html) | conditional | vector4 |
| [image](../documents/definitions/image.html) | texture2d | vector4 |
| [in](../documents/definitions/in.html) | compositing | color4 |
| [inside](../documents/definitions/inside.html) | compositing | color4 |
| [invert](../documents/definitions/invert.html) | math | vector4 |
| [invertmatrix](../documents/definitions/invertmatrix.html) | math | matrix44 |
| [layer](../documents/definitions/layer.html) | pbr | BSDF |
| [light](../documents/definitions/light.html) | pbr | lightshader |
| [lin_adobergb_to_lin_rec709](../documents/definitions/lin_adobergb_to_lin_rec709.html) | colortransform | color4 |
| [lin_displayp3_to_lin_rec709](../documents/definitions/lin_displayp3_to_lin_rec709.html) | colortransform | color4 |
| [line](../documents/definitions/line.html) | procedural2d | float |
| [ln](../documents/definitions/ln.html) | math | vector4 |
| [luminance](../documents/definitions/luminance.html) | adjustment | color4 |
| [magnitude](../documents/definitions/magnitude.html) | math | float |
| [mask](../documents/definitions/mask.html) | compositing | color4 |
| [matte](../documents/definitions/matte.html) | compositing | color4 |
| [max](../documents/definitions/max.html) | math | vector4 |
| [measured_edf](../documents/definitions/measured_edf.html) | pbr | EDF |
| [min](../documents/definitions/min.html) | math | vector4 |
| [minus](../documents/definitions/minus.html) | compositing | color4 |
| [mix](../documents/definitions/mix.html) | compositing | volumeshader |
| [modulo](../documents/definitions/modulo.html) | math | vector4 |
| [multiply](../documents/definitions/multiply.html) | math | vector4 |
| [noise2d](../documents/definitions/noise2d.html) | procedural2d | vector4 |
| [noise3d](../documents/definitions/noise3d.html) | procedural3d | vector4 |
| [normal](../documents/definitions/normal.html) | geometric | vector3 |
| [normalize](../documents/definitions/normalize.html) | math | vector4 |
| [normalmap](../documents/definitions/normalmap.html) | math | vector3 |
| [open_pbr_surface](../documents/definitions/open_pbr_surface.html) | pbr | surfaceshader |
| [oren_nayar_diffuse_bsdf](../documents/definitions/oren_nayar_diffuse_bsdf.html) | pbr | BSDF |
| [out](../documents/definitions/out.html) | compositing | color4 |
| [outside](../documents/definitions/outside.html) | compositing | color4 |
| [over](../documents/definitions/over.html) | compositing | color4 |
| [overlay](../documents/definitions/overlay.html) | compositing | color4 |
| [place2d](../documents/definitions/place2d.html) | math | vector2 |
| [plus](../documents/definitions/plus.html) | compositing | color4 |
| [point_light](../documents/definitions/point_light.html) | light | lightshader |
| [position](../documents/definitions/position.html) | geometric | vector3 |
| [power](../documents/definitions/power.html) | math | vector4 |
| [premult](../documents/definitions/premult.html) | compositing | color4 |
| [ramp4](../documents/definitions/ramp4.html) | procedural2d | vector4 |
| [ramplr](../documents/definitions/ramplr.html) | procedural2d | vector4 |
| [ramptb](../documents/definitions/ramptb.html) | procedural2d | vector4 |
| [randomcolor](../documents/definitions/randomcolor.html) | procedural3d | color3 |
| [randomfloat](../documents/definitions/randomfloat.html) | procedural | float |
| [range](../documents/definitions/range.html) | adjustment | vector4 |
| [rec709_display_to_lin_rec709](../documents/definitions/rec709_display_to_lin_rec709.html) | colortransform | color4 |
| [remap](../documents/definitions/remap.html) | adjustment | vector4 |
| [rgbtohsv](../documents/definitions/rgbtohsv.html) | adjustment | color4 |
| [rotate2d](../documents/definitions/rotate2d.html) | math | vector2 |
| [rotate3d](../documents/definitions/rotate3d.html) | math | vector3 |
| [roughness_anisotropy](../documents/definitions/roughness_anisotropy.html) | pbr | vector2 |
| [roughness_dual](../documents/definitions/roughness_dual.html) | pbr | vector2 |
| [saturate](../documents/definitions/saturate.html) | adjustment | color4 |
| [screen](../documents/definitions/screen.html) | compositing | color4 |
| [separate2](../documents/definitions/separate2.html) | channel | multioutput |
| [separate3](../documents/definitions/separate3.html) | channel | multioutput |
| [separate4](../documents/definitions/separate4.html) | channel | multioutput |
| [sheen_bsdf](../documents/definitions/sheen_bsdf.html) | pbr | BSDF |
| [sign](../documents/definitions/sign.html) | math | vector4 |
| [sin](../documents/definitions/sin.html) | math | vector4 |
| [smoothstep](../documents/definitions/smoothstep.html) | adjustment | vector4 |
| [splitlr](../documents/definitions/splitlr.html) | procedural2d | vector4 |
| [splittb](../documents/definitions/splittb.html) | procedural2d | vector4 |
| [spot_light](../documents/definitions/spot_light.html) | light | lightshader |
| [sqrt](../documents/definitions/sqrt.html) | math | vector4 |
| [srgb_displayp3_to_lin_rec709](../documents/definitions/srgb_displayp3_to_lin_rec709.html) | colortransform | color4 |
| [srgb_texture_to_lin_rec709](../documents/definitions/srgb_texture_to_lin_rec709.html) | colortransform | color4 |
| [standard_surface](../documents/definitions/standard_surface.html) | pbr | surfaceshader |
| [standard_surface_to_UsdPreviewSurface](../documents/definitions/standard_surface_to_UsdPreviewSurface.html) | translation | multioutput |
| [standard_surface_to_gltf_pbr](../documents/definitions/standard_surface_to_gltf_pbr.html) | translation | multioutput |
| [subsurface_bsdf](../documents/definitions/subsurface_bsdf.html) | pbr | BSDF |
| [subtract](../documents/definitions/subtract.html) | math | matrix44 |
| [surface](../documents/definitions/surface.html) | pbr | surfaceshader |
| [surface_unlit](../documents/definitions/surface_unlit.html) | shader | surfaceshader |
| [surfacematerial](../documents/definitions/surfacematerial.html) | material | material |
| [switch](../documents/definitions/switch.html) | conditional | vector4 |
| [swizzle](../documents/definitions/swizzle.html) | channel | vector4 |
| [tan](../documents/definitions/tan.html) | math | vector4 |
| [tangent](../documents/definitions/tangent.html) | geometric | vector3 |
| [texcoord](../documents/definitions/texcoord.html) | geometric | vector3 |
| [thin_film_bsdf](../documents/definitions/thin_film_bsdf.html) | pbr | BSDF |
| [thin_surface](../documents/definitions/thin_surface.html) | pbr | surfaceshader |
| [tiledcircles](../documents/definitions/tiledcircles.html) | procedural2d | color3 |
| [tiledcloverleafs](../documents/definitions/tiledcloverleafs.html) | procedural2d | color3 |
| [tiledhexagons](../documents/definitions/tiledhexagons.html) | procedural2d | color3 |
| [tiledimage](../documents/definitions/tiledimage.html) | texture2d | vector4 |
| [time](../documents/definitions/time.html) | application | float |
| [transformmatrix](../documents/definitions/transformmatrix.html) | math | vector4 |
| [transformnormal](../documents/definitions/transformnormal.html) | math | vector3 |
| [transformpoint](../documents/definitions/transformpoint.html) | math | vector3 |
| [transformvector](../documents/definitions/transformvector.html) | math | vector3 |
| [translucent_bsdf](../documents/definitions/translucent_bsdf.html) | pbr | BSDF |
| [transpose](../documents/definitions/transpose.html) | math | matrix44 |
| [trianglewave](../documents/definitions/trianglewave.html) | math | float |
| [triplanarprojection](../documents/definitions/triplanarprojection.html) | texture3d | vector4 |
| [unifiednoise2d](../documents/definitions/unifiednoise2d.html) | procedural2d | float |
| [unifiednoise3d](../documents/definitions/unifiednoise3d.html) | procedural3d | float |
| [uniform_edf](../documents/definitions/uniform_edf.html) | pbr | EDF |
| [unpremult](../documents/definitions/unpremult.html) | compositing | color4 |
| [viewdirection](../documents/definitions/viewdirection.html) | npr | vector3 |
| [volume](../documents/definitions/volume.html) | pbr | volumeshader |
| [volumematerial](../documents/definitions/volumematerial.html) | material | material |
| [worleynoise2d](../documents/definitions/worleynoise2d.html) | procedural2d | vector3 |
| [worleynoise3d](../documents/definitions/worleynoise3d.html) | procedural3d | vector3 |

</details>


Additional checks can be added for non definition elements such as geometry properties, and type definitions.
For this notebook we will concentrate on definitions only.

### Detailed Element Comparison

A comparison between 2 Elements in MaterialX is available the the equivalence operator `==` on an Element.
Though this will indicate if they differ, it provides no information as to what has differed.

To accomplish this a new utility called `compareElements` has been written. In addition to providing explicit feedback on what has changed it will explicitly recurse to children by calling itself (instead of recursing using the `==` operator)

The basic properities tested are:
1. Element category
2. Element attribute names
3. Elememt attribute values
4. Child Element count and what children were added or removed.
5. Recursing on child Elements. This would generally be Inputs and Outputs in the case of defintions.
   If comparing graph elements (GraphElement), children could be child Nodes or graphs (GraphElements). 

> Note that this is the exact same logic as the `==` operator at time of writing. It can be extended as needed but the basic logic for checking attrbutes and then recursing on child Elements should remain the same. 

A few additional utilities are added to support comparison logic:

1. `getActiveChild()` and `getActiveChildren()` is added to support class inheritance. If this is not
considered then the children compared will only be the ones that are explicitly specified for a given class.
> Note that this would be a useful API to have and would complement the existing `getActiveInputs()`and `getActiveOutputs()` methods on an <a href="https://materialx.org/docs/api/class_interface_element.html" target="_blank">InterfaceElement</a>. It is mostly useful for comparison of implementations and definitions where the interface is being
compared. It is less useful for a node instance since comparison logic is aimed at what is explicitly specified on
instances.

2. `getActiveAttribute()` and `getActiveAttributes()` is added to support class inheritance when querying for attributes.
This serves the same purpose as the child methods to ensure inherited attributes are not missed.

3. `diffLists()` is added to find the intersection and difference between two lists.

Inheritance functions used the <a href="https://materialx.org/docs/api/class_element.html" target="_blank">
traverseInheritance()</a> API to traverse up the inheritance hierarchy.

In [89]:
def getActiveChild(element, name):
    '''
    Check if a child with a given name exists in the inheritance hierarchy for a given Element.
    '''
    for elem in element.traverseInheritance():
        child = elem.getChild(name)
        if child:
            return child
    return None

def getActiveChildren(element):
    '''
    Get all children (included inherited ones) for a given Element.
    '''
    children = []
    childset = set()
    for elem in element.traverseInheritance():
        for child in elem.getChildren():
            if child.getName() not in childset:
                children.append(child)
                childset.add(child.getName())
    return children    

def getActiveAttribute(element, name):
    ''' 
    Check if an attribute with a given name exists in the inheritance hierarchy for a given Element.
    '''
    for elem in element.traverseInheritance():
        child = elem.getAttribute(name)
        if child:
            return child
    return None

def getActiveAttributes(element):
    '''
    Get all attributes (included inherited ones) for a given Element.
    '''
    children = []
    childset = set()
    for elem in element.traverseInheritance():
        for child in elem.getAttributeNames():
            if child not in childset:
                children.append(child)
                childset.add(child)
    return children

def diffLists(list1, list2):
    '''
    Return items in either list1 or list2, and in both.
    '''
    if list1 != list2:
        # Find items in list1 not in list2
        onlyList1 = [item for item in list1 if item not in list2]
        # Find items in list2 not in list1
        onlyList2 = [item for item in list2 if item not in list1]
        # Find items in both lists
        common = [item for item in list1 if item in list2]
        return onlyList1, onlyList2, common
    return [], [], list1     

Using these utility functions, the logic for `compareElements()` is as follows:
1. Check category between elements
2. Compare attributes

In [90]:
def compareElements(nd1, nd2, indent, log):
    '''
    Compare two elements and print any differences.

    Tests performed:
    1. Element category
    2. Element attribute names
    3. Elememt attribute values
    4. Child Element count and what children are present in one but not the other.
    5. Recursing on child Elements.
    '''
    if not nd1 or not nd2:
        return True

    differs = False

    if nd1.getCategory() != nd2.getCategory():
        log.append(indent + '- Category changed on: `%s` from `%s` to `%s`' % (nd1.getName(), nd2.getCategory(), nd1.getCategory()))
        differs = True

    # Compare attributes.
    # Note: This does not work properly with inheritance yet.
    onlynd1, onlynd2, common = diffLists(getActiveAttributes(nd1), getActiveAttributes(nd2))
    if len(onlynd1) > 0 or len(onlynd2) > 0:
        for item in onlynd1:
            log.append(indent + '- attribute: `%s`=( `%s` ) removed from: `%s`' % 
                       ( item, getActiveAttribute(nd1, item), nd1.getName()))
        for item in onlynd2:
            log.append(indent + '- attribute: `%s`=( `%s` ) added to: `%s`' % 
                       ( item, getActiveAttribute(nd2, item), nd2.getName()))
        differs = True

    for attr in common:
        if nd2.getAttribute(attr) != nd1.getAttribute(attr):
            log.append(indent + '  - attribute: `%s` changed on `%s` from `( %s )` to `( %s )`' % 
                       (attr, nd1.getName(), nd1.getAttribute(attr), nd2.getAttribute(attr)))
            differs = True

    # Compare children. This also checks for children that inherited 
    c1 = [c.getName() for c in getActiveChildren(nd1)] 
    c2 = [c.getName() for c in getActiveChildren(nd2)]
    if len(c1) != len(c2):
        log.append(indent + '- Number of children on: `%s` changed from ( %s ) to ( %s )' % (nd1.getName(), len(c1), len(c2)))
        differs = True

    # Get list of added, removed, and common children.
    onlynd1, onlynd2, common = diffLists(c1, c2)

    # Log children that are removed. Emit value if is a value element.
    for childName in onlynd1:
        child = nd1.getChild(childName)
        childName = child.getNamePath().removeprefix(child.getParent().getNamePath())
        childType = child.getCategory()
        if child.isA(mx.ValueElement):
            log.append(indent + '- %s `%s` removed from `%s`. Value=( %s )' % (childType, childName, nd1.getNamePath(), child.getValueString()))
        else:
            log.append(indent + '- %s `%s` removed from `%s`' % (childType, childName, nd1.getNamePath()))
        differs = True

    # Log child that are added. Emit value if is a value element.
    for childName in onlynd2:
        child = nd2.getChild(childName)
        childName = child.getNamePath().removeprefix(child.getParent().getNamePath())
        childType = child.getCategory()
        if child.isA(mx.ValueElement):
            log.append(indent + '- %s `%s` added to `%s`. Value=( %s )' % (childType, childName, nd2.getNamePath(), child.getValueString()))
        else:
            log.append(indent + '- %s `%s` add to `%s`' % (childType, childName, nd2.getNamePath()))
        differs = True

    # Recurse on common children.
    for childName in common:
        c1child = getActiveChild(nd1, childName)
        c2child = getActiveChild(nd2, childName)
        if compareElements(c1child, c2child, indent, log):
            differs = True

    return differs

> Note that unlike the C++ implementation children are not compared by index as the ordering may have changed between versions, thus resulting in false mismatches appearing. 

> **It may be useful to update the C++ implementation to be independent of ordering.**

Given this utility, we scan through all the definitions which exist in both libraries.

Some items of interest include changes in default value for inputs on some nodes, as well as addition of uniform qualifiers to some nodes.

In [91]:
#text = '### Compare old and current definitions in detail\n'
text = '| Name | Change  |\n'
text = text + '| --- | --- |\n'

changed = 0
for item in compareDetails:
    nd1 = item[0]
    nd2 = item[1]
    # Use the built in comparitor first
    difference = (nd1 != nd2)
    if difference:
        # Perform details comparison
        text = text + '| <a href="../documents/definitions/%s.html">%s</a> | ' % (nd1.getNodeString(), nd1.getName())
        log = []
        difference = compareElements(nd1, nd2, '', log) 
        if difference:
            for line in log:
                text = text + line + '<br>'
            text = text + '|\n'
            changed = changed + 1

delta = '%d definitions modified' % changed
text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
display_markdown(text, raw=True)


<details open><summary>0 definitions modified</summary>

| Name | Change  |
| --- | --- |

</details>


## Target and Implementation Comparison

Next we cover how add logic to see if implementations have been added or removed, or
if existing implementations have changed. This includes checking in the context of whether new `target`s have
been added or not.

### Target Check

The list of targets can be queried using `getTargetDefs()` on each librari respectively and the
compared.

In [92]:
## Get old targets
oldTargets = []
for target in otherLibrary.getTargetDefs():
    oldTargets.append(target.getName())
print('Old targets:', oldTargets)

## Check for new targets
currTargets = []
for target in currLibrary.getTargetDefs():
    currTargets.append(target.getName())
print('Current targets:', currTargets)

newTargets = list(set(currTargets) - set(oldTargets))
if newTargets:
    print('New targets:', newTargets)

Old targets: []
Current targets: ['essl', 'genglsl', 'genmdl', 'genmsl', 'genosl']
New targets: ['genmdl', 'genmsl', 'essl', 'genosl', 'genglsl']


### Implementation Check

As with definitions, implementations can be compared, either in the context of their associated definition or independently. As an implementation is generally of no use unless it is used in a definition we will present the comparison logic using the latter context.

#### Listing Implementations

It can be useful to examine the available implementations based on what, if any, code generation `target``
they are associated with. 
* If a implementation has no target then it is generally a node graph implementation but could also be generated programmatically via a common code generator which has no specific code target. For example the <a href ="https://materialx.org/docs/api/class_shader_generator.html" target="_blank">ShaderGenerator</a> class implements non-target specific code generation for <a href="https://github.com/AcademySoftwareFoundation/MaterialX/tree/main/source/MaterialXGenShader/Nodes" target="_blank">certain node types</a>.
* Different targets may choose to implement logic differently than others. For example one may use a nodegraph, while another may use shader code, or generate the code programmatically.
* Some targets may not support implementations for certain nodes.

A utility called `get_implementations()` scans for all implementations for a given `target``.

In [93]:
def get_implementations(nodedefs, target, allimpls):
    '''
    Build a dictionary of nodedefs and their corresponding implementations.
    
    The dictionary is keyed by the nodedef name and the value is a list of the implementation and 
    source file name.
    '''
    implementations = {}
    for nodedef in nodedefs:
        impl = nodedef.getImplementation(target)
        # This may be a nodegraph or an implementation
        if impl:
            # Skip if already used
            if impl.getName() in allimpls:
                continue
            # Skip if it's a nodegraph and target is not empty
            if len(target) > 0 and impl.isA(mx.NodeGraph):
                continue

            if target != 'essl':
                allimpls.add(impl.getName())
            implementations[nodedef.getName()] = [impl, mx.FilePath(impl.getSourceUri()).getBaseName()]
    return implementations

This utility can then be used to scan through all `target`s for each library.

In [94]:
def print_implementations(nodedefs, targets, allimpls, title):
    '''
    Display all the implementations for the given nodedefs and targets.
    '''
    display_markdown(title, raw=True)
    for target in targets:
        impls = get_implementations(nodedefs, target, allimpls)

        if len(target) == 0:
            result = '| Definition | NodeGraph | File |\n'
        else:
            result = '| Definition | Implementation | File |\n'
        result = result + '| --- | --- | --- |\n'
        for impl in impls:
            result = result + '|' + impl + '|' + impls[impl][0].getName() + '|' + impls[impl][1] + '\n'
        text = '<details><summary>' + str(len(impls)) + (' shaders: ' + target if target else ' node graphs') + '</summary>\n\n' + result+ '\n</details>\n' 
        display_markdown(text, raw=True)

# Scan for all the current targets
title = '**Current Library Definitions / Implementations**\n'
alltargets = currTargets
alltargets.append('')
allimpls = set()
print_implementations(currNodeDefs, alltargets, allimpls, title)

# Scan for all the previous targets
title = '**Previous Library Definitions / Implementations**\n'
alltargets = oldTargets
alltargets.append('')
allimpls = set()
print_implementations(otherNodeDefs, alltargets, allimpls, title)

**Current Library Definitions / Implementations**


<details><summary>534 shaders: essl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genglsl|lights_genglsl_impl.mtlx
|ND_directional_light|IM_directional_light_genglsl|lights_genglsl_impl.mtlx
|ND_spot_light|IM_spot_light_genglsl|lights_genglsl_impl.mtlx
|ND_viewdirection_vector3|IM_viewdirection_vector3_genglsl|nprlib_genglsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surface|IM_surface_genglsl|pbrlib_genglsl_impl.mtlx
|ND_light|IM_light_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_edf|IM_add_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genglsl|pbrlib_genglsl_impl.mtlx
|ND_blackbody|IM_blackbody_genglsl|pbrlib_genglsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genglsl|stdlib_genglsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_float|IM_image_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color3|IM_image_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color4|IM_image_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_float|IM_constant_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_string|IM_constant_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_frame_float|IM_frame_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_time_float|IM_time_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_float|IM_add_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3|IM_add_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4|IM_add_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_float|IM_divide_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_float|IM_invert_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_float|IM_absval_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_float|IM_floor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_float|IM_power_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3|IM_power_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4|IM_power_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_float|IM_sin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_float|IM_cos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_float|IM_tan_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_float|IM_asin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_float|IM_acos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_float|IM_ln_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_float|IM_exp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_float|IM_sign_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_float|IM_min_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3|IM_min_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4|IM_min_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_float|IM_max_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3|IM_max_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4|IM_max_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap|IM_normalmap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap_vector2|IM_normalmap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_float|IM_remap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_float|IM_plus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_float|IM_minus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_float|IM_difference_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_float|IM_burn_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_float|IM_screen_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_in_color4|IM_in_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_out_color4|IM_out_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_over_color4|IM_over_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_float|IM_inside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_float|IM_outside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_float|IM_mix_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_float|IM_switch_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector3_matrix33|IM_creatematrix_vector3_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector3_matrix44|IM_creatematrix_vector3_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector4_matrix44|IM_creatematrix_vector4_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_float|IM_blur_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_float|IM_dot_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_string|IM_dot_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genglsl|stdlib_genglsl_impl.mtlx

</details>


<details><summary>534 shaders: genglsl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genglsl|lights_genglsl_impl.mtlx
|ND_directional_light|IM_directional_light_genglsl|lights_genglsl_impl.mtlx
|ND_spot_light|IM_spot_light_genglsl|lights_genglsl_impl.mtlx
|ND_viewdirection_vector3|IM_viewdirection_vector3_genglsl|nprlib_genglsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surface|IM_surface_genglsl|pbrlib_genglsl_impl.mtlx
|ND_light|IM_light_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_edf|IM_add_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genglsl|pbrlib_genglsl_impl.mtlx
|ND_blackbody|IM_blackbody_genglsl|pbrlib_genglsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genglsl|stdlib_genglsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_float|IM_image_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color3|IM_image_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color4|IM_image_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_float|IM_constant_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_string|IM_constant_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_frame_float|IM_frame_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_time_float|IM_time_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_float|IM_add_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3|IM_add_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4|IM_add_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_float|IM_divide_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_float|IM_invert_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_float|IM_absval_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_float|IM_floor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_float|IM_power_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3|IM_power_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4|IM_power_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_float|IM_sin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_float|IM_cos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_float|IM_tan_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_float|IM_asin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_float|IM_acos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_float|IM_ln_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_float|IM_exp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_float|IM_sign_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_float|IM_min_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3|IM_min_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4|IM_min_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_float|IM_max_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3|IM_max_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4|IM_max_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap|IM_normalmap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap_vector2|IM_normalmap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_float|IM_remap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_float|IM_plus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_float|IM_minus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_float|IM_difference_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_float|IM_burn_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_float|IM_screen_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_in_color4|IM_in_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_out_color4|IM_out_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_over_color4|IM_over_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_float|IM_inside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_float|IM_outside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_float|IM_mix_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_float|IM_switch_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector3_matrix33|IM_creatematrix_vector3_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector3_matrix44|IM_creatematrix_vector3_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_creatematrix_vector4_matrix44|IM_creatematrix_vector4_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_float|IM_blur_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_float|IM_dot_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_string|IM_dot_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genglsl|stdlib_genglsl_impl.mtlx

</details>


<details><summary>543 shaders: genmdl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_viewdirection_vector3|IM_viewdirection_vector3_genmdl|nprlib_genmdl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conical_edf|IM_conical_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_measured_edf|IM_measured_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_absorption_vdf|IM_absorption_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_surface|IM_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_surface|IM_thin_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_volume|IM_volume_genmdl|pbrlib_genmdl_impl.mtlx
|ND_light|IM_light_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genmdl|pbrlib_genmdl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_vdf|IM_mix_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_edf|IM_add_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_vdf|IM_add_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_vdfC|IM_multiply_vdfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_vdfF|IM_multiply_vdfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genmdl|pbrlib_genmdl_impl.mtlx
|ND_blackbody|IM_blackbody_genmdl|pbrlib_genmdl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genmdl|pbrlib_genmdl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genmdl|stdlib_genmdl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_float|IM_image_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color3|IM_image_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color4|IM_image_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_float|IM_constant_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_string|IM_constant_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ambientocclusion_float|IM_ambientocclusion_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_frame_float|IM_frame_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_time_float|IM_time_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_float|IM_add_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3|IM_add_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4|IM_add_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_float|IM_divide_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_float|IM_invert_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_float|IM_absval_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_float|IM_floor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_float|IM_power_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3|IM_power_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4|IM_power_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_float|IM_sin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_float|IM_cos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_float|IM_tan_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_float|IM_asin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_float|IM_acos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_float|IM_ln_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_float|IM_exp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_float|IM_sign_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_float|IM_min_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3|IM_min_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4|IM_min_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_float|IM_max_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3|IM_max_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4|IM_max_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalmap|IM_normalmap_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalmap_vector2|IM_normalmap_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_float|IM_remap_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_float|IM_plus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_float|IM_minus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_float|IM_difference_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_float|IM_burn_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_float|IM_screen_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_in_color4|IM_in_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_out_color4|IM_out_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_over_color4|IM_over_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_float|IM_inside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_float|IM_outside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_float|IM_mix_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_displacementshader|IM_mix_displacementshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_volumeshader|IM_mix_volumeshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_float|IM_switch_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_creatematrix_vector3_matrix33|IM_creatematrix_vector3_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_creatematrix_vector3_matrix44|IM_creatematrix_vector3_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_creatematrix_vector4_matrix44|IM_creatematrix_vector4_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_float|IM_blur_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_float|IM_dot_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_string|IM_dot_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genmdl|stdlib_genmdl_impl.mtlx

</details>


<details><summary>533 shaders: genmsl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genmsl|lights_genmsl_impl.mtlx
|ND_directional_light|IM_directional_light_genmsl|lights_genmsl_impl.mtlx
|ND_spot_light|IM_spot_light_genmsl|lights_genmsl_impl.mtlx
|ND_viewdirection_vector3|IM_viewdirection_vector3_genmsl|nprlib_genmsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_surface|IM_surface_genmsl|pbrlib_genmsl_impl.mtlx
|ND_light|IM_light_genmsl|pbrlib_genmsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genmsl|pbrlib_genmsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genmsl|pbrlib_genmsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_add_edf|IM_add_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genmsl|pbrlib_genmsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genmsl|pbrlib_genmsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genmsl|pbrlib_genmsl_impl.mtlx
|ND_blackbody|IM_blackbody_genmsl|pbrlib_genmsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genmsl|pbrlib_genmsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genmsl|stdlib_genmsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_float|IM_image_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_color3|IM_image_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_color4|IM_image_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_float|IM_constant_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_string|IM_constant_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_frame_float|IM_frame_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_time_float|IM_time_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_float|IM_add_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color3|IM_add_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color4|IM_add_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_float|IM_divide_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_float|IM_invert_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_float|IM_absval_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_float|IM_floor_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_float|IM_power_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color3|IM_power_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color4|IM_power_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_float|IM_sin_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_float|IM_cos_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_float|IM_tan_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_float|IM_asin_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_float|IM_acos_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_float|IM_ln_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_float|IM_exp_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_float|IM_sign_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_float|IM_min_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color3|IM_min_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color4|IM_min_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_float|IM_max_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color3|IM_max_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color4|IM_max_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalmap|IM_normalmap_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalmap_vector2|IM_normalmap_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_float|IM_remap_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_float|IM_plus_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_float|IM_minus_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_float|IM_difference_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_float|IM_burn_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_float|IM_screen_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_in_color4|IM_in_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_out_color4|IM_out_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_over_color4|IM_over_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_float|IM_inside_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_float|IM_outside_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_float|IM_mix_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_float|IM_switch_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_creatematrix_vector3_matrix33|IM_creatematrix_vector3_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_creatematrix_vector3_matrix44|IM_creatematrix_vector3_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_creatematrix_vector4_matrix44|IM_creatematrix_vector4_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_float|IM_blur_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_float|IM_dot_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_string|IM_dot_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genmsl|stdlib_genmsl_impl.mtlx

</details>


<details><summary>531 shaders: genosl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_viewdirection_vector3|IM_viewdirection_vector3_genosl|nprlib_genosl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genosl|pbrlib_genosl_impl.mtlx
|ND_surface|IM_surface_genosl|pbrlib_genosl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genosl|pbrlib_genosl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genosl|pbrlib_genosl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genosl|pbrlib_genosl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_add_edf|IM_add_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genosl|pbrlib_genosl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genosl|pbrlib_genosl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genosl|pbrlib_genosl_impl.mtlx
|ND_blackbody|IM_blackbody_genosl|pbrlib_genosl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genosl|pbrlib_genosl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genosl|stdlib_genosl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genosl|stdlib_genosl_impl.mtlx
|ND_image_float|IM_image_float_genosl|stdlib_genosl_impl.mtlx
|ND_image_color3|IM_image_color3_genosl|stdlib_genosl_impl.mtlx
|ND_image_color4|IM_image_color4_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_float|IM_constant_float_genosl|stdlib_genosl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genosl|stdlib_genosl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genosl|stdlib_genosl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_constant_string|IM_constant_string_genosl|stdlib_genosl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genosl|stdlib_genosl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ambientocclusion_float|IM_ambientocclusion_float_genosl|stdlib_genosl_impl.mtlx
|ND_frame_float|IM_frame_float_genosl|stdlib_genosl_impl.mtlx
|ND_time_float|IM_time_float_genosl|stdlib_genosl_impl.mtlx
|ND_add_float|IM_add_float_genosl|stdlib_genosl_impl.mtlx
|ND_add_color3|IM_add_color3_genosl|stdlib_genosl_impl.mtlx
|ND_add_color4|IM_add_color4_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_float|IM_divide_float_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_float|IM_invert_float_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_absval_float|IM_absval_float_genosl|stdlib_genosl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genosl|stdlib_genosl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_floor_float|IM_floor_float_genosl|stdlib_genosl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genosl|stdlib_genosl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genosl|stdlib_genosl_impl.mtlx
|ND_power_float|IM_power_float_genosl|stdlib_genosl_impl.mtlx
|ND_power_color3|IM_power_color3_genosl|stdlib_genosl_impl.mtlx
|ND_power_color4|IM_power_color4_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_sin_float|IM_sin_float_genosl|stdlib_genosl_impl.mtlx
|ND_cos_float|IM_cos_float_genosl|stdlib_genosl_impl.mtlx
|ND_tan_float|IM_tan_float_genosl|stdlib_genosl_impl.mtlx
|ND_asin_float|IM_asin_float_genosl|stdlib_genosl_impl.mtlx
|ND_acos_float|IM_acos_float_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genosl|stdlib_genosl_impl.mtlx
|ND_ln_float|IM_ln_float_genosl|stdlib_genosl_impl.mtlx
|ND_exp_float|IM_exp_float_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_sign_float|IM_sign_float_genosl|stdlib_genosl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genosl|stdlib_genosl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_float|IM_min_float_genosl|stdlib_genosl_impl.mtlx
|ND_min_color3|IM_min_color3_genosl|stdlib_genosl_impl.mtlx
|ND_min_color4|IM_min_color4_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_float|IM_max_float_genosl|stdlib_genosl_impl.mtlx
|ND_max_color3|IM_max_color3_genosl|stdlib_genosl_impl.mtlx
|ND_max_color4|IM_max_color4_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_normalmap|IM_normalmap_float_genosl|stdlib_genosl_impl.mtlx
|ND_normalmap_vector2|IM_normalmap_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_float|IM_remap_float_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genosl|stdlib_genosl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genosl|stdlib_genosl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genosl|stdlib_genosl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genosl|stdlib_genosl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genosl|stdlib_genosl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genosl|stdlib_genosl_impl.mtlx
|ND_plus_float|IM_plus_float_genosl|stdlib_genosl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genosl|stdlib_genosl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genosl|stdlib_genosl_impl.mtlx
|ND_minus_float|IM_minus_float_genosl|stdlib_genosl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genosl|stdlib_genosl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genosl|stdlib_genosl_impl.mtlx
|ND_difference_float|IM_difference_float_genosl|stdlib_genosl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genosl|stdlib_genosl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genosl|stdlib_genosl_impl.mtlx
|ND_burn_float|IM_burn_float_genosl|stdlib_genosl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genosl|stdlib_genosl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genosl|stdlib_genosl_impl.mtlx
|ND_screen_float|IM_screen_float_genosl|stdlib_genosl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genosl|stdlib_genosl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genosl|stdlib_genosl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genosl|stdlib_genosl_impl.mtlx
|ND_in_color4|IM_in_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genosl|stdlib_genosl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genosl|stdlib_genosl_impl.mtlx
|ND_out_color4|IM_out_color4_genosl|stdlib_genosl_impl.mtlx
|ND_over_color4|IM_over_color4_genosl|stdlib_genosl_impl.mtlx
|ND_inside_float|IM_inside_float_genosl|stdlib_genosl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genosl|stdlib_genosl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genosl|stdlib_genosl_impl.mtlx
|ND_outside_float|IM_outside_float_genosl|stdlib_genosl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genosl|stdlib_genosl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_float|IM_mix_float_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genosl|stdlib_genosl_impl.mtlx
|ND_switch_float|IM_switch_float_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genosl|stdlib_genosl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genosl|stdlib_genosl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_creatematrix_vector3_matrix33|IM_creatematrix_vector3_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_creatematrix_vector3_matrix44|IM_creatematrix_vector3_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_creatematrix_vector4_matrix44|IM_creatematrix_vector4_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_blur_float|IM_blur_float_genosl|stdlib_genosl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genosl|stdlib_genosl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_float|IM_dot_float_genosl|stdlib_genosl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genosl|stdlib_genosl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_dot_string|IM_dot_string_genosl|stdlib_genosl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genosl|stdlib_genosl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genosl|stdlib_genosl_impl.mtlx

</details>


<details><summary>176 node graphs</summary>

| Definition | NodeGraph | File |
| --- | --- | --- |
|ND_disney_brdf_2012_surface|IM_disney_brdf_2012_surface_brdf_explorer|disney_brdf_2012.mtlx
|ND_disney_bsdf_2015_surface|IM_disney_bsdf_2015_surface_pbrt|disney_brdf_2015.mtlx
|ND_gltf_pbr_surfaceshader|IMPL_gltf_pbr_surfaceshader|gltf_pbr.mtlx
|ND_gltf_colorimage|NG_gltf_colorimage|gltf_pbr.mtlx
|ND_gltf_image_color3_color3_1_0|NG_NG_gltf_image_color3_color3_1_0|gltf_pbr.mtlx
|ND_gltf_image_color4_color4_1_0|NG_gltf_image_color4_color4_1_0|gltf_pbr.mtlx
|ND_gltf_image_float_float_1_0|NG_gltf_image_float_float_1_0|gltf_pbr.mtlx
|ND_gltf_image_vector3_vector3_1_0|NG_gltf_image_vector3_vector3_1_0|gltf_pbr.mtlx
|ND_gltf_normalmap_vector3_1_0|NG_gltf_normalmap_vector3_1_0|gltf_pbr.mtlx
|ND_gltf_iridescence_thickness_float_1_0|NG_gltf_iridescence_thickness_float_1_0|gltf_pbr.mtlx
|ND_open_pbr_surface_surfaceshader|NG_open_pbr_surface_surfaceshader|open_pbr_surface.mtlx
|ND_standard_surface_surfaceshader|NG_standard_surface_surfaceshader_100|standard_surface.mtlx
|ND_UsdPreviewSurface_surfaceshader|IMP_UsdPreviewSurface_surfaceshader|usd_preview_surface.mtlx
|ND_UsdUVTexture|IMP_UsdUVTexture_22|usd_preview_surface.mtlx
|ND_UsdUVTexture_23|IMP_UsdUVTexture_23|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_integer|IMP_UsdPrimvarReader_integer|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_boolean|IMP_UsdPrimvarReader_boolean|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_string|IMP_UsdPrimvarReader_string|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_float|IMP_UsdPrimvarReader_float|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector2|IMP_UsdPrimvarReader_vector2|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector3|IMP_UsdPrimvarReader_vector3|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector4|IMP_UsdPrimvarReader_vector4|usd_preview_surface.mtlx
|ND_UsdTransform2d|IMP_UsdTransform2d|usd_preview_surface.mtlx
|ND_lama_add_bsdf|NG_lama_add_bsdf|lama_add.mtlx
|ND_lama_add_edf|NG_lama_add_edf|lama_add.mtlx
|ND_lama_conductor|IMPL_lama_conductor|lama_conductor.mtlx
|ND_lama_dielectric|IMPL_lama_dielectric|lama_dielectric.mtlx
|ND_lama_diffuse|NG_lama_diffuse|lama_diffuse.mtlx
|ND_lama_emission|IMPL_lama_emission|lama_emission.mtlx
|ND_lama_layer_bsdf|NG_lama_layer_bsdf|lama_layer.mtlx
|ND_lama_mix_bsdf|NG_lama_mix_bsdf|lama_mix.mtlx
|ND_lama_mix_edf|NG_lama_mix_edf|lama_mix.mtlx
|ND_lama_sheen|IMPL_lama_sheen|lama_sheen.mtlx
|ND_lama_sss|IMPL_lama_sss|lama_sss.mtlx
|ND_lama_translucent|NG_lama_translucent|lama_translucent.mtlx
|ND_standard_surface_to_gltf_pbr|NG_standard_surface_to_gltf_pbr|standard_surface_to_gltf_pbr.mtlx
|ND_standard_surface_to_UsdPreviewSurface|NG_standard_surface_to_UsdPreviewSurface|standard_surface_to_usd.mtlx
|ND_g18_rec709_to_lin_rec709_color3|NG_g18_rec709_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_g18_rec709_to_lin_rec709_color4|NG_g18_rec709_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_g22_rec709_to_lin_rec709_color3|NG_g22_rec709_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_g22_rec709_to_lin_rec709_color4|NG_g22_rec709_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_rec709_display_to_lin_rec709_color3|NG_rec709_display_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_rec709_display_to_lin_rec709_color4|NG_rec709_display_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_acescg_to_lin_rec709_color3|NG_acescg_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_acescg_to_lin_rec709_color4|NG_acescg_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_g22_ap1_to_lin_rec709_color3|NG_g22_ap1_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_g22_ap1_to_lin_rec709_color4|NG_g22_ap1_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_srgb_texture_to_lin_rec709_color3|NG_srgb_texture_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_srgb_texture_to_lin_rec709_color4|NG_srgb_texture_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_lin_adobergb_to_lin_rec709_color3|NG_lin_adobergb_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_lin_adobergb_to_lin_rec709_color4|NG_lin_adobergb_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_adobergb_to_lin_rec709_color3|NG_adobergb_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_adobergb_to_lin_rec709_color4|NG_adobergb_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_srgb_displayp3_to_lin_rec709_color3|NG_srgb_displayp3_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_srgb_displayp3_to_lin_rec709_color4|NG_srgb_displayp3_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_lin_displayp3_to_lin_rec709_color3|NG_lin_displayp3_to_lin_rec709_color3|cmlib_ng.mtlx
|ND_lin_displayp3_to_lin_rec709_color4|NG_lin_displayp3_to_lin_rec709_color4|cmlib_ng.mtlx
|ND_facingratio_float|NG_facingratio_float|nprlib_ng.mtlx
|ND_glossiness_anisotropy|IMP_glossiness_anisotropy|pbrlib_ng.mtlx
|ND_tiledimage_float|NG_tiledimage_float|stdlib_ng.mtlx
|ND_tiledimage_color3|NG_tiledimage_color3|stdlib_ng.mtlx
|ND_tiledimage_color4|NG_tiledimage_color4|stdlib_ng.mtlx
|ND_tiledimage_vector2|NG_tiledimage_vector2|stdlib_ng.mtlx
|ND_tiledimage_vector3|NG_tiledimage_vector3|stdlib_ng.mtlx
|ND_tiledimage_vector4|NG_tiledimage_vector4|stdlib_ng.mtlx
|ND_triplanarprojection_float|NG_triplanarprojection_float|stdlib_ng.mtlx
|ND_triplanarprojection_color3|NG_triplanarprojection_color3|stdlib_ng.mtlx
|ND_triplanarprojection_color4|NG_triplanarprojection_color4|stdlib_ng.mtlx
|ND_triplanarprojection_vector2|NG_triplanarprojection_vector2|stdlib_ng.mtlx
|ND_triplanarprojection_vector3|NG_triplanarprojection_vector3|stdlib_ng.mtlx
|ND_triplanarprojection_vector4|NG_triplanarprojection_vector4|stdlib_ng.mtlx
|ND_ramp4_float|NG_ramp4_float|stdlib_ng.mtlx
|ND_ramp4_color3|NG_ramp4_color3|stdlib_ng.mtlx
|ND_ramp4_color4|NG_ramp4_color4|stdlib_ng.mtlx
|ND_ramp4_vector2|NG_ramp4_vector2|stdlib_ng.mtlx
|ND_ramp4_vector3|NG_ramp4_vector3|stdlib_ng.mtlx
|ND_ramp4_vector4|NG_ramp4_vector4|stdlib_ng.mtlx
|ND_noise2d_color3|NG_noise2d_color3|stdlib_ng.mtlx
|ND_noise2d_color4|NG_noise2d_color4|stdlib_ng.mtlx
|ND_noise2d_color3FA|NG_noise2d_color3FA|stdlib_ng.mtlx
|ND_noise2d_color4FA|NG_noise2d_color4FA|stdlib_ng.mtlx
|ND_noise2d_vector2FA|NG_noise2d_vector2FA|stdlib_ng.mtlx
|ND_noise2d_vector3FA|NG_noise2d_vector3FA|stdlib_ng.mtlx
|ND_noise2d_vector4FA|NG_noise2d_vector4FA|stdlib_ng.mtlx
|ND_noise3d_color3|NG_noise3d_color3|stdlib_ng.mtlx
|ND_noise3d_color4|NG_noise3d_color4|stdlib_ng.mtlx
|ND_noise3d_color3FA|NG_noise3d_color3FA|stdlib_ng.mtlx
|ND_noise3d_color4FA|NG_noise3d_color4FA|stdlib_ng.mtlx
|ND_noise3d_vector2FA|NG_noise3d_vector2FA|stdlib_ng.mtlx
|ND_noise3d_vector3FA|NG_noise3d_vector3FA|stdlib_ng.mtlx
|ND_noise3d_vector4FA|NG_noise3d_vector4FA|stdlib_ng.mtlx
|ND_fractal3d_color3|NG_fractal3d_color3|stdlib_ng.mtlx
|ND_fractal3d_color4|NG_fractal3d_color4|stdlib_ng.mtlx
|ND_fractal3d_color3FA|NG_fractal3d_color3FA|stdlib_ng.mtlx
|ND_fractal3d_color4FA|NG_fractal3d_color4FA|stdlib_ng.mtlx
|ND_fractal3d_vector2FA|NG_fractal3d_vector2FA|stdlib_ng.mtlx
|ND_fractal3d_vector3FA|NG_fractal3d_vector3FA|stdlib_ng.mtlx
|ND_fractal3d_vector4FA|NG_fractal3d_vector4FA|stdlib_ng.mtlx
|ND_unifiednoise2d_float|NG_unifiednoise2d_float|stdlib_ng.mtlx
|ND_unifiednoise3d_float|NG_unifiednoise3d_float|stdlib_ng.mtlx
|ND_randomfloat_float|NG_randomfloat_float|stdlib_ng.mtlx
|ND_randomfloat_integer|NG_randomfloat_integer|stdlib_ng.mtlx
|ND_randomcolor_float|NG_randomcolor_float|stdlib_ng.mtlx
|ND_randomcolor_integer|NG_randomcolor_integer|stdlib_ng.mtlx
|ND_checkerboard_color3|NG_checkerboard_color3|stdlib_ng.mtlx
|ND_line_float|NG_line_float|stdlib_ng.mtlx
|ND_circle_float|NG_circle_float|stdlib_ng.mtlx
|ND_cloverleaf_float|NG_cloverleaf_float|stdlib_ng.mtlx
|ND_hexagon_float|NG_hexagon_float|stdlib_ng.mtlx
|ND_grid_color3|NG_grid_color3|stdlib_ng.mtlx
|ND_crosshatch_color3|NG_crosshatch_color3|stdlib_ng.mtlx
|ND_tiledcircles_color3|NG_tiledcircles_color3|stdlib_ng.mtlx
|ND_tiledcloverleafs_color3|NG_tiledcloverleafs_color3|stdlib_ng.mtlx
|ND_tiledhexagons_color3|NG_tiledhexagons_color3|stdlib_ng.mtlx
|ND_bump_vector3|NG_bump_vector3|stdlib_ng.mtlx
|ND_distance_vector2|NG_distance_vector2|stdlib_ng.mtlx
|ND_distance_vector3|NG_distance_vector3|stdlib_ng.mtlx
|ND_distance_vector4|NG_distance_vector4|stdlib_ng.mtlx
|ND_place2d_vector2|NG_place2d_vector2|stdlib_ng.mtlx
|ND_trianglewave_float|NG_trianglewave_float|stdlib_ng.mtlx
|ND_smoothstep_color3|NG_smoothstep_color3|stdlib_ng.mtlx
|ND_smoothstep_color4|NG_smoothstep_color4|stdlib_ng.mtlx
|ND_smoothstep_color3FA|NG_smoothstep_color3FA|stdlib_ng.mtlx
|ND_smoothstep_color4FA|NG_smoothstep_color4FA|stdlib_ng.mtlx
|ND_smoothstep_vector2FA|NG_smoothstep_vector2FA|stdlib_ng.mtlx
|ND_smoothstep_vector3FA|NG_smoothstep_vector3FA|stdlib_ng.mtlx
|ND_smoothstep_vector4FA|NG_smoothstep_vector4FA|stdlib_ng.mtlx
|ND_contrast_float|NG_contrast_float|stdlib_ng.mtlx
|ND_contrast_color3|NG_contrast_color3|stdlib_ng.mtlx
|ND_contrast_color4|NG_contrast_color4|stdlib_ng.mtlx
|ND_contrast_vector2|NG_contrast_vector2|stdlib_ng.mtlx
|ND_contrast_vector3|NG_contrast_vector3|stdlib_ng.mtlx
|ND_contrast_vector4|NG_contrast_vector4|stdlib_ng.mtlx
|ND_contrast_color3FA|NG_contrast_color3FA|stdlib_ng.mtlx
|ND_contrast_color4FA|NG_contrast_color4FA|stdlib_ng.mtlx
|ND_contrast_vector2FA|NG_contrast_vector2FA|stdlib_ng.mtlx
|ND_contrast_vector3FA|NG_contrast_vector3FA|stdlib_ng.mtlx
|ND_contrast_vector4FA|NG_contrast_vector4FA|stdlib_ng.mtlx
|ND_range_float|NG_range_float|stdlib_ng.mtlx
|ND_range_color3|NG_range_color3|stdlib_ng.mtlx
|ND_range_color4|NG_range_color4|stdlib_ng.mtlx
|ND_range_vector2|NG_range_vector2|stdlib_ng.mtlx
|ND_range_vector3|NG_range_vector3|stdlib_ng.mtlx
|ND_range_vector4|NG_range_vector4|stdlib_ng.mtlx
|ND_range_color3FA|NG_range_color3FA|stdlib_ng.mtlx
|ND_range_color4FA|NG_range_color4FA|stdlib_ng.mtlx
|ND_range_vector2FA|NG_range_vector2FA|stdlib_ng.mtlx
|ND_range_vector3FA|NG_range_vector3FA|stdlib_ng.mtlx
|ND_range_vector4FA|NG_range_vector4FA|stdlib_ng.mtlx
|ND_hsvadjust_color3|NG_hsvadjust_color3|stdlib_ng.mtlx
|ND_hsvadjust_color4|NG_hsvadjust_color4|stdlib_ng.mtlx
|ND_saturate_color3|NG_saturate_color3|stdlib_ng.mtlx
|ND_saturate_color4|NG_saturate_color4|stdlib_ng.mtlx
|ND_colorcorrect_color3|NG_colorcorrect_color3|stdlib_ng.mtlx
|ND_colorcorrect_color4|NG_colorcorrect_color4|stdlib_ng.mtlx
|ND_overlay_float|NG_overlay_float|stdlib_ng.mtlx
|ND_overlay_color3|NG_overlay_color3|stdlib_ng.mtlx
|ND_overlay_color4|NG_overlay_color4|stdlib_ng.mtlx
|ND_convert_color3_surfaceshader|NG_convert_color3_surfaceshader|stdlib_ng.mtlx
|ND_convert_color4_surfaceshader|NG_convert_color4_surfaceshader|stdlib_ng.mtlx
|ND_convert_float_surfaceshader|NG_convert_float_surfaceshader|stdlib_ng.mtlx
|ND_convert_vector2_surfaceshader|NG_convert_vector2_surfaceshader|stdlib_ng.mtlx
|ND_convert_vector3_surfaceshader|NG_convert_vector3_surfaceshader|stdlib_ng.mtlx
|ND_convert_vector4_surfaceshader|NG_convert_vector4_surfaceshader|stdlib_ng.mtlx
|ND_convert_integer_surfaceshader|NG_convert_integer_surfaceshader|stdlib_ng.mtlx
|ND_convert_boolean_surfaceshader|NG_convert_boolean_surfaceshader|stdlib_ng.mtlx
|ND_extract_color3|NG_extract_color3|stdlib_ng.mtlx
|ND_extract_color4|NG_extract_color4|stdlib_ng.mtlx
|ND_extract_vector2|NG_extract_vector2|stdlib_ng.mtlx
|ND_extract_vector3|NG_extract_vector3|stdlib_ng.mtlx
|ND_extract_vector4|NG_extract_vector4|stdlib_ng.mtlx
|ND_separate2_vector2|NG_separate2_vector2|stdlib_ng.mtlx
|ND_separate3_color3|NG_separate3_color3|stdlib_ng.mtlx
|ND_separate3_vector3|NG_separate3_vector3|stdlib_ng.mtlx
|ND_separate4_color4|NG_separate4_color4|stdlib_ng.mtlx
|ND_separate4_vector4|NG_separate4_vector4|stdlib_ng.mtlx

</details>


**Previous Library Definitions / Implementations**


<details><summary>0 node graphs</summary>

| Definition | NodeGraph | File |
| --- | --- | --- |

</details>


We can see from the results, that 2 new targets were introduced as well as a number of new implementations written as source code and node graphs. The discrepancy, in the source code counts gives a hint as to the extent support for certain node definitions. That is, implementations for a given target may not be available for certain nodes. An example is `ambientocclusion` which is not supported by hardware shader targets at time of
writing.

#### Comparing Implementations

We will first check the number of implementations which are associated with a definition in each library.
We use the `getNodeDef()` to skip any unassociated implementations.
 
We must check for implementations which are either:
1. Source implementations using <a href="https://materialx.org/docs/api/class_document.html" target="_blank">
getImplementations()</a> but also check explicity for:
2. Node graphs using  <a href="https://materialx.org/docs/api/class_document.html" target="_blank">
getNodeGraphs()</a>

This differs from when getting an implementation from a node definition which will return either a source
implementation or a node graph implementation when using `getImplementation()`.`

In [95]:
# Collect all Implementation and NodeGraph elements from two libraries
allImpls = set()
implsNoDef = set()
for i in currLibrary.getImplementations():
    if i.getNodeDef():
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
for i in currLibrary.getNodeGraphs():
    if i.getNodeDef():
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
print('Current implementation count:', len(allImpls))

allImpls2 = set()
for i in otherLibrary.getImplementations():
    if i.getNodeDef():
        allImpls2.add(i.getName())
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
for i in otherLibrary.getNodeGraphs():
    if i.getNodeDef():
        allImpls2.add(i.getName())
        allImpls.add(i.getName())
print('Previous implementation count:', len(allImpls2))

Current implementation count: 2319
Previous implementation count: 0


As with definitions we can compare the implementations which are in both libraries in detail.

In [96]:
title = '| Name | Node Category | Node Type |\n'
title = title + '| --- | --- | --- |\n'

allImpls = sorted(allImpls)
added = 0
addedText = title
removed = 0
removedText = title

title = '| Name | Change |\n| --- | --- |\n'
changed = 0
changedText = title

for item in allImpls:
    
    # Check for new implementations. Note that we check for source implementations
    # as well as node graph implementations.
    impl = otherLibrary.getImplementation(item)
    if not impl:
        impl = otherLibrary.getNodeGraph(item)
    if not impl:
        ns = 'None'
        nt = 'None'
        curimpl = currLibrary.getChild(item)
        if curimpl:
            nd = curimpl.getNodeDef()
            if nd:
                ns = nd.getNodeString()
                nt = nd.getType()
        addedText = addedText + '| %s | <a href="../documents/definitions/%s.html">%s</a> | %s | \n' % (item, ns, ns, nt) 
        added = added + 1
        continue
    
    # Check for implementations which have been removed. Note that we check for source implementations
    # as well as node graph implementations.
    impl2 = currLibrary.getImplementation(item)
    if not impl2:
        impl2 = currLibrary.getNodeGraph(item)
    if not impl2:
        oldimpl = currLibrary.getChild(item)
        if oldimpl:
            nd = oldimpl.getNodeDef()
            if nd:
                ns = nd.getNodeString()
                nt = nd.getType()        
        removedText = removedText + '| %s | %s | %s | \n' % (item, ns, nt)
        removed = removed + 1
        continue
    
    # Check for implementations which have been modified.
    # Use the built in comparitor first before performing a detailed comparison.
    difference = (impl != impl2)
    if difference:
        ns = 'None'
        nt = 'None'
        if impl2:
            nd = impl2.getNodeDef()
            if nd:
                ns = nd.getNodeString()                
        changedText = changedText + '| <a href="../documents/definitions/%s.html">%s</a> | ' % (ns, item)
        # Perform detailed comparison
        log = []
        difference = compareElements(impl, impl2, '', log) 
        if difference:
            for line in log:
                changedText = changedText + line + '<br>'
            changedText = changedText + '|\n'
            changed = changed + 1


delta = '%d implementations added' % added
text = '<details><summary>' + delta + '</summary>\n\n' + addedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)

delta = '%d implementations removed' % removed
text = '<details><summary>' + delta + '</summary>\n\n' +  removedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)

delta = '%d implementations modified' % changed
text = '<details><summary>' + delta + '</summary>\n\n' + changedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)


<details><summary>2319 implementations added</summary>

| Name | Node Category | Node Type |
| --- | --- | --- |
| IMPL_gltf_pbr_surfaceshader | <a href="../documents/definitions/gltf_pbr.html">gltf_pbr</a> | surfaceshader | 
| IMPL_lama_conductor | <a href="../documents/definitions/LamaConductor.html">LamaConductor</a> | BSDF | 
| IMPL_lama_dielectric | <a href="../documents/definitions/LamaDielectric.html">LamaDielectric</a> | BSDF | 
| IMPL_lama_emission | <a href="../documents/definitions/LamaEmission.html">LamaEmission</a> | EDF | 
| IMPL_lama_sheen | <a href="../documents/definitions/LamaSheen.html">LamaSheen</a> | BSDF | 
| IMPL_lama_sss | <a href="../documents/definitions/LamaSSS.html">LamaSSS</a> | BSDF | 
| IMPL_standard_surface_surfaceshader_100 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| IMPL_standard_surface_surfaceshader_101 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| IMP_UsdPreviewSurface_surfaceshader | <a href="../documents/definitions/UsdPreviewSurface.html">UsdPreviewSurface</a> | surfaceshader | 
| IMP_UsdPrimvarReader_boolean | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | boolean | 
| IMP_UsdPrimvarReader_float | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | float | 
| IMP_UsdPrimvarReader_integer | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | integer | 
| IMP_UsdPrimvarReader_string | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | string | 
| IMP_UsdPrimvarReader_vector2 | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | vector2 | 
| IMP_UsdPrimvarReader_vector3 | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | vector3 | 
| IMP_UsdPrimvarReader_vector4 | <a href="../documents/definitions/UsdPrimvarReader.html">UsdPrimvarReader</a> | vector4 | 
| IMP_UsdTransform2d | <a href="../documents/definitions/UsdTransform2d.html">UsdTransform2d</a> | vector2 | 
| IMP_UsdUVTexture_22 | <a href="../documents/definitions/UsdUVTexture.html">UsdUVTexture</a> | multioutput | 
| IMP_UsdUVTexture_23 | <a href="../documents/definitions/UsdUVTexture.html">UsdUVTexture</a> | multioutput | 
| IMP_glossiness_anisotropy | <a href="../documents/definitions/glossiness_anisotropy.html">glossiness_anisotropy</a> | vector2 | 
| IM_absorption_vdf_genmdl | <a href="../documents/definitions/absorption_vdf.html">absorption_vdf</a> | VDF | 
| IM_absval_color3_genglsl | <a href="../documents/definitions/absval.html">absval</a> | color3 | 
| IM_absval_color3_genmdl | <a href="../documents/definitions/absval.html">absval</a> | color3 | 
| IM_absval_color3_genmsl | <a href="../documents/definitions/absval.html">absval</a> | color3 | 
| IM_absval_color3_genosl | <a href="../documents/definitions/absval.html">absval</a> | color3 | 
| IM_absval_color4_genglsl | <a href="../documents/definitions/absval.html">absval</a> | color4 | 
| IM_absval_color4_genmdl | <a href="../documents/definitions/absval.html">absval</a> | color4 | 
| IM_absval_color4_genmsl | <a href="../documents/definitions/absval.html">absval</a> | color4 | 
| IM_absval_color4_genosl | <a href="../documents/definitions/absval.html">absval</a> | color4 | 
| IM_absval_float_genglsl | <a href="../documents/definitions/absval.html">absval</a> | float | 
| IM_absval_float_genmdl | <a href="../documents/definitions/absval.html">absval</a> | float | 
| IM_absval_float_genmsl | <a href="../documents/definitions/absval.html">absval</a> | float | 
| IM_absval_float_genosl | <a href="../documents/definitions/absval.html">absval</a> | float | 
| IM_absval_vector2_genglsl | <a href="../documents/definitions/absval.html">absval</a> | vector2 | 
| IM_absval_vector2_genmdl | <a href="../documents/definitions/absval.html">absval</a> | vector2 | 
| IM_absval_vector2_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector2 | 
| IM_absval_vector2_genosl | <a href="../documents/definitions/absval.html">absval</a> | vector2 | 
| IM_absval_vector3_genglsl | <a href="../documents/definitions/absval.html">absval</a> | vector3 | 
| IM_absval_vector3_genmdl | <a href="../documents/definitions/absval.html">absval</a> | vector3 | 
| IM_absval_vector3_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector3 | 
| IM_absval_vector3_genosl | <a href="../documents/definitions/absval.html">absval</a> | vector3 | 
| IM_absval_vector4_genglsl | <a href="../documents/definitions/absval.html">absval</a> | vector4 | 
| IM_absval_vector4_genmdl | <a href="../documents/definitions/absval.html">absval</a> | vector4 | 
| IM_absval_vector4_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector4 | 
| IM_absval_vector4_genosl | <a href="../documents/definitions/absval.html">absval</a> | vector4 | 
| IM_acos_float_genglsl | <a href="../documents/definitions/acos.html">acos</a> | float | 
| IM_acos_float_genmdl | <a href="../documents/definitions/acos.html">acos</a> | float | 
| IM_acos_float_genmsl | <a href="../documents/definitions/acos.html">acos</a> | float | 
| IM_acos_float_genosl | <a href="../documents/definitions/acos.html">acos</a> | float | 
| IM_acos_vector2_genglsl | <a href="../documents/definitions/acos.html">acos</a> | vector2 | 
| IM_acos_vector2_genmdl | <a href="../documents/definitions/acos.html">acos</a> | vector2 | 
| IM_acos_vector2_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector2 | 
| IM_acos_vector2_genosl | <a href="../documents/definitions/acos.html">acos</a> | vector2 | 
| IM_acos_vector3_genglsl | <a href="../documents/definitions/acos.html">acos</a> | vector3 | 
| IM_acos_vector3_genmdl | <a href="../documents/definitions/acos.html">acos</a> | vector3 | 
| IM_acos_vector3_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector3 | 
| IM_acos_vector3_genosl | <a href="../documents/definitions/acos.html">acos</a> | vector3 | 
| IM_acos_vector4_genglsl | <a href="../documents/definitions/acos.html">acos</a> | vector4 | 
| IM_acos_vector4_genmdl | <a href="../documents/definitions/acos.html">acos</a> | vector4 | 
| IM_acos_vector4_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector4 | 
| IM_acos_vector4_genosl | <a href="../documents/definitions/acos.html">acos</a> | vector4 | 
| IM_add_bsdf_genglsl | <a href="../documents/definitions/add.html">add</a> | BSDF | 
| IM_add_bsdf_genmdl | <a href="../documents/definitions/add.html">add</a> | BSDF | 
| IM_add_bsdf_genmsl | <a href="../documents/definitions/add.html">add</a> | BSDF | 
| IM_add_bsdf_genosl | <a href="../documents/definitions/add.html">add</a> | BSDF | 
| IM_add_color3FA_genglsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3FA_genmdl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3FA_genmsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3FA_genosl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3_genglsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3_genmdl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3_genmsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3_genosl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color4FA_genglsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4FA_genmdl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4FA_genmsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4FA_genosl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4_genglsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4_genmdl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4_genmsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4_genosl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_edf_genglsl | <a href="../documents/definitions/add.html">add</a> | EDF | 
| IM_add_edf_genmdl | <a href="../documents/definitions/add.html">add</a> | EDF | 
| IM_add_edf_genmsl | <a href="../documents/definitions/add.html">add</a> | EDF | 
| IM_add_edf_genosl | <a href="../documents/definitions/add.html">add</a> | EDF | 
| IM_add_float_genglsl | <a href="../documents/definitions/add.html">add</a> | float | 
| IM_add_float_genmdl | <a href="../documents/definitions/add.html">add</a> | float | 
| IM_add_float_genmsl | <a href="../documents/definitions/add.html">add</a> | float | 
| IM_add_float_genosl | <a href="../documents/definitions/add.html">add</a> | float | 
| IM_add_matrix33FA_genglsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33FA_genmdl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33FA_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33FA_genosl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33_genglsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33_genmdl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33_genosl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix44FA_genglsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44FA_genmdl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44FA_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44FA_genosl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44_genglsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44_genmdl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44_genosl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_vdf_genmdl | <a href="../documents/definitions/add.html">add</a> | VDF | 
| IM_add_vector2FA_genglsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2FA_genmdl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2FA_genosl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2_genglsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2_genmdl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2_genmsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2_genosl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector3FA_genglsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3FA_genmdl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3FA_genosl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3_genglsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3_genmdl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3_genmsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3_genosl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector4FA_genglsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4FA_genmdl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4FA_genosl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4_genglsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4_genmdl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4_genmsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4_genosl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_ambientocclusion_float_genmdl | <a href="../documents/definitions/ambientocclusion.html">ambientocclusion</a> | float | 
| IM_ambientocclusion_float_genosl | <a href="../documents/definitions/ambientocclusion.html">ambientocclusion</a> | float | 
| IM_anisotropic_vdf_genglsl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_anisotropic_vdf_genmdl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_anisotropic_vdf_genmsl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_anisotropic_vdf_genosl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_artistic_ior_genglsl | <a href="../documents/definitions/artistic_ior.html">artistic_ior</a> | multioutput | 
| IM_artistic_ior_genmdl | <a href="../documents/definitions/artistic_ior.html">artistic_ior</a> | multioutput | 
| IM_artistic_ior_genmsl | <a href="../documents/definitions/artistic_ior.html">artistic_ior</a> | multioutput | 
| IM_artistic_ior_genosl | <a href="../documents/definitions/artistic_ior.html">artistic_ior</a> | multioutput | 
| IM_asin_float_genglsl | <a href="../documents/definitions/asin.html">asin</a> | float | 
| IM_asin_float_genmdl | <a href="../documents/definitions/asin.html">asin</a> | float | 
| IM_asin_float_genmsl | <a href="../documents/definitions/asin.html">asin</a> | float | 
| IM_asin_float_genosl | <a href="../documents/definitions/asin.html">asin</a> | float | 
| IM_asin_vector2_genglsl | <a href="../documents/definitions/asin.html">asin</a> | vector2 | 
| IM_asin_vector2_genmdl | <a href="../documents/definitions/asin.html">asin</a> | vector2 | 
| IM_asin_vector2_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector2 | 
| IM_asin_vector2_genosl | <a href="../documents/definitions/asin.html">asin</a> | vector2 | 
| IM_asin_vector3_genglsl | <a href="../documents/definitions/asin.html">asin</a> | vector3 | 
| IM_asin_vector3_genmdl | <a href="../documents/definitions/asin.html">asin</a> | vector3 | 
| IM_asin_vector3_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector3 | 
| IM_asin_vector3_genosl | <a href="../documents/definitions/asin.html">asin</a> | vector3 | 
| IM_asin_vector4_genglsl | <a href="../documents/definitions/asin.html">asin</a> | vector4 | 
| IM_asin_vector4_genmdl | <a href="../documents/definitions/asin.html">asin</a> | vector4 | 
| IM_asin_vector4_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector4 | 
| IM_asin_vector4_genosl | <a href="../documents/definitions/asin.html">asin</a> | vector4 | 
| IM_atan2_float_genglsl | <a href="../documents/definitions/atan2.html">atan2</a> | float | 
| IM_atan2_float_genmdl | <a href="../documents/definitions/atan2.html">atan2</a> | float | 
| IM_atan2_float_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | float | 
| IM_atan2_float_genosl | <a href="../documents/definitions/atan2.html">atan2</a> | float | 
| IM_atan2_vector2_genglsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector2 | 
| IM_atan2_vector2_genmdl | <a href="../documents/definitions/atan2.html">atan2</a> | vector2 | 
| IM_atan2_vector2_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector2 | 
| IM_atan2_vector2_genosl | <a href="../documents/definitions/atan2.html">atan2</a> | vector2 | 
| IM_atan2_vector3_genglsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector3 | 
| IM_atan2_vector3_genmdl | <a href="../documents/definitions/atan2.html">atan2</a> | vector3 | 
| IM_atan2_vector3_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector3 | 
| IM_atan2_vector3_genosl | <a href="../documents/definitions/atan2.html">atan2</a> | vector3 | 
| IM_atan2_vector4_genglsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector4 | 
| IM_atan2_vector4_genmdl | <a href="../documents/definitions/atan2.html">atan2</a> | vector4 | 
| IM_atan2_vector4_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector4 | 
| IM_atan2_vector4_genosl | <a href="../documents/definitions/atan2.html">atan2</a> | vector4 | 
| IM_bitangent_vector3_genglsl | <a href="../documents/definitions/bitangent.html">bitangent</a> | vector3 | 
| IM_bitangent_vector3_genmdl | <a href="../documents/definitions/bitangent.html">bitangent</a> | vector3 | 
| IM_bitangent_vector3_genmsl | <a href="../documents/definitions/bitangent.html">bitangent</a> | vector3 | 
| IM_bitangent_vector3_genosl | <a href="../documents/definitions/bitangent.html">bitangent</a> | vector3 | 
| IM_blackbody_genglsl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blackbody_genmdl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blackbody_genmsl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blackbody_genosl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blur_color3_genglsl | <a href="../documents/definitions/blur.html">blur</a> | color3 | 
| IM_blur_color3_genmdl | <a href="../documents/definitions/blur.html">blur</a> | color3 | 
| IM_blur_color3_genmsl | <a href="../documents/definitions/blur.html">blur</a> | color3 | 
| IM_blur_color3_genosl | <a href="../documents/definitions/blur.html">blur</a> | color3 | 
| IM_blur_color4_genglsl | <a href="../documents/definitions/blur.html">blur</a> | color4 | 
| IM_blur_color4_genmdl | <a href="../documents/definitions/blur.html">blur</a> | color4 | 
| IM_blur_color4_genmsl | <a href="../documents/definitions/blur.html">blur</a> | color4 | 
| IM_blur_color4_genosl | <a href="../documents/definitions/blur.html">blur</a> | color4 | 
| IM_blur_float_genglsl | <a href="../documents/definitions/blur.html">blur</a> | float | 
| IM_blur_float_genmdl | <a href="../documents/definitions/blur.html">blur</a> | float | 
| IM_blur_float_genmsl | <a href="../documents/definitions/blur.html">blur</a> | float | 
| IM_blur_float_genosl | <a href="../documents/definitions/blur.html">blur</a> | float | 
| IM_blur_vector2_genglsl | <a href="../documents/definitions/blur.html">blur</a> | vector2 | 
| IM_blur_vector2_genmdl | <a href="../documents/definitions/blur.html">blur</a> | vector2 | 
| IM_blur_vector2_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector2 | 
| IM_blur_vector2_genosl | <a href="../documents/definitions/blur.html">blur</a> | vector2 | 
| IM_blur_vector3_genglsl | <a href="../documents/definitions/blur.html">blur</a> | vector3 | 
| IM_blur_vector3_genmdl | <a href="../documents/definitions/blur.html">blur</a> | vector3 | 
| IM_blur_vector3_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector3 | 
| IM_blur_vector3_genosl | <a href="../documents/definitions/blur.html">blur</a> | vector3 | 
| IM_blur_vector4_genglsl | <a href="../documents/definitions/blur.html">blur</a> | vector4 | 
| IM_blur_vector4_genmdl | <a href="../documents/definitions/blur.html">blur</a> | vector4 | 
| IM_blur_vector4_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector4 | 
| IM_blur_vector4_genosl | <a href="../documents/definitions/blur.html">blur</a> | vector4 | 
| IM_burley_diffuse_bsdf_genglsl | <a href="../documents/definitions/burley_diffuse_bsdf.html">burley_diffuse_bsdf</a> | BSDF | 
| IM_burley_diffuse_bsdf_genmdl | <a href="../documents/definitions/burley_diffuse_bsdf.html">burley_diffuse_bsdf</a> | BSDF | 
| IM_burley_diffuse_bsdf_genmsl | <a href="../documents/definitions/burley_diffuse_bsdf.html">burley_diffuse_bsdf</a> | BSDF | 
| IM_burley_diffuse_bsdf_genosl | <a href="../documents/definitions/burley_diffuse_bsdf.html">burley_diffuse_bsdf</a> | BSDF | 
| IM_burn_color3_genglsl | <a href="../documents/definitions/burn.html">burn</a> | color3 | 
| IM_burn_color3_genmdl | <a href="../documents/definitions/burn.html">burn</a> | color3 | 
| IM_burn_color3_genmsl | <a href="../documents/definitions/burn.html">burn</a> | color3 | 
| IM_burn_color3_genosl | <a href="../documents/definitions/burn.html">burn</a> | color3 | 
| IM_burn_color4_genglsl | <a href="../documents/definitions/burn.html">burn</a> | color4 | 
| IM_burn_color4_genmdl | <a href="../documents/definitions/burn.html">burn</a> | color4 | 
| IM_burn_color4_genmsl | <a href="../documents/definitions/burn.html">burn</a> | color4 | 
| IM_burn_color4_genosl | <a href="../documents/definitions/burn.html">burn</a> | color4 | 
| IM_burn_float_genglsl | <a href="../documents/definitions/burn.html">burn</a> | float | 
| IM_burn_float_genmdl | <a href="../documents/definitions/burn.html">burn</a> | float | 
| IM_burn_float_genmsl | <a href="../documents/definitions/burn.html">burn</a> | float | 
| IM_burn_float_genosl | <a href="../documents/definitions/burn.html">burn</a> | float | 
| IM_ceil_color3_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | color3 | 
| IM_ceil_color3_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | color3 | 
| IM_ceil_color3_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | color3 | 
| IM_ceil_color3_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | color3 | 
| IM_ceil_color4_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | color4 | 
| IM_ceil_color4_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | color4 | 
| IM_ceil_color4_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | color4 | 
| IM_ceil_color4_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | color4 | 
| IM_ceil_float_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | float | 
| IM_ceil_float_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | float | 
| IM_ceil_float_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | float | 
| IM_ceil_float_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | float | 
| IM_ceil_integer_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_vector2_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector2 | 
| IM_ceil_vector2_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | vector2 | 
| IM_ceil_vector2_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector2 | 
| IM_ceil_vector2_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | vector2 | 
| IM_ceil_vector3_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector3 | 
| IM_ceil_vector3_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | vector3 | 
| IM_ceil_vector3_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector3 | 
| IM_ceil_vector3_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | vector3 | 
| IM_ceil_vector4_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector4 | 
| IM_ceil_vector4_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | vector4 | 
| IM_ceil_vector4_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector4 | 
| IM_ceil_vector4_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | vector4 | 
| IM_cellnoise2d_float_genglsl | <a href="../documents/definitions/cellnoise2d.html">cellnoise2d</a> | float | 
| IM_cellnoise2d_float_genmdl | <a href="../documents/definitions/cellnoise2d.html">cellnoise2d</a> | float | 
| IM_cellnoise2d_float_genmsl | <a href="../documents/definitions/cellnoise2d.html">cellnoise2d</a> | float | 
| IM_cellnoise2d_float_genosl | <a href="../documents/definitions/cellnoise2d.html">cellnoise2d</a> | float | 
| IM_cellnoise3d_float_genglsl | <a href="../documents/definitions/cellnoise3d.html">cellnoise3d</a> | float | 
| IM_cellnoise3d_float_genmdl | <a href="../documents/definitions/cellnoise3d.html">cellnoise3d</a> | float | 
| IM_cellnoise3d_float_genmsl | <a href="../documents/definitions/cellnoise3d.html">cellnoise3d</a> | float | 
| IM_cellnoise3d_float_genosl | <a href="../documents/definitions/cellnoise3d.html">cellnoise3d</a> | float | 
| IM_clamp_color3FA_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3FA_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3FA_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color4FA_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4FA_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4FA_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_float_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | float | 
| IM_clamp_float_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | float | 
| IM_clamp_float_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | float | 
| IM_clamp_float_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | float | 
| IM_clamp_vector2FA_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2FA_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2FA_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector3FA_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3FA_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3FA_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector4FA_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4FA_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4FA_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4_genglsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4_genmdl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4_genosl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_combine2_color4CF_genglsl | <a href="../documents/definitions/combine2.html">combine2</a> | color4 | 
| IM_combine2_color4CF_genmdl | <a href="../documents/definitions/combine2.html">combine2</a> | color4 | 
| IM_combine2_color4CF_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | color4 | 
| IM_combine2_color4CF_genosl | <a href="../documents/definitions/combine2.html">combine2</a> | color4 | 
| IM_combine2_vector2_genglsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector2 | 
| IM_combine2_vector2_genmdl | <a href="../documents/definitions/combine2.html">combine2</a> | vector2 | 
| IM_combine2_vector2_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector2 | 
| IM_combine2_vector2_genosl | <a href="../documents/definitions/combine2.html">combine2</a> | vector2 | 
| IM_combine2_vector4VF_genglsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VF_genmdl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VF_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VF_genosl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VV_genglsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VV_genmdl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VV_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VV_genosl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine3_color3_genglsl | <a href="../documents/definitions/combine3.html">combine3</a> | color3 | 
| IM_combine3_color3_genmdl | <a href="../documents/definitions/combine3.html">combine3</a> | color3 | 
| IM_combine3_color3_genmsl | <a href="../documents/definitions/combine3.html">combine3</a> | color3 | 
| IM_combine3_color3_genosl | <a href="../documents/definitions/combine3.html">combine3</a> | color3 | 
| IM_combine3_vector3_genglsl | <a href="../documents/definitions/combine3.html">combine3</a> | vector3 | 
| IM_combine3_vector3_genmdl | <a href="../documents/definitions/combine3.html">combine3</a> | vector3 | 
| IM_combine3_vector3_genmsl | <a href="../documents/definitions/combine3.html">combine3</a> | vector3 | 
| IM_combine3_vector3_genosl | <a href="../documents/definitions/combine3.html">combine3</a> | vector3 | 
| IM_combine4_color4_genglsl | <a href="../documents/definitions/combine4.html">combine4</a> | color4 | 
| IM_combine4_color4_genmdl | <a href="../documents/definitions/combine4.html">combine4</a> | color4 | 
| IM_combine4_color4_genmsl | <a href="../documents/definitions/combine4.html">combine4</a> | color4 | 
| IM_combine4_color4_genosl | <a href="../documents/definitions/combine4.html">combine4</a> | color4 | 
| IM_combine4_vector4_genglsl | <a href="../documents/definitions/combine4.html">combine4</a> | vector4 | 
| IM_combine4_vector4_genmdl | <a href="../documents/definitions/combine4.html">combine4</a> | vector4 | 
| IM_combine4_vector4_genmsl | <a href="../documents/definitions/combine4.html">combine4</a> | vector4 | 
| IM_combine4_vector4_genosl | <a href="../documents/definitions/combine4.html">combine4</a> | vector4 | 
| IM_conductor_bsdf_genglsl | <a href="../documents/definitions/conductor_bsdf.html">conductor_bsdf</a> | BSDF | 
| IM_conductor_bsdf_genmdl | <a href="../documents/definitions/conductor_bsdf.html">conductor_bsdf</a> | BSDF | 
| IM_conductor_bsdf_genmsl | <a href="../documents/definitions/conductor_bsdf.html">conductor_bsdf</a> | BSDF | 
| IM_conductor_bsdf_genosl | <a href="../documents/definitions/conductor_bsdf.html">conductor_bsdf</a> | BSDF | 
| IM_conical_edf_genmdl | <a href="../documents/definitions/conical_edf.html">conical_edf</a> | EDF | 
| IM_constant_boolean_genglsl | <a href="../documents/definitions/constant.html">constant</a> | boolean | 
| IM_constant_boolean_genmdl | <a href="../documents/definitions/constant.html">constant</a> | boolean | 
| IM_constant_boolean_genmsl | <a href="../documents/definitions/constant.html">constant</a> | boolean | 
| IM_constant_boolean_genosl | <a href="../documents/definitions/constant.html">constant</a> | boolean | 
| IM_constant_color3_genglsl | <a href="../documents/definitions/constant.html">constant</a> | color3 | 
| IM_constant_color3_genmdl | <a href="../documents/definitions/constant.html">constant</a> | color3 | 
| IM_constant_color3_genmsl | <a href="../documents/definitions/constant.html">constant</a> | color3 | 
| IM_constant_color3_genosl | <a href="../documents/definitions/constant.html">constant</a> | color3 | 
| IM_constant_color4_genglsl | <a href="../documents/definitions/constant.html">constant</a> | color4 | 
| IM_constant_color4_genmdl | <a href="../documents/definitions/constant.html">constant</a> | color4 | 
| IM_constant_color4_genmsl | <a href="../documents/definitions/constant.html">constant</a> | color4 | 
| IM_constant_color4_genosl | <a href="../documents/definitions/constant.html">constant</a> | color4 | 
| IM_constant_filename_genglsl | <a href="../documents/definitions/constant.html">constant</a> | filename | 
| IM_constant_filename_genmdl | <a href="../documents/definitions/constant.html">constant</a> | filename | 
| IM_constant_filename_genmsl | <a href="../documents/definitions/constant.html">constant</a> | filename | 
| IM_constant_filename_genosl | <a href="../documents/definitions/constant.html">constant</a> | filename | 
| IM_constant_float_genglsl | <a href="../documents/definitions/constant.html">constant</a> | float | 
| IM_constant_float_genmdl | <a href="../documents/definitions/constant.html">constant</a> | float | 
| IM_constant_float_genmsl | <a href="../documents/definitions/constant.html">constant</a> | float | 
| IM_constant_float_genosl | <a href="../documents/definitions/constant.html">constant</a> | float | 
| IM_constant_integer_genglsl | <a href="../documents/definitions/constant.html">constant</a> | integer | 
| IM_constant_integer_genmdl | <a href="../documents/definitions/constant.html">constant</a> | integer | 
| IM_constant_integer_genmsl | <a href="../documents/definitions/constant.html">constant</a> | integer | 
| IM_constant_integer_genosl | <a href="../documents/definitions/constant.html">constant</a> | integer | 
| IM_constant_matrix33_genglsl | <a href="../documents/definitions/constant.html">constant</a> | matrix33 | 
| IM_constant_matrix33_genmdl | <a href="../documents/definitions/constant.html">constant</a> | matrix33 | 
| IM_constant_matrix33_genmsl | <a href="../documents/definitions/constant.html">constant</a> | matrix33 | 
| IM_constant_matrix33_genosl | <a href="../documents/definitions/constant.html">constant</a> | matrix33 | 
| IM_constant_matrix44_genglsl | <a href="../documents/definitions/constant.html">constant</a> | matrix44 | 
| IM_constant_matrix44_genmdl | <a href="../documents/definitions/constant.html">constant</a> | matrix44 | 
| IM_constant_matrix44_genmsl | <a href="../documents/definitions/constant.html">constant</a> | matrix44 | 
| IM_constant_matrix44_genosl | <a href="../documents/definitions/constant.html">constant</a> | matrix44 | 
| IM_constant_string_genglsl | <a href="../documents/definitions/constant.html">constant</a> | string | 
| IM_constant_string_genmdl | <a href="../documents/definitions/constant.html">constant</a> | string | 
| IM_constant_string_genmsl | <a href="../documents/definitions/constant.html">constant</a> | string | 
| IM_constant_string_genosl | <a href="../documents/definitions/constant.html">constant</a> | string | 
| IM_constant_vector2_genglsl | <a href="../documents/definitions/constant.html">constant</a> | vector2 | 
| IM_constant_vector2_genmdl | <a href="../documents/definitions/constant.html">constant</a> | vector2 | 
| IM_constant_vector2_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector2 | 
| IM_constant_vector2_genosl | <a href="../documents/definitions/constant.html">constant</a> | vector2 | 
| IM_constant_vector3_genglsl | <a href="../documents/definitions/constant.html">constant</a> | vector3 | 
| IM_constant_vector3_genmdl | <a href="../documents/definitions/constant.html">constant</a> | vector3 | 
| IM_constant_vector3_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector3 | 
| IM_constant_vector3_genosl | <a href="../documents/definitions/constant.html">constant</a> | vector3 | 
| IM_constant_vector4_genglsl | <a href="../documents/definitions/constant.html">constant</a> | vector4 | 
| IM_constant_vector4_genmdl | <a href="../documents/definitions/constant.html">constant</a> | vector4 | 
| IM_constant_vector4_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector4 | 
| IM_constant_vector4_genosl | <a href="../documents/definitions/constant.html">constant</a> | vector4 | 
| IM_convert_boolean_float_genglsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_boolean_float_genmdl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_boolean_float_genmsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_boolean_float_genosl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_color3_color4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_color3_color4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_color3_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_color3_color4_genosl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_color3_vector3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_color3_vector3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_color3_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_color3_vector3_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_color4_color3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_color4_color3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_color4_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_color4_color3_genosl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_color4_vector4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_color4_vector4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_color4_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_color4_vector4_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_float_color3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_float_color3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_float_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_float_color3_genosl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_float_color4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_float_color4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_float_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_float_color4_genosl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_float_vector2_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_float_vector2_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_float_vector2_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_float_vector2_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_float_vector3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_float_vector3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_float_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_float_vector3_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_float_vector4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_float_vector4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_float_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_float_vector4_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_integer_float_genglsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_integer_float_genmdl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_integer_float_genmsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_integer_float_genosl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_vector2_vector3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector2_vector3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector2_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector2_vector3_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector3_color3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_vector3_color3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_vector3_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_vector3_color3_genosl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_vector3_vector2_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_vector3_vector2_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_vector3_vector2_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_vector3_vector2_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_vector3_vector4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_vector3_vector4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_vector3_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_vector3_vector4_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_vector4_color4_genglsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_vector4_color4_genmdl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_vector4_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_vector4_color4_genosl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_vector4_vector3_genglsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector4_vector3_genmdl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector4_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector4_vector3_genosl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_cos_float_genglsl | <a href="../documents/definitions/cos.html">cos</a> | float | 
| IM_cos_float_genmdl | <a href="../documents/definitions/cos.html">cos</a> | float | 
| IM_cos_float_genmsl | <a href="../documents/definitions/cos.html">cos</a> | float | 
| IM_cos_float_genosl | <a href="../documents/definitions/cos.html">cos</a> | float | 
| IM_cos_vector2_genglsl | <a href="../documents/definitions/cos.html">cos</a> | vector2 | 
| IM_cos_vector2_genmdl | <a href="../documents/definitions/cos.html">cos</a> | vector2 | 
| IM_cos_vector2_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector2 | 
| IM_cos_vector2_genosl | <a href="../documents/definitions/cos.html">cos</a> | vector2 | 
| IM_cos_vector3_genglsl | <a href="../documents/definitions/cos.html">cos</a> | vector3 | 
| IM_cos_vector3_genmdl | <a href="../documents/definitions/cos.html">cos</a> | vector3 | 
| IM_cos_vector3_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector3 | 
| IM_cos_vector3_genosl | <a href="../documents/definitions/cos.html">cos</a> | vector3 | 
| IM_cos_vector4_genglsl | <a href="../documents/definitions/cos.html">cos</a> | vector4 | 
| IM_cos_vector4_genmdl | <a href="../documents/definitions/cos.html">cos</a> | vector4 | 
| IM_cos_vector4_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector4 | 
| IM_cos_vector4_genosl | <a href="../documents/definitions/cos.html">cos</a> | vector4 | 
| IM_creatematrix_vector3_matrix33_genglsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix33 | 
| IM_creatematrix_vector3_matrix33_genmdl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix33 | 
| IM_creatematrix_vector3_matrix33_genmsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix33 | 
| IM_creatematrix_vector3_matrix33_genosl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix33 | 
| IM_creatematrix_vector3_matrix44_genglsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector3_matrix44_genmdl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector3_matrix44_genmsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector3_matrix44_genosl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector4_matrix44_genglsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector4_matrix44_genmdl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector4_matrix44_genmsl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_creatematrix_vector4_matrix44_genosl | <a href="../documents/definitions/creatematrix.html">creatematrix</a> | matrix44 | 
| IM_crossproduct_vector3_genglsl | <a href="../documents/definitions/crossproduct.html">crossproduct</a> | vector3 | 
| IM_crossproduct_vector3_genmdl | <a href="../documents/definitions/crossproduct.html">crossproduct</a> | vector3 | 
| IM_crossproduct_vector3_genmsl | <a href="../documents/definitions/crossproduct.html">crossproduct</a> | vector3 | 
| IM_crossproduct_vector3_genosl | <a href="../documents/definitions/crossproduct.html">crossproduct</a> | vector3 | 
| IM_determinant_matrix33_genglsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix33_genmdl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix33_genmsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix33_genosl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix44_genglsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix44_genmdl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix44_genmsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix44_genosl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_dielectric_bsdf_genglsl | <a href="../documents/definitions/dielectric_bsdf.html">dielectric_bsdf</a> | BSDF | 
| IM_dielectric_bsdf_genmdl | <a href="../documents/definitions/dielectric_bsdf.html">dielectric_bsdf</a> | BSDF | 
| IM_dielectric_bsdf_genmsl | <a href="../documents/definitions/dielectric_bsdf.html">dielectric_bsdf</a> | BSDF | 
| IM_dielectric_bsdf_genosl | <a href="../documents/definitions/dielectric_bsdf.html">dielectric_bsdf</a> | BSDF | 
| IM_difference_color3_genglsl | <a href="../documents/definitions/difference.html">difference</a> | color3 | 
| IM_difference_color3_genmdl | <a href="../documents/definitions/difference.html">difference</a> | color3 | 
| IM_difference_color3_genmsl | <a href="../documents/definitions/difference.html">difference</a> | color3 | 
| IM_difference_color3_genosl | <a href="../documents/definitions/difference.html">difference</a> | color3 | 
| IM_difference_color4_genglsl | <a href="../documents/definitions/difference.html">difference</a> | color4 | 
| IM_difference_color4_genmdl | <a href="../documents/definitions/difference.html">difference</a> | color4 | 
| IM_difference_color4_genmsl | <a href="../documents/definitions/difference.html">difference</a> | color4 | 
| IM_difference_color4_genosl | <a href="../documents/definitions/difference.html">difference</a> | color4 | 
| IM_difference_float_genglsl | <a href="../documents/definitions/difference.html">difference</a> | float | 
| IM_difference_float_genmdl | <a href="../documents/definitions/difference.html">difference</a> | float | 
| IM_difference_float_genmsl | <a href="../documents/definitions/difference.html">difference</a> | float | 
| IM_difference_float_genosl | <a href="../documents/definitions/difference.html">difference</a> | float | 
| IM_directional_light_genglsl | <a href="../documents/definitions/directional_light.html">directional_light</a> | lightshader | 
| IM_directional_light_genmsl | <a href="../documents/definitions/directional_light.html">directional_light</a> | lightshader | 
| IM_disjointover_color4_genglsl | <a href="../documents/definitions/disjointover.html">disjointover</a> | color4 | 
| IM_disjointover_color4_genmdl | <a href="../documents/definitions/disjointover.html">disjointover</a> | color4 | 
| IM_disjointover_color4_genmsl | <a href="../documents/definitions/disjointover.html">disjointover</a> | color4 | 
| IM_disjointover_color4_genosl | <a href="../documents/definitions/disjointover.html">disjointover</a> | color4 | 
| IM_disney_brdf_2012_surface_brdf_explorer | <a href="../documents/definitions/disney_brdf_2012.html">disney_brdf_2012</a> | surfaceshader | 
| IM_disney_bsdf_2015_surface_pbrt | <a href="../documents/definitions/disney_bsdf_2015.html">disney_bsdf_2015</a> | surfaceshader | 
| IM_displacement_float_genglsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_float_genmdl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_float_genmsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_float_genosl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genglsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genmdl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genmsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genosl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_divide_color3FA_genglsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3FA_genmdl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3FA_genosl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3_genglsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3_genmdl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3_genosl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color4FA_genglsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4FA_genmdl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4FA_genosl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4_genglsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4_genmdl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4_genosl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_float_genglsl | <a href="../documents/definitions/divide.html">divide</a> | float | 
| IM_divide_float_genmdl | <a href="../documents/definitions/divide.html">divide</a> | float | 
| IM_divide_float_genmsl | <a href="../documents/definitions/divide.html">divide</a> | float | 
| IM_divide_float_genosl | <a href="../documents/definitions/divide.html">divide</a> | float | 
| IM_divide_matrix33_genglsl | <a href="../documents/definitions/divide.html">divide</a> | matrix33 | 
| IM_divide_matrix33_genmdl | <a href="../documents/definitions/divide.html">divide</a> | matrix33 | 
| IM_divide_matrix33_genmsl | <a href="../documents/definitions/divide.html">divide</a> | matrix33 | 
| IM_divide_matrix33_genosl | <a href="../documents/definitions/divide.html">divide</a> | matrix33 | 
| IM_divide_matrix44_genglsl | <a href="../documents/definitions/divide.html">divide</a> | matrix44 | 
| IM_divide_matrix44_genmdl | <a href="../documents/definitions/divide.html">divide</a> | matrix44 | 
| IM_divide_matrix44_genmsl | <a href="../documents/definitions/divide.html">divide</a> | matrix44 | 
| IM_divide_matrix44_genosl | <a href="../documents/definitions/divide.html">divide</a> | matrix44 | 
| IM_divide_vector2FA_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2FA_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2FA_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector3FA_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3FA_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3FA_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector4FA_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4FA_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4FA_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4_genglsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4_genmdl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4_genosl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_dodge_color3_genglsl | <a href="../documents/definitions/dodge.html">dodge</a> | color3 | 
| IM_dodge_color3_genmdl | <a href="../documents/definitions/dodge.html">dodge</a> | color3 | 
| IM_dodge_color3_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | color3 | 
| IM_dodge_color3_genosl | <a href="../documents/definitions/dodge.html">dodge</a> | color3 | 
| IM_dodge_color4_genglsl | <a href="../documents/definitions/dodge.html">dodge</a> | color4 | 
| IM_dodge_color4_genmdl | <a href="../documents/definitions/dodge.html">dodge</a> | color4 | 
| IM_dodge_color4_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | color4 | 
| IM_dodge_color4_genosl | <a href="../documents/definitions/dodge.html">dodge</a> | color4 | 
| IM_dodge_float_genglsl | <a href="../documents/definitions/dodge.html">dodge</a> | float | 
| IM_dodge_float_genmdl | <a href="../documents/definitions/dodge.html">dodge</a> | float | 
| IM_dodge_float_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | float | 
| IM_dodge_float_genosl | <a href="../documents/definitions/dodge.html">dodge</a> | float | 
| IM_dot_boolean_genglsl | <a href="../documents/definitions/dot.html">dot</a> | boolean | 
| IM_dot_boolean_genmdl | <a href="../documents/definitions/dot.html">dot</a> | boolean | 
| IM_dot_boolean_genmsl | <a href="../documents/definitions/dot.html">dot</a> | boolean | 
| IM_dot_boolean_genosl | <a href="../documents/definitions/dot.html">dot</a> | boolean | 
| IM_dot_color3_genglsl | <a href="../documents/definitions/dot.html">dot</a> | color3 | 
| IM_dot_color3_genmdl | <a href="../documents/definitions/dot.html">dot</a> | color3 | 
| IM_dot_color3_genmsl | <a href="../documents/definitions/dot.html">dot</a> | color3 | 
| IM_dot_color3_genosl | <a href="../documents/definitions/dot.html">dot</a> | color3 | 
| IM_dot_color4_genglsl | <a href="../documents/definitions/dot.html">dot</a> | color4 | 
| IM_dot_color4_genmdl | <a href="../documents/definitions/dot.html">dot</a> | color4 | 
| IM_dot_color4_genmsl | <a href="../documents/definitions/dot.html">dot</a> | color4 | 
| IM_dot_color4_genosl | <a href="../documents/definitions/dot.html">dot</a> | color4 | 
| IM_dot_displacementshader_genglsl | <a href="../documents/definitions/dot.html">dot</a> | displacementshader | 
| IM_dot_displacementshader_genmdl | <a href="../documents/definitions/dot.html">dot</a> | displacementshader | 
| IM_dot_displacementshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | displacementshader | 
| IM_dot_displacementshader_genosl | <a href="../documents/definitions/dot.html">dot</a> | displacementshader | 
| IM_dot_filename_genglsl | <a href="../documents/definitions/dot.html">dot</a> | filename | 
| IM_dot_filename_genmdl | <a href="../documents/definitions/dot.html">dot</a> | filename | 
| IM_dot_filename_genmsl | <a href="../documents/definitions/dot.html">dot</a> | filename | 
| IM_dot_filename_genosl | <a href="../documents/definitions/dot.html">dot</a> | filename | 
| IM_dot_float_genglsl | <a href="../documents/definitions/dot.html">dot</a> | float | 
| IM_dot_float_genmdl | <a href="../documents/definitions/dot.html">dot</a> | float | 
| IM_dot_float_genmsl | <a href="../documents/definitions/dot.html">dot</a> | float | 
| IM_dot_float_genosl | <a href="../documents/definitions/dot.html">dot</a> | float | 
| IM_dot_integer_genglsl | <a href="../documents/definitions/dot.html">dot</a> | integer | 
| IM_dot_integer_genmdl | <a href="../documents/definitions/dot.html">dot</a> | integer | 
| IM_dot_integer_genmsl | <a href="../documents/definitions/dot.html">dot</a> | integer | 
| IM_dot_integer_genosl | <a href="../documents/definitions/dot.html">dot</a> | integer | 
| IM_dot_lightshader_genglsl | <a href="../documents/definitions/dot.html">dot</a> | lightshader | 
| IM_dot_lightshader_genmdl | <a href="../documents/definitions/dot.html">dot</a> | lightshader | 
| IM_dot_lightshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | lightshader | 
| IM_dot_lightshader_genosl | <a href="../documents/definitions/dot.html">dot</a> | lightshader | 
| IM_dot_matrix33_genglsl | <a href="../documents/definitions/dot.html">dot</a> | matrix33 | 
| IM_dot_matrix33_genmdl | <a href="../documents/definitions/dot.html">dot</a> | matrix33 | 
| IM_dot_matrix33_genmsl | <a href="../documents/definitions/dot.html">dot</a> | matrix33 | 
| IM_dot_matrix33_genosl | <a href="../documents/definitions/dot.html">dot</a> | matrix33 | 
| IM_dot_matrix44_genglsl | <a href="../documents/definitions/dot.html">dot</a> | matrix44 | 
| IM_dot_matrix44_genmdl | <a href="../documents/definitions/dot.html">dot</a> | matrix44 | 
| IM_dot_matrix44_genmsl | <a href="../documents/definitions/dot.html">dot</a> | matrix44 | 
| IM_dot_matrix44_genosl | <a href="../documents/definitions/dot.html">dot</a> | matrix44 | 
| IM_dot_string_genglsl | <a href="../documents/definitions/dot.html">dot</a> | string | 
| IM_dot_string_genmdl | <a href="../documents/definitions/dot.html">dot</a> | string | 
| IM_dot_string_genmsl | <a href="../documents/definitions/dot.html">dot</a> | string | 
| IM_dot_string_genosl | <a href="../documents/definitions/dot.html">dot</a> | string | 
| IM_dot_surfaceshader_genglsl | <a href="../documents/definitions/dot.html">dot</a> | surfaceshader | 
| IM_dot_surfaceshader_genmdl | <a href="../documents/definitions/dot.html">dot</a> | surfaceshader | 
| IM_dot_surfaceshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | surfaceshader | 
| IM_dot_surfaceshader_genosl | <a href="../documents/definitions/dot.html">dot</a> | surfaceshader | 
| IM_dot_vector2_genglsl | <a href="../documents/definitions/dot.html">dot</a> | vector2 | 
| IM_dot_vector2_genmdl | <a href="../documents/definitions/dot.html">dot</a> | vector2 | 
| IM_dot_vector2_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector2 | 
| IM_dot_vector2_genosl | <a href="../documents/definitions/dot.html">dot</a> | vector2 | 
| IM_dot_vector3_genglsl | <a href="../documents/definitions/dot.html">dot</a> | vector3 | 
| IM_dot_vector3_genmdl | <a href="../documents/definitions/dot.html">dot</a> | vector3 | 
| IM_dot_vector3_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector3 | 
| IM_dot_vector3_genosl | <a href="../documents/definitions/dot.html">dot</a> | vector3 | 
| IM_dot_vector4_genglsl | <a href="../documents/definitions/dot.html">dot</a> | vector4 | 
| IM_dot_vector4_genmdl | <a href="../documents/definitions/dot.html">dot</a> | vector4 | 
| IM_dot_vector4_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector4 | 
| IM_dot_vector4_genosl | <a href="../documents/definitions/dot.html">dot</a> | vector4 | 
| IM_dot_volumeshader_genglsl | <a href="../documents/definitions/dot.html">dot</a> | volumeshader | 
| IM_dot_volumeshader_genmdl | <a href="../documents/definitions/dot.html">dot</a> | volumeshader | 
| IM_dot_volumeshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | volumeshader | 
| IM_dot_volumeshader_genosl | <a href="../documents/definitions/dot.html">dot</a> | volumeshader | 
| IM_dotproduct_vector2_genglsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector2_genmdl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector2_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector2_genosl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector3_genglsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector3_genmdl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector3_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector3_genosl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector4_genglsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector4_genmdl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector4_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector4_genosl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_exp_float_genglsl | <a href="../documents/definitions/exp.html">exp</a> | float | 
| IM_exp_float_genmdl | <a href="../documents/definitions/exp.html">exp</a> | float | 
| IM_exp_float_genmsl | <a href="../documents/definitions/exp.html">exp</a> | float | 
| IM_exp_float_genosl | <a href="../documents/definitions/exp.html">exp</a> | float | 
| IM_exp_vector2_genglsl | <a href="../documents/definitions/exp.html">exp</a> | vector2 | 
| IM_exp_vector2_genmdl | <a href="../documents/definitions/exp.html">exp</a> | vector2 | 
| IM_exp_vector2_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector2 | 
| IM_exp_vector2_genosl | <a href="../documents/definitions/exp.html">exp</a> | vector2 | 
| IM_exp_vector3_genglsl | <a href="../documents/definitions/exp.html">exp</a> | vector3 | 
| IM_exp_vector3_genmdl | <a href="../documents/definitions/exp.html">exp</a> | vector3 | 
| IM_exp_vector3_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector3 | 
| IM_exp_vector3_genosl | <a href="../documents/definitions/exp.html">exp</a> | vector3 | 
| IM_exp_vector4_genglsl | <a href="../documents/definitions/exp.html">exp</a> | vector4 | 
| IM_exp_vector4_genmdl | <a href="../documents/definitions/exp.html">exp</a> | vector4 | 
| IM_exp_vector4_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector4 | 
| IM_exp_vector4_genosl | <a href="../documents/definitions/exp.html">exp</a> | vector4 | 
| IM_floor_color3_genglsl | <a href="../documents/definitions/floor.html">floor</a> | color3 | 
| IM_floor_color3_genmdl | <a href="../documents/definitions/floor.html">floor</a> | color3 | 
| IM_floor_color3_genmsl | <a href="../documents/definitions/floor.html">floor</a> | color3 | 
| IM_floor_color3_genosl | <a href="../documents/definitions/floor.html">floor</a> | color3 | 
| IM_floor_color4_genglsl | <a href="../documents/definitions/floor.html">floor</a> | color4 | 
| IM_floor_color4_genmdl | <a href="../documents/definitions/floor.html">floor</a> | color4 | 
| IM_floor_color4_genmsl | <a href="../documents/definitions/floor.html">floor</a> | color4 | 
| IM_floor_color4_genosl | <a href="../documents/definitions/floor.html">floor</a> | color4 | 
| IM_floor_float_genglsl | <a href="../documents/definitions/floor.html">floor</a> | float | 
| IM_floor_float_genmdl | <a href="../documents/definitions/floor.html">floor</a> | float | 
| IM_floor_float_genmsl | <a href="../documents/definitions/floor.html">floor</a> | float | 
| IM_floor_float_genosl | <a href="../documents/definitions/floor.html">floor</a> | float | 
| IM_floor_integer_genglsl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genmdl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genmsl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genosl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_vector2_genglsl | <a href="../documents/definitions/floor.html">floor</a> | vector2 | 
| IM_floor_vector2_genmdl | <a href="../documents/definitions/floor.html">floor</a> | vector2 | 
| IM_floor_vector2_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector2 | 
| IM_floor_vector2_genosl | <a href="../documents/definitions/floor.html">floor</a> | vector2 | 
| IM_floor_vector3_genglsl | <a href="../documents/definitions/floor.html">floor</a> | vector3 | 
| IM_floor_vector3_genmdl | <a href="../documents/definitions/floor.html">floor</a> | vector3 | 
| IM_floor_vector3_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector3 | 
| IM_floor_vector3_genosl | <a href="../documents/definitions/floor.html">floor</a> | vector3 | 
| IM_floor_vector4_genglsl | <a href="../documents/definitions/floor.html">floor</a> | vector4 | 
| IM_floor_vector4_genmdl | <a href="../documents/definitions/floor.html">floor</a> | vector4 | 
| IM_floor_vector4_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector4 | 
| IM_floor_vector4_genosl | <a href="../documents/definitions/floor.html">floor</a> | vector4 | 
| IM_fractal3d_float_genglsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | float | 
| IM_fractal3d_float_genmdl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | float | 
| IM_fractal3d_float_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | float | 
| IM_fractal3d_float_genosl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | float | 
| IM_fractal3d_vector2_genglsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector2_genmdl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector2_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector2_genosl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector3_genglsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector3_genmdl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector3_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector3_genosl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector4_genglsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_fractal3d_vector4_genmdl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_fractal3d_vector4_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_fractal3d_vector4_genosl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_frame_float_genglsl | <a href="../documents/definitions/frame.html">frame</a> | float | 
| IM_frame_float_genmdl | <a href="../documents/definitions/frame.html">frame</a> | float | 
| IM_frame_float_genmsl | <a href="../documents/definitions/frame.html">frame</a> | float | 
| IM_frame_float_genosl | <a href="../documents/definitions/frame.html">frame</a> | float | 
| IM_generalized_schlick_bsdf_genglsl | <a href="../documents/definitions/generalized_schlick_bsdf.html">generalized_schlick_bsdf</a> | BSDF | 
| IM_generalized_schlick_bsdf_genmdl | <a href="../documents/definitions/generalized_schlick_bsdf.html">generalized_schlick_bsdf</a> | BSDF | 
| IM_generalized_schlick_bsdf_genmsl | <a href="../documents/definitions/generalized_schlick_bsdf.html">generalized_schlick_bsdf</a> | BSDF | 
| IM_generalized_schlick_bsdf_genosl | <a href="../documents/definitions/generalized_schlick_bsdf.html">generalized_schlick_bsdf</a> | BSDF | 
| IM_generalized_schlick_edf_genglsl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_generalized_schlick_edf_genmdl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_generalized_schlick_edf_genosl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_geomcolor_color3_genglsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color3 | 
| IM_geomcolor_color3_genmdl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color3 | 
| IM_geomcolor_color3_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color3 | 
| IM_geomcolor_color3_genosl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color3 | 
| IM_geomcolor_color4_genglsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color4 | 
| IM_geomcolor_color4_genmdl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color4 | 
| IM_geomcolor_color4_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color4 | 
| IM_geomcolor_color4_genosl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color4 | 
| IM_geomcolor_float_genglsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | float | 
| IM_geomcolor_float_genmdl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | float | 
| IM_geomcolor_float_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | float | 
| IM_geomcolor_float_genosl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | float | 
| IM_geompropvalue_boolean_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | boolean | 
| IM_geompropvalue_boolean_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | boolean | 
| IM_geompropvalue_boolean_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | boolean | 
| IM_geompropvalue_boolean_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | boolean | 
| IM_geompropvalue_color3_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color3 | 
| IM_geompropvalue_color3_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color3 | 
| IM_geompropvalue_color3_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color3 | 
| IM_geompropvalue_color3_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color3 | 
| IM_geompropvalue_color4_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color4 | 
| IM_geompropvalue_color4_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color4 | 
| IM_geompropvalue_color4_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color4 | 
| IM_geompropvalue_color4_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color4 | 
| IM_geompropvalue_float_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | float | 
| IM_geompropvalue_float_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | float | 
| IM_geompropvalue_float_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | float | 
| IM_geompropvalue_float_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | float | 
| IM_geompropvalue_integer_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | integer | 
| IM_geompropvalue_integer_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | integer | 
| IM_geompropvalue_integer_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | integer | 
| IM_geompropvalue_integer_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | integer | 
| IM_geompropvalue_string_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | string | 
| IM_geompropvalue_string_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | string | 
| IM_geompropvalue_string_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | string | 
| IM_geompropvalue_string_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | string | 
| IM_geompropvalue_vector2_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector2 | 
| IM_geompropvalue_vector2_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector2 | 
| IM_geompropvalue_vector2_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector2 | 
| IM_geompropvalue_vector2_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector2 | 
| IM_geompropvalue_vector3_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector3 | 
| IM_geompropvalue_vector3_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector3 | 
| IM_geompropvalue_vector3_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector3 | 
| IM_geompropvalue_vector3_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector3 | 
| IM_geompropvalue_vector4_genglsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector4 | 
| IM_geompropvalue_vector4_genmdl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector4 | 
| IM_geompropvalue_vector4_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector4 | 
| IM_geompropvalue_vector4_genosl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector4 | 
| IM_heighttonormal_vector3_genglsl | <a href="../documents/definitions/heighttonormal.html">heighttonormal</a> | vector3 | 
| IM_heighttonormal_vector3_genmdl | <a href="../documents/definitions/heighttonormal.html">heighttonormal</a> | vector3 | 
| IM_heighttonormal_vector3_genmsl | <a href="../documents/definitions/heighttonormal.html">heighttonormal</a> | vector3 | 
| IM_heighttonormal_vector3_genosl | <a href="../documents/definitions/heighttonormal.html">heighttonormal</a> | vector3 | 
| IM_hsvtorgb_color3_genglsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color3 | 
| IM_hsvtorgb_color3_genmdl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color3 | 
| IM_hsvtorgb_color3_genmsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color3 | 
| IM_hsvtorgb_color3_genosl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color3 | 
| IM_hsvtorgb_color4_genglsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color4 | 
| IM_hsvtorgb_color4_genmdl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color4 | 
| IM_hsvtorgb_color4_genmsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color4 | 
| IM_hsvtorgb_color4_genosl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color4 | 
| IM_ifequal_color3B_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3B_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3B_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3I_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3I_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3I_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color4B_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4B_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4B_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4I_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4I_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4I_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_floatB_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatB_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatB_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatB_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatI_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatI_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatI_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatI_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_float_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_float_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_float_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_float_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_vector2B_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2B_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2B_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2I_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2I_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2I_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector3B_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3B_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3B_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3I_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3I_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3I_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector4B_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4B_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4B_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4I_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4I_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4I_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4_genglsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4_genmdl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4_genosl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifgreater_color3I_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3I_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3I_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color4I_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4I_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4I_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_floatI_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_floatI_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_floatI_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_floatI_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_float_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_float_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_float_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_float_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_vector2I_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2I_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2I_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector3I_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3I_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3I_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector4I_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4I_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4I_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4_genglsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4_genmdl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4_genosl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreatereq_color3I_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3I_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3I_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color4I_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4I_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4I_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_floatI_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_floatI_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_floatI_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_floatI_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_float_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_float_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_float_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_float_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_vector2I_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2I_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2I_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector3I_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3I_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3I_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector4I_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4I_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4I_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4_genglsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4_genmdl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4_genosl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_image_color3_genglsl | <a href="../documents/definitions/image.html">image</a> | color3 | 
| IM_image_color3_genmdl | <a href="../documents/definitions/image.html">image</a> | color3 | 
| IM_image_color3_genmsl | <a href="../documents/definitions/image.html">image</a> | color3 | 
| IM_image_color3_genosl | <a href="../documents/definitions/image.html">image</a> | color3 | 
| IM_image_color4_genglsl | <a href="../documents/definitions/image.html">image</a> | color4 | 
| IM_image_color4_genmdl | <a href="../documents/definitions/image.html">image</a> | color4 | 
| IM_image_color4_genmsl | <a href="../documents/definitions/image.html">image</a> | color4 | 
| IM_image_color4_genosl | <a href="../documents/definitions/image.html">image</a> | color4 | 
| IM_image_float_genglsl | <a href="../documents/definitions/image.html">image</a> | float | 
| IM_image_float_genmdl | <a href="../documents/definitions/image.html">image</a> | float | 
| IM_image_float_genmsl | <a href="../documents/definitions/image.html">image</a> | float | 
| IM_image_float_genosl | <a href="../documents/definitions/image.html">image</a> | float | 
| IM_image_vector2_genglsl | <a href="../documents/definitions/image.html">image</a> | vector2 | 
| IM_image_vector2_genmdl | <a href="../documents/definitions/image.html">image</a> | vector2 | 
| IM_image_vector2_genmsl | <a href="../documents/definitions/image.html">image</a> | vector2 | 
| IM_image_vector2_genosl | <a href="../documents/definitions/image.html">image</a> | vector2 | 
| IM_image_vector3_genglsl | <a href="../documents/definitions/image.html">image</a> | vector3 | 
| IM_image_vector3_genmdl | <a href="../documents/definitions/image.html">image</a> | vector3 | 
| IM_image_vector3_genmsl | <a href="../documents/definitions/image.html">image</a> | vector3 | 
| IM_image_vector3_genosl | <a href="../documents/definitions/image.html">image</a> | vector3 | 
| IM_image_vector4_genglsl | <a href="../documents/definitions/image.html">image</a> | vector4 | 
| IM_image_vector4_genmdl | <a href="../documents/definitions/image.html">image</a> | vector4 | 
| IM_image_vector4_genmsl | <a href="../documents/definitions/image.html">image</a> | vector4 | 
| IM_image_vector4_genosl | <a href="../documents/definitions/image.html">image</a> | vector4 | 
| IM_in_color4_genglsl | <a href="../documents/definitions/in.html">in</a> | color4 | 
| IM_in_color4_genmdl | <a href="../documents/definitions/in.html">in</a> | color4 | 
| IM_in_color4_genmsl | <a href="../documents/definitions/in.html">in</a> | color4 | 
| IM_in_color4_genosl | <a href="../documents/definitions/in.html">in</a> | color4 | 
| IM_inside_color3_genglsl | <a href="../documents/definitions/inside.html">inside</a> | color3 | 
| IM_inside_color3_genmdl | <a href="../documents/definitions/inside.html">inside</a> | color3 | 
| IM_inside_color3_genmsl | <a href="../documents/definitions/inside.html">inside</a> | color3 | 
| IM_inside_color3_genosl | <a href="../documents/definitions/inside.html">inside</a> | color3 | 
| IM_inside_color4_genglsl | <a href="../documents/definitions/inside.html">inside</a> | color4 | 
| IM_inside_color4_genmdl | <a href="../documents/definitions/inside.html">inside</a> | color4 | 
| IM_inside_color4_genmsl | <a href="../documents/definitions/inside.html">inside</a> | color4 | 
| IM_inside_color4_genosl | <a href="../documents/definitions/inside.html">inside</a> | color4 | 
| IM_inside_float_genglsl | <a href="../documents/definitions/inside.html">inside</a> | float | 
| IM_inside_float_genmdl | <a href="../documents/definitions/inside.html">inside</a> | float | 
| IM_inside_float_genmsl | <a href="../documents/definitions/inside.html">inside</a> | float | 
| IM_inside_float_genosl | <a href="../documents/definitions/inside.html">inside</a> | float | 
| IM_invert_color3FA_genglsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3FA_genmdl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3FA_genosl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3_genglsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3_genmdl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3_genosl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color4FA_genglsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4FA_genmdl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4FA_genosl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4_genglsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4_genmdl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4_genosl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_float_genglsl | <a href="../documents/definitions/invert.html">invert</a> | float | 
| IM_invert_float_genmdl | <a href="../documents/definitions/invert.html">invert</a> | float | 
| IM_invert_float_genmsl | <a href="../documents/definitions/invert.html">invert</a> | float | 
| IM_invert_float_genosl | <a href="../documents/definitions/invert.html">invert</a> | float | 
| IM_invert_vector2FA_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2FA_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2FA_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector3FA_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3FA_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3FA_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector4FA_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4FA_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4FA_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4_genglsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4_genmdl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4_genosl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invertmatrix_matrix33_genglsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix33 | 
| IM_invertmatrix_matrix33_genmdl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix33 | 
| IM_invertmatrix_matrix33_genmsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix33 | 
| IM_invertmatrix_matrix33_genosl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix33 | 
| IM_invertmatrix_matrix44_genglsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix44 | 
| IM_invertmatrix_matrix44_genmdl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix44 | 
| IM_invertmatrix_matrix44_genmsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix44 | 
| IM_invertmatrix_matrix44_genosl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix44 | 
| IM_layer_bsdf_genglsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_bsdf_genmdl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_bsdf_genmsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_bsdf_genosl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genglsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genmdl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genmsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genosl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_light_genglsl | <a href="../documents/definitions/light.html">light</a> | lightshader | 
| IM_light_genmdl | <a href="../documents/definitions/light.html">light</a> | lightshader | 
| IM_light_genmsl | <a href="../documents/definitions/light.html">light</a> | lightshader | 
| IM_ln_float_genglsl | <a href="../documents/definitions/ln.html">ln</a> | float | 
| IM_ln_float_genmdl | <a href="../documents/definitions/ln.html">ln</a> | float | 
| IM_ln_float_genmsl | <a href="../documents/definitions/ln.html">ln</a> | float | 
| IM_ln_float_genosl | <a href="../documents/definitions/ln.html">ln</a> | float | 
| IM_ln_vector2_genglsl | <a href="../documents/definitions/ln.html">ln</a> | vector2 | 
| IM_ln_vector2_genmdl | <a href="../documents/definitions/ln.html">ln</a> | vector2 | 
| IM_ln_vector2_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector2 | 
| IM_ln_vector2_genosl | <a href="../documents/definitions/ln.html">ln</a> | vector2 | 
| IM_ln_vector3_genglsl | <a href="../documents/definitions/ln.html">ln</a> | vector3 | 
| IM_ln_vector3_genmdl | <a href="../documents/definitions/ln.html">ln</a> | vector3 | 
| IM_ln_vector3_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector3 | 
| IM_ln_vector3_genosl | <a href="../documents/definitions/ln.html">ln</a> | vector3 | 
| IM_ln_vector4_genglsl | <a href="../documents/definitions/ln.html">ln</a> | vector4 | 
| IM_ln_vector4_genmdl | <a href="../documents/definitions/ln.html">ln</a> | vector4 | 
| IM_ln_vector4_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector4 | 
| IM_ln_vector4_genosl | <a href="../documents/definitions/ln.html">ln</a> | vector4 | 
| IM_luminance_color3_genglsl | <a href="../documents/definitions/luminance.html">luminance</a> | color3 | 
| IM_luminance_color3_genmdl | <a href="../documents/definitions/luminance.html">luminance</a> | color3 | 
| IM_luminance_color3_genmsl | <a href="../documents/definitions/luminance.html">luminance</a> | color3 | 
| IM_luminance_color3_genosl | <a href="../documents/definitions/luminance.html">luminance</a> | color3 | 
| IM_luminance_color4_genglsl | <a href="../documents/definitions/luminance.html">luminance</a> | color4 | 
| IM_luminance_color4_genmdl | <a href="../documents/definitions/luminance.html">luminance</a> | color4 | 
| IM_luminance_color4_genmsl | <a href="../documents/definitions/luminance.html">luminance</a> | color4 | 
| IM_luminance_color4_genosl | <a href="../documents/definitions/luminance.html">luminance</a> | color4 | 
| IM_magnitude_vector2_genglsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector2_genmdl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector2_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector2_genosl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector3_genglsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector3_genmdl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector3_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector3_genosl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector4_genglsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector4_genmdl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector4_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector4_genosl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_mask_color4_genglsl | <a href="../documents/definitions/mask.html">mask</a> | color4 | 
| IM_mask_color4_genmdl | <a href="../documents/definitions/mask.html">mask</a> | color4 | 
| IM_mask_color4_genmsl | <a href="../documents/definitions/mask.html">mask</a> | color4 | 
| IM_mask_color4_genosl | <a href="../documents/definitions/mask.html">mask</a> | color4 | 
| IM_matte_color4_genglsl | <a href="../documents/definitions/matte.html">matte</a> | color4 | 
| IM_matte_color4_genmdl | <a href="../documents/definitions/matte.html">matte</a> | color4 | 
| IM_matte_color4_genmsl | <a href="../documents/definitions/matte.html">matte</a> | color4 | 
| IM_matte_color4_genosl | <a href="../documents/definitions/matte.html">matte</a> | color4 | 
| IM_max_color3FA_genglsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3FA_genmdl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3FA_genmsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3FA_genosl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3_genglsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3_genmdl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3_genmsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3_genosl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color4FA_genglsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4FA_genmdl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4FA_genmsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4FA_genosl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4_genglsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4_genmdl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4_genmsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4_genosl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_float_genglsl | <a href="../documents/definitions/max.html">max</a> | float | 
| IM_max_float_genmdl | <a href="../documents/definitions/max.html">max</a> | float | 
| IM_max_float_genmsl | <a href="../documents/definitions/max.html">max</a> | float | 
| IM_max_float_genosl | <a href="../documents/definitions/max.html">max</a> | float | 
| IM_max_vector2FA_genglsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2FA_genmdl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2FA_genosl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2_genglsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2_genmdl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2_genmsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2_genosl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector3FA_genglsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3FA_genmdl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3FA_genosl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3_genglsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3_genmdl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3_genmsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3_genosl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector4FA_genglsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4FA_genmdl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4FA_genosl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4_genglsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4_genmdl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4_genmsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4_genosl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_measured_edf_genmdl | <a href="../documents/definitions/measured_edf.html">measured_edf</a> | EDF | 
| IM_min_color3FA_genglsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3FA_genmdl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3FA_genmsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3FA_genosl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3_genglsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3_genmdl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3_genmsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3_genosl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color4FA_genglsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4FA_genmdl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4FA_genmsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4FA_genosl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4_genglsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4_genmdl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4_genmsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4_genosl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_float_genglsl | <a href="../documents/definitions/min.html">min</a> | float | 
| IM_min_float_genmdl | <a href="../documents/definitions/min.html">min</a> | float | 
| IM_min_float_genmsl | <a href="../documents/definitions/min.html">min</a> | float | 
| IM_min_float_genosl | <a href="../documents/definitions/min.html">min</a> | float | 
| IM_min_vector2FA_genglsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2FA_genmdl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2FA_genosl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2_genglsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2_genmdl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2_genmsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2_genosl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector3FA_genglsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3FA_genmdl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3FA_genosl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3_genglsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3_genmdl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3_genmsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3_genosl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector4FA_genglsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4FA_genmdl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4FA_genosl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4_genglsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4_genmdl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4_genmsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4_genosl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_minus_color3_genglsl | <a href="../documents/definitions/minus.html">minus</a> | color3 | 
| IM_minus_color3_genmdl | <a href="../documents/definitions/minus.html">minus</a> | color3 | 
| IM_minus_color3_genmsl | <a href="../documents/definitions/minus.html">minus</a> | color3 | 
| IM_minus_color3_genosl | <a href="../documents/definitions/minus.html">minus</a> | color3 | 
| IM_minus_color4_genglsl | <a href="../documents/definitions/minus.html">minus</a> | color4 | 
| IM_minus_color4_genmdl | <a href="../documents/definitions/minus.html">minus</a> | color4 | 
| IM_minus_color4_genmsl | <a href="../documents/definitions/minus.html">minus</a> | color4 | 
| IM_minus_color4_genosl | <a href="../documents/definitions/minus.html">minus</a> | color4 | 
| IM_minus_float_genglsl | <a href="../documents/definitions/minus.html">minus</a> | float | 
| IM_minus_float_genmdl | <a href="../documents/definitions/minus.html">minus</a> | float | 
| IM_minus_float_genmsl | <a href="../documents/definitions/minus.html">minus</a> | float | 
| IM_minus_float_genosl | <a href="../documents/definitions/minus.html">minus</a> | float | 
| IM_mix_bsdf_genglsl | <a href="../documents/definitions/mix.html">mix</a> | BSDF | 
| IM_mix_bsdf_genmdl | <a href="../documents/definitions/mix.html">mix</a> | BSDF | 
| IM_mix_bsdf_genmsl | <a href="../documents/definitions/mix.html">mix</a> | BSDF | 
| IM_mix_bsdf_genosl | <a href="../documents/definitions/mix.html">mix</a> | BSDF | 
| IM_mix_color3_color3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genosl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_genosl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color4_color4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genosl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_genosl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_displacementshader_genmdl | <a href="../documents/definitions/mix.html">mix</a> | displacementshader | 
| IM_mix_edf_genglsl | <a href="../documents/definitions/mix.html">mix</a> | EDF | 
| IM_mix_edf_genmdl | <a href="../documents/definitions/mix.html">mix</a> | EDF | 
| IM_mix_edf_genmsl | <a href="../documents/definitions/mix.html">mix</a> | EDF | 
| IM_mix_edf_genosl | <a href="../documents/definitions/mix.html">mix</a> | EDF | 
| IM_mix_float_genglsl | <a href="../documents/definitions/mix.html">mix</a> | float | 
| IM_mix_float_genmdl | <a href="../documents/definitions/mix.html">mix</a> | float | 
| IM_mix_float_genmsl | <a href="../documents/definitions/mix.html">mix</a> | float | 
| IM_mix_float_genosl | <a href="../documents/definitions/mix.html">mix</a> | float | 
| IM_mix_surfaceshader_genglsl | <a href="../documents/definitions/mix.html">mix</a> | surfaceshader | 
| IM_mix_surfaceshader_genmdl | <a href="../documents/definitions/mix.html">mix</a> | surfaceshader | 
| IM_mix_surfaceshader_genmsl | <a href="../documents/definitions/mix.html">mix</a> | surfaceshader | 
| IM_mix_surfaceshader_genosl | <a href="../documents/definitions/mix.html">mix</a> | surfaceshader | 
| IM_mix_vdf_genmdl | <a href="../documents/definitions/mix.html">mix</a> | VDF | 
| IM_mix_vector2_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_volumeshader_genmdl | <a href="../documents/definitions/mix.html">mix</a> | volumeshader | 
| IM_modulo_color3FA_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3FA_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3FA_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color4FA_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4FA_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4FA_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_float_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | float | 
| IM_modulo_float_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | float | 
| IM_modulo_float_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | float | 
| IM_modulo_float_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | float | 
| IM_modulo_vector2FA_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2FA_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2FA_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector3FA_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3FA_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3FA_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector4FA_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4FA_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4FA_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4_genglsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4_genmdl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4_genosl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_multiply_bsdfC_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfC_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfC_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfC_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfF_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfF_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfF_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfF_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_color3FA_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3FA_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3FA_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color4FA_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4FA_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4FA_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_edfC_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfC_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfC_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfC_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfF_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfF_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfF_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfF_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_float_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | float | 
| IM_multiply_float_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | float | 
| IM_multiply_float_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | float | 
| IM_multiply_float_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | float | 
| IM_multiply_matrix33_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix33 | 
| IM_multiply_matrix33_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix33 | 
| IM_multiply_matrix33_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix33 | 
| IM_multiply_matrix33_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix33 | 
| IM_multiply_matrix44_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix44 | 
| IM_multiply_matrix44_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix44 | 
| IM_multiply_matrix44_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix44 | 
| IM_multiply_matrix44_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix44 | 
| IM_multiply_vdfC_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | VDF | 
| IM_multiply_vdfF_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | VDF | 
| IM_multiply_vector2FA_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2FA_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2FA_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector3FA_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3FA_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3FA_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector4FA_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4FA_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4FA_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4_genglsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4_genosl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_noise2d_float_genglsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | float | 
| IM_noise2d_float_genmdl | <a href="../documents/definitions/noise2d.html">noise2d</a> | float | 
| IM_noise2d_float_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | float | 
| IM_noise2d_float_genosl | <a href="../documents/definitions/noise2d.html">noise2d</a> | float | 
| IM_noise2d_vector2_genglsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector2_genmdl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector2_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector2_genosl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector3_genglsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector3_genmdl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector3_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector3_genosl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector4_genglsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise2d_vector4_genmdl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise2d_vector4_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise2d_vector4_genosl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise3d_float_genglsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | float | 
| IM_noise3d_float_genmdl | <a href="../documents/definitions/noise3d.html">noise3d</a> | float | 
| IM_noise3d_float_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | float | 
| IM_noise3d_float_genosl | <a href="../documents/definitions/noise3d.html">noise3d</a> | float | 
| IM_noise3d_vector2_genglsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector2_genmdl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector2_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector2_genosl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector3_genglsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector3_genmdl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector3_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector3_genosl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector4_genglsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_noise3d_vector4_genmdl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_noise3d_vector4_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_noise3d_vector4_genosl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_normal_vector3_genglsl | <a href="../documents/definitions/normal.html">normal</a> | vector3 | 
| IM_normal_vector3_genmdl | <a href="../documents/definitions/normal.html">normal</a> | vector3 | 
| IM_normal_vector3_genmsl | <a href="../documents/definitions/normal.html">normal</a> | vector3 | 
| IM_normal_vector3_genosl | <a href="../documents/definitions/normal.html">normal</a> | vector3 | 
| IM_normalize_vector2_genglsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector2 | 
| IM_normalize_vector2_genmdl | <a href="../documents/definitions/normalize.html">normalize</a> | vector2 | 
| IM_normalize_vector2_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector2 | 
| IM_normalize_vector2_genosl | <a href="../documents/definitions/normalize.html">normalize</a> | vector2 | 
| IM_normalize_vector3_genglsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector3 | 
| IM_normalize_vector3_genmdl | <a href="../documents/definitions/normalize.html">normalize</a> | vector3 | 
| IM_normalize_vector3_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector3 | 
| IM_normalize_vector3_genosl | <a href="../documents/definitions/normalize.html">normalize</a> | vector3 | 
| IM_normalize_vector4_genglsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector4 | 
| IM_normalize_vector4_genmdl | <a href="../documents/definitions/normalize.html">normalize</a> | vector4 | 
| IM_normalize_vector4_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector4 | 
| IM_normalize_vector4_genosl | <a href="../documents/definitions/normalize.html">normalize</a> | vector4 | 
| IM_normalmap_float_genglsl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_float_genmdl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_float_genmsl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_float_genosl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_vector2_genglsl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_vector2_genmdl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_vector2_genmsl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_normalmap_vector2_genosl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_oren_nayar_diffuse_bsdf_genglsl | <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">oren_nayar_diffuse_bsdf</a> | BSDF | 
| IM_oren_nayar_diffuse_bsdf_genmdl | <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">oren_nayar_diffuse_bsdf</a> | BSDF | 
| IM_oren_nayar_diffuse_bsdf_genmsl | <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">oren_nayar_diffuse_bsdf</a> | BSDF | 
| IM_oren_nayar_diffuse_bsdf_genosl | <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">oren_nayar_diffuse_bsdf</a> | BSDF | 
| IM_out_color4_genglsl | <a href="../documents/definitions/out.html">out</a> | color4 | 
| IM_out_color4_genmdl | <a href="../documents/definitions/out.html">out</a> | color4 | 
| IM_out_color4_genmsl | <a href="../documents/definitions/out.html">out</a> | color4 | 
| IM_out_color4_genosl | <a href="../documents/definitions/out.html">out</a> | color4 | 
| IM_outside_color3_genglsl | <a href="../documents/definitions/outside.html">outside</a> | color3 | 
| IM_outside_color3_genmdl | <a href="../documents/definitions/outside.html">outside</a> | color3 | 
| IM_outside_color3_genmsl | <a href="../documents/definitions/outside.html">outside</a> | color3 | 
| IM_outside_color3_genosl | <a href="../documents/definitions/outside.html">outside</a> | color3 | 
| IM_outside_color4_genglsl | <a href="../documents/definitions/outside.html">outside</a> | color4 | 
| IM_outside_color4_genmdl | <a href="../documents/definitions/outside.html">outside</a> | color4 | 
| IM_outside_color4_genmsl | <a href="../documents/definitions/outside.html">outside</a> | color4 | 
| IM_outside_color4_genosl | <a href="../documents/definitions/outside.html">outside</a> | color4 | 
| IM_outside_float_genglsl | <a href="../documents/definitions/outside.html">outside</a> | float | 
| IM_outside_float_genmdl | <a href="../documents/definitions/outside.html">outside</a> | float | 
| IM_outside_float_genmsl | <a href="../documents/definitions/outside.html">outside</a> | float | 
| IM_outside_float_genosl | <a href="../documents/definitions/outside.html">outside</a> | float | 
| IM_over_color4_genglsl | <a href="../documents/definitions/over.html">over</a> | color4 | 
| IM_over_color4_genmdl | <a href="../documents/definitions/over.html">over</a> | color4 | 
| IM_over_color4_genmsl | <a href="../documents/definitions/over.html">over</a> | color4 | 
| IM_over_color4_genosl | <a href="../documents/definitions/over.html">over</a> | color4 | 
| IM_plus_color3_genglsl | <a href="../documents/definitions/plus.html">plus</a> | color3 | 
| IM_plus_color3_genmdl | <a href="../documents/definitions/plus.html">plus</a> | color3 | 
| IM_plus_color3_genmsl | <a href="../documents/definitions/plus.html">plus</a> | color3 | 
| IM_plus_color3_genosl | <a href="../documents/definitions/plus.html">plus</a> | color3 | 
| IM_plus_color4_genglsl | <a href="../documents/definitions/plus.html">plus</a> | color4 | 
| IM_plus_color4_genmdl | <a href="../documents/definitions/plus.html">plus</a> | color4 | 
| IM_plus_color4_genmsl | <a href="../documents/definitions/plus.html">plus</a> | color4 | 
| IM_plus_color4_genosl | <a href="../documents/definitions/plus.html">plus</a> | color4 | 
| IM_plus_float_genglsl | <a href="../documents/definitions/plus.html">plus</a> | float | 
| IM_plus_float_genmdl | <a href="../documents/definitions/plus.html">plus</a> | float | 
| IM_plus_float_genmsl | <a href="../documents/definitions/plus.html">plus</a> | float | 
| IM_plus_float_genosl | <a href="../documents/definitions/plus.html">plus</a> | float | 
| IM_point_light_genglsl | <a href="../documents/definitions/point_light.html">point_light</a> | lightshader | 
| IM_point_light_genmsl | <a href="../documents/definitions/point_light.html">point_light</a> | lightshader | 
| IM_position_vector3_genglsl | <a href="../documents/definitions/position.html">position</a> | vector3 | 
| IM_position_vector3_genmdl | <a href="../documents/definitions/position.html">position</a> | vector3 | 
| IM_position_vector3_genmsl | <a href="../documents/definitions/position.html">position</a> | vector3 | 
| IM_position_vector3_genosl | <a href="../documents/definitions/position.html">position</a> | vector3 | 
| IM_power_color3FA_genglsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3FA_genmdl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3FA_genmsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3FA_genosl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3_genglsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3_genmdl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3_genmsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3_genosl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color4FA_genglsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4FA_genmdl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4FA_genmsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4FA_genosl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4_genglsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4_genmdl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4_genmsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4_genosl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_float_genglsl | <a href="../documents/definitions/power.html">power</a> | float | 
| IM_power_float_genmdl | <a href="../documents/definitions/power.html">power</a> | float | 
| IM_power_float_genmsl | <a href="../documents/definitions/power.html">power</a> | float | 
| IM_power_float_genosl | <a href="../documents/definitions/power.html">power</a> | float | 
| IM_power_vector2FA_genglsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2FA_genmdl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2FA_genosl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2_genglsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2_genmdl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2_genmsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2_genosl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector3FA_genglsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3FA_genmdl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3FA_genosl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3_genglsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3_genmdl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3_genmsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3_genosl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector4FA_genglsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4FA_genmdl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4FA_genosl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4_genglsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4_genmdl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4_genmsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4_genosl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_premult_color4_genglsl | <a href="../documents/definitions/premult.html">premult</a> | color4 | 
| IM_premult_color4_genmdl | <a href="../documents/definitions/premult.html">premult</a> | color4 | 
| IM_premult_color4_genmsl | <a href="../documents/definitions/premult.html">premult</a> | color4 | 
| IM_premult_color4_genosl | <a href="../documents/definitions/premult.html">premult</a> | color4 | 
| IM_ramplr_color3_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color3 | 
| IM_ramplr_color3_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color3 | 
| IM_ramplr_color3_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color3 | 
| IM_ramplr_color3_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color3 | 
| IM_ramplr_color4_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color4 | 
| IM_ramplr_color4_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color4 | 
| IM_ramplr_color4_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color4 | 
| IM_ramplr_color4_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color4 | 
| IM_ramplr_float_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | float | 
| IM_ramplr_float_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | float | 
| IM_ramplr_float_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | float | 
| IM_ramplr_float_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | float | 
| IM_ramplr_vector2_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector2 | 
| IM_ramplr_vector2_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector2 | 
| IM_ramplr_vector2_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector2 | 
| IM_ramplr_vector2_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector2 | 
| IM_ramplr_vector3_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector3 | 
| IM_ramplr_vector3_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector3 | 
| IM_ramplr_vector3_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector3 | 
| IM_ramplr_vector3_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector3 | 
| IM_ramplr_vector4_genglsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector4 | 
| IM_ramplr_vector4_genmdl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector4 | 
| IM_ramplr_vector4_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector4 | 
| IM_ramplr_vector4_genosl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector4 | 
| IM_ramptb_color3_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color3 | 
| IM_ramptb_color3_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color3 | 
| IM_ramptb_color3_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color3 | 
| IM_ramptb_color3_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color3 | 
| IM_ramptb_color4_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color4 | 
| IM_ramptb_color4_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color4 | 
| IM_ramptb_color4_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color4 | 
| IM_ramptb_color4_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color4 | 
| IM_ramptb_float_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | float | 
| IM_ramptb_float_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | float | 
| IM_ramptb_float_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | float | 
| IM_ramptb_float_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | float | 
| IM_ramptb_vector2_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector2 | 
| IM_ramptb_vector2_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector2 | 
| IM_ramptb_vector2_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector2 | 
| IM_ramptb_vector2_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector2 | 
| IM_ramptb_vector3_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector3 | 
| IM_ramptb_vector3_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector3 | 
| IM_ramptb_vector3_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector3 | 
| IM_ramptb_vector3_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector3 | 
| IM_ramptb_vector4_genglsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector4 | 
| IM_ramptb_vector4_genmdl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector4 | 
| IM_ramptb_vector4_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector4 | 
| IM_ramptb_vector4_genosl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector4 | 
| IM_remap_color3FA_genglsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3FA_genmdl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3FA_genosl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3_genglsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3_genmdl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3_genosl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color4FA_genglsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4FA_genmdl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4FA_genosl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4_genglsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4_genmdl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4_genosl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_float_genglsl | <a href="../documents/definitions/remap.html">remap</a> | float | 
| IM_remap_float_genmdl | <a href="../documents/definitions/remap.html">remap</a> | float | 
| IM_remap_float_genmsl | <a href="../documents/definitions/remap.html">remap</a> | float | 
| IM_remap_float_genosl | <a href="../documents/definitions/remap.html">remap</a> | float | 
| IM_remap_vector2FA_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2FA_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2FA_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector3FA_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3FA_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3FA_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector4FA_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4FA_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4FA_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4_genglsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4_genmdl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4_genosl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_rgbtohsv_color3_genglsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color3 | 
| IM_rgbtohsv_color3_genmdl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color3 | 
| IM_rgbtohsv_color3_genmsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color3 | 
| IM_rgbtohsv_color3_genosl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color3 | 
| IM_rgbtohsv_color4_genglsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color4 | 
| IM_rgbtohsv_color4_genmdl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color4 | 
| IM_rgbtohsv_color4_genmsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color4 | 
| IM_rgbtohsv_color4_genosl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color4 | 
| IM_rotate2d_vector2_genglsl | <a href="../documents/definitions/rotate2d.html">rotate2d</a> | vector2 | 
| IM_rotate2d_vector2_genmdl | <a href="../documents/definitions/rotate2d.html">rotate2d</a> | vector2 | 
| IM_rotate2d_vector2_genmsl | <a href="../documents/definitions/rotate2d.html">rotate2d</a> | vector2 | 
| IM_rotate2d_vector2_genosl | <a href="../documents/definitions/rotate2d.html">rotate2d</a> | vector2 | 
| IM_rotate3d_vector3_genglsl | <a href="../documents/definitions/rotate3d.html">rotate3d</a> | vector3 | 
| IM_rotate3d_vector3_genmdl | <a href="../documents/definitions/rotate3d.html">rotate3d</a> | vector3 | 
| IM_rotate3d_vector3_genmsl | <a href="../documents/definitions/rotate3d.html">rotate3d</a> | vector3 | 
| IM_rotate3d_vector3_genosl | <a href="../documents/definitions/rotate3d.html">rotate3d</a> | vector3 | 
| IM_roughness_anisotropy_genglsl | <a href="../documents/definitions/roughness_anisotropy.html">roughness_anisotropy</a> | vector2 | 
| IM_roughness_anisotropy_genmdl | <a href="../documents/definitions/roughness_anisotropy.html">roughness_anisotropy</a> | vector2 | 
| IM_roughness_anisotropy_genmsl | <a href="../documents/definitions/roughness_anisotropy.html">roughness_anisotropy</a> | vector2 | 
| IM_roughness_anisotropy_genosl | <a href="../documents/definitions/roughness_anisotropy.html">roughness_anisotropy</a> | vector2 | 
| IM_roughness_dual_genglsl | <a href="../documents/definitions/roughness_dual.html">roughness_dual</a> | vector2 | 
| IM_roughness_dual_genmdl | <a href="../documents/definitions/roughness_dual.html">roughness_dual</a> | vector2 | 
| IM_roughness_dual_genmsl | <a href="../documents/definitions/roughness_dual.html">roughness_dual</a> | vector2 | 
| IM_roughness_dual_genosl | <a href="../documents/definitions/roughness_dual.html">roughness_dual</a> | vector2 | 
| IM_screen_color3_genglsl | <a href="../documents/definitions/screen.html">screen</a> | color3 | 
| IM_screen_color3_genmdl | <a href="../documents/definitions/screen.html">screen</a> | color3 | 
| IM_screen_color3_genmsl | <a href="../documents/definitions/screen.html">screen</a> | color3 | 
| IM_screen_color3_genosl | <a href="../documents/definitions/screen.html">screen</a> | color3 | 
| IM_screen_color4_genglsl | <a href="../documents/definitions/screen.html">screen</a> | color4 | 
| IM_screen_color4_genmdl | <a href="../documents/definitions/screen.html">screen</a> | color4 | 
| IM_screen_color4_genmsl | <a href="../documents/definitions/screen.html">screen</a> | color4 | 
| IM_screen_color4_genosl | <a href="../documents/definitions/screen.html">screen</a> | color4 | 
| IM_screen_float_genglsl | <a href="../documents/definitions/screen.html">screen</a> | float | 
| IM_screen_float_genmdl | <a href="../documents/definitions/screen.html">screen</a> | float | 
| IM_screen_float_genmsl | <a href="../documents/definitions/screen.html">screen</a> | float | 
| IM_screen_float_genosl | <a href="../documents/definitions/screen.html">screen</a> | float | 
| IM_sheen_bsdf_genglsl | <a href="../documents/definitions/sheen_bsdf.html">sheen_bsdf</a> | BSDF | 
| IM_sheen_bsdf_genmdl | <a href="../documents/definitions/sheen_bsdf.html">sheen_bsdf</a> | BSDF | 
| IM_sheen_bsdf_genmsl | <a href="../documents/definitions/sheen_bsdf.html">sheen_bsdf</a> | BSDF | 
| IM_sheen_bsdf_genosl | <a href="../documents/definitions/sheen_bsdf.html">sheen_bsdf</a> | BSDF | 
| IM_sign_color3_genglsl | <a href="../documents/definitions/sign.html">sign</a> | color3 | 
| IM_sign_color3_genmdl | <a href="../documents/definitions/sign.html">sign</a> | color3 | 
| IM_sign_color3_genmsl | <a href="../documents/definitions/sign.html">sign</a> | color3 | 
| IM_sign_color3_genosl | <a href="../documents/definitions/sign.html">sign</a> | color3 | 
| IM_sign_color4_genglsl | <a href="../documents/definitions/sign.html">sign</a> | color4 | 
| IM_sign_color4_genmdl | <a href="../documents/definitions/sign.html">sign</a> | color4 | 
| IM_sign_color4_genmsl | <a href="../documents/definitions/sign.html">sign</a> | color4 | 
| IM_sign_color4_genosl | <a href="../documents/definitions/sign.html">sign</a> | color4 | 
| IM_sign_float_genglsl | <a href="../documents/definitions/sign.html">sign</a> | float | 
| IM_sign_float_genmdl | <a href="../documents/definitions/sign.html">sign</a> | float | 
| IM_sign_float_genmsl | <a href="../documents/definitions/sign.html">sign</a> | float | 
| IM_sign_float_genosl | <a href="../documents/definitions/sign.html">sign</a> | float | 
| IM_sign_vector2_genglsl | <a href="../documents/definitions/sign.html">sign</a> | vector2 | 
| IM_sign_vector2_genmdl | <a href="../documents/definitions/sign.html">sign</a> | vector2 | 
| IM_sign_vector2_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector2 | 
| IM_sign_vector2_genosl | <a href="../documents/definitions/sign.html">sign</a> | vector2 | 
| IM_sign_vector3_genglsl | <a href="../documents/definitions/sign.html">sign</a> | vector3 | 
| IM_sign_vector3_genmdl | <a href="../documents/definitions/sign.html">sign</a> | vector3 | 
| IM_sign_vector3_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector3 | 
| IM_sign_vector3_genosl | <a href="../documents/definitions/sign.html">sign</a> | vector3 | 
| IM_sign_vector4_genglsl | <a href="../documents/definitions/sign.html">sign</a> | vector4 | 
| IM_sign_vector4_genmdl | <a href="../documents/definitions/sign.html">sign</a> | vector4 | 
| IM_sign_vector4_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector4 | 
| IM_sign_vector4_genosl | <a href="../documents/definitions/sign.html">sign</a> | vector4 | 
| IM_sin_float_genglsl | <a href="../documents/definitions/sin.html">sin</a> | float | 
| IM_sin_float_genmdl | <a href="../documents/definitions/sin.html">sin</a> | float | 
| IM_sin_float_genmsl | <a href="../documents/definitions/sin.html">sin</a> | float | 
| IM_sin_float_genosl | <a href="../documents/definitions/sin.html">sin</a> | float | 
| IM_sin_vector2_genglsl | <a href="../documents/definitions/sin.html">sin</a> | vector2 | 
| IM_sin_vector2_genmdl | <a href="../documents/definitions/sin.html">sin</a> | vector2 | 
| IM_sin_vector2_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector2 | 
| IM_sin_vector2_genosl | <a href="../documents/definitions/sin.html">sin</a> | vector2 | 
| IM_sin_vector3_genglsl | <a href="../documents/definitions/sin.html">sin</a> | vector3 | 
| IM_sin_vector3_genmdl | <a href="../documents/definitions/sin.html">sin</a> | vector3 | 
| IM_sin_vector3_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector3 | 
| IM_sin_vector3_genosl | <a href="../documents/definitions/sin.html">sin</a> | vector3 | 
| IM_sin_vector4_genglsl | <a href="../documents/definitions/sin.html">sin</a> | vector4 | 
| IM_sin_vector4_genmdl | <a href="../documents/definitions/sin.html">sin</a> | vector4 | 
| IM_sin_vector4_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector4 | 
| IM_sin_vector4_genosl | <a href="../documents/definitions/sin.html">sin</a> | vector4 | 
| IM_smoothstep_float_genglsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | float | 
| IM_smoothstep_float_genmdl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | float | 
| IM_smoothstep_float_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | float | 
| IM_smoothstep_float_genosl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | float | 
| IM_smoothstep_vector2_genglsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector2_genmdl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector2_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector2_genosl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector3_genglsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector3_genmdl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector3_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector3_genosl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector4_genglsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_smoothstep_vector4_genmdl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_smoothstep_vector4_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_smoothstep_vector4_genosl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_splitlr_color3_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color3 | 
| IM_splitlr_color3_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color3 | 
| IM_splitlr_color3_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color3 | 
| IM_splitlr_color3_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color3 | 
| IM_splitlr_color4_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color4 | 
| IM_splitlr_color4_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color4 | 
| IM_splitlr_color4_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color4 | 
| IM_splitlr_color4_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color4 | 
| IM_splitlr_float_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | float | 
| IM_splitlr_float_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | float | 
| IM_splitlr_float_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | float | 
| IM_splitlr_float_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | float | 
| IM_splitlr_vector2_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector2 | 
| IM_splitlr_vector2_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector2 | 
| IM_splitlr_vector2_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector2 | 
| IM_splitlr_vector2_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector2 | 
| IM_splitlr_vector3_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector3 | 
| IM_splitlr_vector3_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector3 | 
| IM_splitlr_vector3_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector3 | 
| IM_splitlr_vector3_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector3 | 
| IM_splitlr_vector4_genglsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector4 | 
| IM_splitlr_vector4_genmdl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector4 | 
| IM_splitlr_vector4_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector4 | 
| IM_splitlr_vector4_genosl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector4 | 
| IM_splittb_color3_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | color3 | 
| IM_splittb_color3_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | color3 | 
| IM_splittb_color3_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | color3 | 
| IM_splittb_color3_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | color3 | 
| IM_splittb_color4_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | color4 | 
| IM_splittb_color4_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | color4 | 
| IM_splittb_color4_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | color4 | 
| IM_splittb_color4_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | color4 | 
| IM_splittb_float_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | float | 
| IM_splittb_float_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | float | 
| IM_splittb_float_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | float | 
| IM_splittb_float_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | float | 
| IM_splittb_vector2_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector2 | 
| IM_splittb_vector2_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | vector2 | 
| IM_splittb_vector2_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector2 | 
| IM_splittb_vector2_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | vector2 | 
| IM_splittb_vector3_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector3 | 
| IM_splittb_vector3_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | vector3 | 
| IM_splittb_vector3_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector3 | 
| IM_splittb_vector3_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | vector3 | 
| IM_splittb_vector4_genglsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector4 | 
| IM_splittb_vector4_genmdl | <a href="../documents/definitions/splittb.html">splittb</a> | vector4 | 
| IM_splittb_vector4_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector4 | 
| IM_splittb_vector4_genosl | <a href="../documents/definitions/splittb.html">splittb</a> | vector4 | 
| IM_spot_light_genglsl | <a href="../documents/definitions/spot_light.html">spot_light</a> | lightshader | 
| IM_spot_light_genmsl | <a href="../documents/definitions/spot_light.html">spot_light</a> | lightshader | 
| IM_sqrt_float_genglsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | float | 
| IM_sqrt_float_genmdl | <a href="../documents/definitions/sqrt.html">sqrt</a> | float | 
| IM_sqrt_float_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | float | 
| IM_sqrt_float_genosl | <a href="../documents/definitions/sqrt.html">sqrt</a> | float | 
| IM_sqrt_vector2_genglsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector2 | 
| IM_sqrt_vector2_genmdl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector2 | 
| IM_sqrt_vector2_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector2 | 
| IM_sqrt_vector2_genosl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector2 | 
| IM_sqrt_vector3_genglsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector3 | 
| IM_sqrt_vector3_genmdl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector3 | 
| IM_sqrt_vector3_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector3 | 
| IM_sqrt_vector3_genosl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector3 | 
| IM_sqrt_vector4_genglsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector4 | 
| IM_sqrt_vector4_genmdl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector4 | 
| IM_sqrt_vector4_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector4 | 
| IM_sqrt_vector4_genosl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector4 | 
| IM_subsurface_bsdf_genglsl | <a href="../documents/definitions/subsurface_bsdf.html">subsurface_bsdf</a> | BSDF | 
| IM_subsurface_bsdf_genmdl | <a href="../documents/definitions/subsurface_bsdf.html">subsurface_bsdf</a> | BSDF | 
| IM_subsurface_bsdf_genmsl | <a href="../documents/definitions/subsurface_bsdf.html">subsurface_bsdf</a> | BSDF | 
| IM_subsurface_bsdf_genosl | <a href="../documents/definitions/subsurface_bsdf.html">subsurface_bsdf</a> | BSDF | 
| IM_subtract_color3FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color4FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_float_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | float | 
| IM_subtract_float_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | float | 
| IM_subtract_float_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | float | 
| IM_subtract_float_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | float | 
| IM_subtract_matrix33FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix44FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_vector2FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector3FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector4FA_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4FA_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4FA_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4_genglsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4_genmdl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4_genosl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_surface_genglsl | <a href="../documents/definitions/surface.html">surface</a> | surfaceshader | 
| IM_surface_genmdl | <a href="../documents/definitions/surface.html">surface</a> | surfaceshader | 
| IM_surface_genmsl | <a href="../documents/definitions/surface.html">surface</a> | surfaceshader | 
| IM_surface_genosl | <a href="../documents/definitions/surface.html">surface</a> | surfaceshader | 
| IM_surface_unlit_genglsl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genmdl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genmsl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genosl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surfacematerial_genglsl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genmdl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genmsl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genosl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_switch_color3I_genglsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3I_genmdl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3I_genosl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3_genglsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3_genmdl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3_genosl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color4I_genglsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4I_genmdl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4I_genosl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4_genglsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4_genmdl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4_genosl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_floatI_genglsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_floatI_genmdl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_floatI_genmsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_floatI_genosl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_float_genglsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_float_genmdl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_float_genmsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_float_genosl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_vector2I_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2I_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2I_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector3I_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3I_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3I_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector4I_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4I_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4I_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4_genglsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4_genmdl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4_genosl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_swizzle_color3_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color3_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color3_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color3_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color3_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color3_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color3_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color3_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color3_float_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color3_float_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color3_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color3_float_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color3_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color3_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color3_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color3_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color3_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color3_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color3_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color3_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color3_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color3_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color3_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color3_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color4_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color4_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color4_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color4_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color4_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color4_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color4_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color4_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color4_float_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color4_float_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color4_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color4_float_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color4_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color4_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color4_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color4_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color4_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color4_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color4_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color4_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color4_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color4_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color4_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color4_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_float_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_float_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_float_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_float_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_float_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_float_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_float_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_float_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_float_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_float_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_float_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_float_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_float_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_float_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_float_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_float_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_float_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_float_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_float_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_float_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector2_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector2_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector2_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector2_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector2_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector2_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector2_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector2_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector2_float_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector2_float_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector2_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector2_float_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector2_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector2_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector2_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector2_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector2_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector2_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector2_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector2_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector2_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector2_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector2_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector2_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector3_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector3_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector3_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector3_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector3_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector3_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector3_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector3_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector3_float_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector3_float_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector3_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector3_float_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector3_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector3_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector3_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector3_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector3_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector3_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector3_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector3_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector3_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector3_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector3_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector3_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector4_color3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector4_color3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector4_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector4_color3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector4_color4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector4_color4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector4_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector4_color4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector4_float_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector4_float_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector4_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector4_float_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector4_vector2_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector4_vector2_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector4_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector4_vector2_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector4_vector3_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector4_vector3_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector4_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector4_vector3_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector4_vector4_genglsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector4_vector4_genmdl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector4_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector4_vector4_genosl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_tan_float_genglsl | <a href="../documents/definitions/tan.html">tan</a> | float | 
| IM_tan_float_genmdl | <a href="../documents/definitions/tan.html">tan</a> | float | 
| IM_tan_float_genmsl | <a href="../documents/definitions/tan.html">tan</a> | float | 
| IM_tan_float_genosl | <a href="../documents/definitions/tan.html">tan</a> | float | 
| IM_tan_vector2_genglsl | <a href="../documents/definitions/tan.html">tan</a> | vector2 | 
| IM_tan_vector2_genmdl | <a href="../documents/definitions/tan.html">tan</a> | vector2 | 
| IM_tan_vector2_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector2 | 
| IM_tan_vector2_genosl | <a href="../documents/definitions/tan.html">tan</a> | vector2 | 
| IM_tan_vector3_genglsl | <a href="../documents/definitions/tan.html">tan</a> | vector3 | 
| IM_tan_vector3_genmdl | <a href="../documents/definitions/tan.html">tan</a> | vector3 | 
| IM_tan_vector3_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector3 | 
| IM_tan_vector3_genosl | <a href="../documents/definitions/tan.html">tan</a> | vector3 | 
| IM_tan_vector4_genglsl | <a href="../documents/definitions/tan.html">tan</a> | vector4 | 
| IM_tan_vector4_genmdl | <a href="../documents/definitions/tan.html">tan</a> | vector4 | 
| IM_tan_vector4_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector4 | 
| IM_tan_vector4_genosl | <a href="../documents/definitions/tan.html">tan</a> | vector4 | 
| IM_tangent_vector3_genglsl | <a href="../documents/definitions/tangent.html">tangent</a> | vector3 | 
| IM_tangent_vector3_genmdl | <a href="../documents/definitions/tangent.html">tangent</a> | vector3 | 
| IM_tangent_vector3_genmsl | <a href="../documents/definitions/tangent.html">tangent</a> | vector3 | 
| IM_tangent_vector3_genosl | <a href="../documents/definitions/tangent.html">tangent</a> | vector3 | 
| IM_texcoord_vector2_genglsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector2 | 
| IM_texcoord_vector2_genmdl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector2 | 
| IM_texcoord_vector2_genmsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector2 | 
| IM_texcoord_vector2_genosl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector2 | 
| IM_texcoord_vector3_genglsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector3 | 
| IM_texcoord_vector3_genmdl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector3 | 
| IM_texcoord_vector3_genmsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector3 | 
| IM_texcoord_vector3_genosl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector3 | 
| IM_thin_film_bsdf_genglsl | <a href="../documents/definitions/thin_film_bsdf.html">thin_film_bsdf</a> | BSDF | 
| IM_thin_film_bsdf_genmdl | <a href="../documents/definitions/thin_film_bsdf.html">thin_film_bsdf</a> | BSDF | 
| IM_thin_film_bsdf_genmsl | <a href="../documents/definitions/thin_film_bsdf.html">thin_film_bsdf</a> | BSDF | 
| IM_thin_film_bsdf_genosl | <a href="../documents/definitions/thin_film_bsdf.html">thin_film_bsdf</a> | BSDF | 
| IM_thin_surface_genmdl | <a href="../documents/definitions/thin_surface.html">thin_surface</a> | surfaceshader | 
| IM_time_float_genglsl | <a href="../documents/definitions/time.html">time</a> | float | 
| IM_time_float_genmdl | <a href="../documents/definitions/time.html">time</a> | float | 
| IM_time_float_genmsl | <a href="../documents/definitions/time.html">time</a> | float | 
| IM_time_float_genosl | <a href="../documents/definitions/time.html">time</a> | float | 
| IM_transformmatrix_vector2M3_genglsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector2 | 
| IM_transformmatrix_vector2M3_genmdl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector2 | 
| IM_transformmatrix_vector2M3_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector2 | 
| IM_transformmatrix_vector2M3_genosl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector2 | 
| IM_transformmatrix_vector3M4_genglsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3M4_genmdl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3M4_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3M4_genosl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3_genglsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3_genmdl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3_genosl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector4_genglsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector4 | 
| IM_transformmatrix_vector4_genmdl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector4 | 
| IM_transformmatrix_vector4_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector4 | 
| IM_transformmatrix_vector4_genosl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector4 | 
| IM_transformnormal_vector3_genglsl | <a href="../documents/definitions/transformnormal.html">transformnormal</a> | vector3 | 
| IM_transformnormal_vector3_genmdl | <a href="../documents/definitions/transformnormal.html">transformnormal</a> | vector3 | 
| IM_transformnormal_vector3_genmsl | <a href="../documents/definitions/transformnormal.html">transformnormal</a> | vector3 | 
| IM_transformnormal_vector3_genosl | <a href="../documents/definitions/transformnormal.html">transformnormal</a> | vector3 | 
| IM_transformpoint_vector3_genglsl | <a href="../documents/definitions/transformpoint.html">transformpoint</a> | vector3 | 
| IM_transformpoint_vector3_genmdl | <a href="../documents/definitions/transformpoint.html">transformpoint</a> | vector3 | 
| IM_transformpoint_vector3_genmsl | <a href="../documents/definitions/transformpoint.html">transformpoint</a> | vector3 | 
| IM_transformpoint_vector3_genosl | <a href="../documents/definitions/transformpoint.html">transformpoint</a> | vector3 | 
| IM_transformvector_vector3_genglsl | <a href="../documents/definitions/transformvector.html">transformvector</a> | vector3 | 
| IM_transformvector_vector3_genmdl | <a href="../documents/definitions/transformvector.html">transformvector</a> | vector3 | 
| IM_transformvector_vector3_genmsl | <a href="../documents/definitions/transformvector.html">transformvector</a> | vector3 | 
| IM_transformvector_vector3_genosl | <a href="../documents/definitions/transformvector.html">transformvector</a> | vector3 | 
| IM_translucent_bsdf_genglsl | <a href="../documents/definitions/translucent_bsdf.html">translucent_bsdf</a> | BSDF | 
| IM_translucent_bsdf_genmdl | <a href="../documents/definitions/translucent_bsdf.html">translucent_bsdf</a> | BSDF | 
| IM_translucent_bsdf_genmsl | <a href="../documents/definitions/translucent_bsdf.html">translucent_bsdf</a> | BSDF | 
| IM_translucent_bsdf_genosl | <a href="../documents/definitions/translucent_bsdf.html">translucent_bsdf</a> | BSDF | 
| IM_transpose_matrix33_genglsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix33 | 
| IM_transpose_matrix33_genmdl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix33 | 
| IM_transpose_matrix33_genmsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix33 | 
| IM_transpose_matrix33_genosl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix33 | 
| IM_transpose_matrix44_genglsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix44 | 
| IM_transpose_matrix44_genmdl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix44 | 
| IM_transpose_matrix44_genmsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix44 | 
| IM_transpose_matrix44_genosl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix44 | 
| IM_uniform_edf_genglsl | <a href="../documents/definitions/uniform_edf.html">uniform_edf</a> | EDF | 
| IM_uniform_edf_genmdl | <a href="../documents/definitions/uniform_edf.html">uniform_edf</a> | EDF | 
| IM_uniform_edf_genmsl | <a href="../documents/definitions/uniform_edf.html">uniform_edf</a> | EDF | 
| IM_uniform_edf_genosl | <a href="../documents/definitions/uniform_edf.html">uniform_edf</a> | EDF | 
| IM_unpremult_color4_genglsl | <a href="../documents/definitions/unpremult.html">unpremult</a> | color4 | 
| IM_unpremult_color4_genmdl | <a href="../documents/definitions/unpremult.html">unpremult</a> | color4 | 
| IM_unpremult_color4_genmsl | <a href="../documents/definitions/unpremult.html">unpremult</a> | color4 | 
| IM_unpremult_color4_genosl | <a href="../documents/definitions/unpremult.html">unpremult</a> | color4 | 
| IM_viewdirection_vector3_genglsl | <a href="../documents/definitions/viewdirection.html">viewdirection</a> | vector3 | 
| IM_viewdirection_vector3_genmdl | <a href="../documents/definitions/viewdirection.html">viewdirection</a> | vector3 | 
| IM_viewdirection_vector3_genmsl | <a href="../documents/definitions/viewdirection.html">viewdirection</a> | vector3 | 
| IM_viewdirection_vector3_genosl | <a href="../documents/definitions/viewdirection.html">viewdirection</a> | vector3 | 
| IM_volume_genmdl | <a href="../documents/definitions/volume.html">volume</a> | volumeshader | 
| IM_worleynoise2d_float_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_vector2_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector3_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise3d_float_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_vector2_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector3_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| NG_NG_gltf_image_color3_color3_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | color3 | 
| NG_acescg_to_lin_rec709_color3 | <a href="../documents/definitions/acescg_to_lin_rec709.html">acescg_to_lin_rec709</a> | color3 | 
| NG_acescg_to_lin_rec709_color4 | <a href="../documents/definitions/acescg_to_lin_rec709.html">acescg_to_lin_rec709</a> | color4 | 
| NG_adobergb_to_lin_rec709_color3 | <a href="../documents/definitions/adobergb_to_lin_rec709.html">adobergb_to_lin_rec709</a> | color3 | 
| NG_adobergb_to_lin_rec709_color4 | <a href="../documents/definitions/adobergb_to_lin_rec709.html">adobergb_to_lin_rec709</a> | color4 | 
| NG_bump_vector3 | <a href="../documents/definitions/bump.html">bump</a> | vector3 | 
| NG_checkerboard_color3 | <a href="../documents/definitions/checkerboard.html">checkerboard</a> | color3 | 
| NG_circle_float | <a href="../documents/definitions/circle.html">circle</a> | float | 
| NG_cloverleaf_float | <a href="../documents/definitions/cloverleaf.html">cloverleaf</a> | float | 
| NG_colorcorrect_color3 | <a href="../documents/definitions/colorcorrect.html">colorcorrect</a> | color3 | 
| NG_colorcorrect_color4 | <a href="../documents/definitions/colorcorrect.html">colorcorrect</a> | color4 | 
| NG_contrast_color3 | <a href="../documents/definitions/contrast.html">contrast</a> | color3 | 
| NG_contrast_color3FA | <a href="../documents/definitions/contrast.html">contrast</a> | color3 | 
| NG_contrast_color4 | <a href="../documents/definitions/contrast.html">contrast</a> | color4 | 
| NG_contrast_color4FA | <a href="../documents/definitions/contrast.html">contrast</a> | color4 | 
| NG_contrast_float | <a href="../documents/definitions/contrast.html">contrast</a> | float | 
| NG_contrast_vector2 | <a href="../documents/definitions/contrast.html">contrast</a> | vector2 | 
| NG_contrast_vector2FA | <a href="../documents/definitions/contrast.html">contrast</a> | vector2 | 
| NG_contrast_vector3 | <a href="../documents/definitions/contrast.html">contrast</a> | vector3 | 
| NG_contrast_vector3FA | <a href="../documents/definitions/contrast.html">contrast</a> | vector3 | 
| NG_contrast_vector4 | <a href="../documents/definitions/contrast.html">contrast</a> | vector4 | 
| NG_contrast_vector4FA | <a href="../documents/definitions/contrast.html">contrast</a> | vector4 | 
| NG_convert_boolean_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_color3_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_color4_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_float_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_integer_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector2_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector3_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector4_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_crosshatch_color3 | <a href="../documents/definitions/crosshatch.html">crosshatch</a> | color3 | 
| NG_distance_vector2 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_distance_vector3 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_distance_vector4 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_extract_color3 | <a href="../documents/definitions/extract.html">extract</a> | float | 
| NG_extract_color4 | <a href="../documents/definitions/extract.html">extract</a> | float | 
| NG_extract_vector2 | <a href="../documents/definitions/extract.html">extract</a> | float | 
| NG_extract_vector3 | <a href="../documents/definitions/extract.html">extract</a> | float | 
| NG_extract_vector4 | <a href="../documents/definitions/extract.html">extract</a> | float | 
| NG_facingratio_float | <a href="../documents/definitions/facingratio.html">facingratio</a> | float | 
| NG_fractal3d_color3 | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color3 | 
| NG_fractal3d_color3FA | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color3 | 
| NG_fractal3d_color4 | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color4 | 
| NG_fractal3d_color4FA | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color4 | 
| NG_fractal3d_vector2FA | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| NG_fractal3d_vector3FA | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| NG_fractal3d_vector4FA | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| NG_g18_rec709_to_lin_rec709_color3 | <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">g18_rec709_to_lin_rec709</a> | color3 | 
| NG_g18_rec709_to_lin_rec709_color4 | <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">g18_rec709_to_lin_rec709</a> | color4 | 
| NG_g22_ap1_to_lin_rec709_color3 | <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">g22_ap1_to_lin_rec709</a> | color3 | 
| NG_g22_ap1_to_lin_rec709_color4 | <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">g22_ap1_to_lin_rec709</a> | color4 | 
| NG_g22_rec709_to_lin_rec709_color3 | <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">g22_rec709_to_lin_rec709</a> | color3 | 
| NG_g22_rec709_to_lin_rec709_color4 | <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">g22_rec709_to_lin_rec709</a> | color4 | 
| NG_gltf_colorimage | <a href="../documents/definitions/gltf_colorimage.html">gltf_colorimage</a> | multioutput | 
| NG_gltf_image_color4_color4_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | color4 | 
| NG_gltf_image_float_float_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | float | 
| NG_gltf_image_vector3_vector3_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | vector3 | 
| NG_gltf_iridescence_thickness_float_1_0 | <a href="../documents/definitions/gltf_iridescence_thickness.html">gltf_iridescence_thickness</a> | float | 
| NG_gltf_normalmap_vector3_1_0 | <a href="../documents/definitions/gltf_normalmap.html">gltf_normalmap</a> | vector3 | 
| NG_grid_color3 | <a href="../documents/definitions/grid.html">grid</a> | color3 | 
| NG_hexagon_float | <a href="../documents/definitions/hexagon.html">hexagon</a> | float | 
| NG_hsvadjust_color3 | <a href="../documents/definitions/hsvadjust.html">hsvadjust</a> | color3 | 
| NG_hsvadjust_color4 | <a href="../documents/definitions/hsvadjust.html">hsvadjust</a> | color4 | 
| NG_lama_add_bsdf | <a href="../documents/definitions/LamaAdd.html">LamaAdd</a> | BSDF | 
| NG_lama_add_edf | <a href="../documents/definitions/LamaAdd.html">LamaAdd</a> | EDF | 
| NG_lama_diffuse | <a href="../documents/definitions/LamaDiffuse.html">LamaDiffuse</a> | BSDF | 
| NG_lama_layer_bsdf | <a href="../documents/definitions/LamaLayer.html">LamaLayer</a> | BSDF | 
| NG_lama_mix_bsdf | <a href="../documents/definitions/LamaMix.html">LamaMix</a> | BSDF | 
| NG_lama_mix_edf | <a href="../documents/definitions/LamaMix.html">LamaMix</a> | EDF | 
| NG_lama_translucent | <a href="../documents/definitions/LamaTranslucent.html">LamaTranslucent</a> | BSDF | 
| NG_lin_adobergb_to_lin_rec709_color3 | <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">lin_adobergb_to_lin_rec709</a> | color3 | 
| NG_lin_adobergb_to_lin_rec709_color4 | <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">lin_adobergb_to_lin_rec709</a> | color4 | 
| NG_lin_displayp3_to_lin_rec709_color3 | <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">lin_displayp3_to_lin_rec709</a> | color3 | 
| NG_lin_displayp3_to_lin_rec709_color4 | <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">lin_displayp3_to_lin_rec709</a> | color4 | 
| NG_line_float | <a href="../documents/definitions/line.html">line</a> | float | 
| NG_noise2d_color3 | <a href="../documents/definitions/noise2d.html">noise2d</a> | color3 | 
| NG_noise2d_color3FA | <a href="../documents/definitions/noise2d.html">noise2d</a> | color3 | 
| NG_noise2d_color4 | <a href="../documents/definitions/noise2d.html">noise2d</a> | color4 | 
| NG_noise2d_color4FA | <a href="../documents/definitions/noise2d.html">noise2d</a> | color4 | 
| NG_noise2d_vector2FA | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| NG_noise2d_vector3FA | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| NG_noise2d_vector4FA | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| NG_noise3d_color3 | <a href="../documents/definitions/noise3d.html">noise3d</a> | color3 | 
| NG_noise3d_color3FA | <a href="../documents/definitions/noise3d.html">noise3d</a> | color3 | 
| NG_noise3d_color4 | <a href="../documents/definitions/noise3d.html">noise3d</a> | color4 | 
| NG_noise3d_color4FA | <a href="../documents/definitions/noise3d.html">noise3d</a> | color4 | 
| NG_noise3d_vector2FA | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| NG_noise3d_vector3FA | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| NG_noise3d_vector4FA | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| NG_open_pbr_surface_surfaceshader | <a href="../documents/definitions/open_pbr_surface.html">open_pbr_surface</a> | surfaceshader | 
| NG_overlay_color3 | <a href="../documents/definitions/overlay.html">overlay</a> | color3 | 
| NG_overlay_color4 | <a href="../documents/definitions/overlay.html">overlay</a> | color4 | 
| NG_overlay_float | <a href="../documents/definitions/overlay.html">overlay</a> | float | 
| NG_place2d_vector2 | <a href="../documents/definitions/place2d.html">place2d</a> | vector2 | 
| NG_ramp4_color3 | <a href="../documents/definitions/ramp4.html">ramp4</a> | color3 | 
| NG_ramp4_color4 | <a href="../documents/definitions/ramp4.html">ramp4</a> | color4 | 
| NG_ramp4_float | <a href="../documents/definitions/ramp4.html">ramp4</a> | float | 
| NG_ramp4_vector2 | <a href="../documents/definitions/ramp4.html">ramp4</a> | vector2 | 
| NG_ramp4_vector3 | <a href="../documents/definitions/ramp4.html">ramp4</a> | vector3 | 
| NG_ramp4_vector4 | <a href="../documents/definitions/ramp4.html">ramp4</a> | vector4 | 
| NG_randomcolor_float | <a href="../documents/definitions/randomcolor.html">randomcolor</a> | color3 | 
| NG_randomcolor_integer | <a href="../documents/definitions/randomcolor.html">randomcolor</a> | color3 | 
| NG_randomfloat_float | <a href="../documents/definitions/randomfloat.html">randomfloat</a> | float | 
| NG_randomfloat_integer | <a href="../documents/definitions/randomfloat.html">randomfloat</a> | float | 
| NG_range_color3 | <a href="../documents/definitions/range.html">range</a> | color3 | 
| NG_range_color3FA | <a href="../documents/definitions/range.html">range</a> | color3 | 
| NG_range_color4 | <a href="../documents/definitions/range.html">range</a> | color4 | 
| NG_range_color4FA | <a href="../documents/definitions/range.html">range</a> | color4 | 
| NG_range_float | <a href="../documents/definitions/range.html">range</a> | float | 
| NG_range_vector2 | <a href="../documents/definitions/range.html">range</a> | vector2 | 
| NG_range_vector2FA | <a href="../documents/definitions/range.html">range</a> | vector2 | 
| NG_range_vector3 | <a href="../documents/definitions/range.html">range</a> | vector3 | 
| NG_range_vector3FA | <a href="../documents/definitions/range.html">range</a> | vector3 | 
| NG_range_vector4 | <a href="../documents/definitions/range.html">range</a> | vector4 | 
| NG_range_vector4FA | <a href="../documents/definitions/range.html">range</a> | vector4 | 
| NG_rec709_display_to_lin_rec709_color3 | <a href="../documents/definitions/rec709_display_to_lin_rec709.html">rec709_display_to_lin_rec709</a> | color3 | 
| NG_rec709_display_to_lin_rec709_color4 | <a href="../documents/definitions/rec709_display_to_lin_rec709.html">rec709_display_to_lin_rec709</a> | color4 | 
| NG_saturate_color3 | <a href="../documents/definitions/saturate.html">saturate</a> | color3 | 
| NG_saturate_color4 | <a href="../documents/definitions/saturate.html">saturate</a> | color4 | 
| NG_separate2_vector2 | <a href="../documents/definitions/separate2.html">separate2</a> | multioutput | 
| NG_separate3_color3 | <a href="../documents/definitions/separate3.html">separate3</a> | multioutput | 
| NG_separate3_vector3 | <a href="../documents/definitions/separate3.html">separate3</a> | multioutput | 
| NG_separate4_color4 | <a href="../documents/definitions/separate4.html">separate4</a> | multioutput | 
| NG_separate4_vector4 | <a href="../documents/definitions/separate4.html">separate4</a> | multioutput | 
| NG_smoothstep_color3 | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color3 | 
| NG_smoothstep_color3FA | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color3 | 
| NG_smoothstep_color4 | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color4 | 
| NG_smoothstep_color4FA | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color4 | 
| NG_smoothstep_vector2FA | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| NG_smoothstep_vector3FA | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| NG_smoothstep_vector4FA | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| NG_srgb_displayp3_to_lin_rec709_color3 | <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">srgb_displayp3_to_lin_rec709</a> | color3 | 
| NG_srgb_displayp3_to_lin_rec709_color4 | <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">srgb_displayp3_to_lin_rec709</a> | color4 | 
| NG_srgb_texture_to_lin_rec709_color3 | <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">srgb_texture_to_lin_rec709</a> | color3 | 
| NG_srgb_texture_to_lin_rec709_color4 | <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">srgb_texture_to_lin_rec709</a> | color4 | 
| NG_standard_surface_surfaceshader_100 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| NG_standard_surface_to_UsdPreviewSurface | <a href="../documents/definitions/standard_surface_to_UsdPreviewSurface.html">standard_surface_to_UsdPreviewSurface</a> | multioutput | 
| NG_standard_surface_to_gltf_pbr | <a href="../documents/definitions/standard_surface_to_gltf_pbr.html">standard_surface_to_gltf_pbr</a> | multioutput | 
| NG_tiledcircles_color3 | <a href="../documents/definitions/tiledcircles.html">tiledcircles</a> | color3 | 
| NG_tiledcloverleafs_color3 | <a href="../documents/definitions/tiledcloverleafs.html">tiledcloverleafs</a> | color3 | 
| NG_tiledhexagons_color3 | <a href="../documents/definitions/tiledhexagons.html">tiledhexagons</a> | color3 | 
| NG_tiledimage_color3 | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | color3 | 
| NG_tiledimage_color4 | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | color4 | 
| NG_tiledimage_float | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | float | 
| NG_tiledimage_vector2 | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | vector2 | 
| NG_tiledimage_vector3 | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | vector3 | 
| NG_tiledimage_vector4 | <a href="../documents/definitions/tiledimage.html">tiledimage</a> | vector4 | 
| NG_trianglewave_float | <a href="../documents/definitions/trianglewave.html">trianglewave</a> | float | 
| NG_triplanarprojection_color3 | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | color3 | 
| NG_triplanarprojection_color4 | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | color4 | 
| NG_triplanarprojection_float | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | float | 
| NG_triplanarprojection_vector2 | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | vector2 | 
| NG_triplanarprojection_vector3 | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | vector3 | 
| NG_triplanarprojection_vector4 | <a href="../documents/definitions/triplanarprojection.html">triplanarprojection</a> | vector4 | 
| NG_unifiednoise2d_float | <a href="../documents/definitions/unifiednoise2d.html">unifiednoise2d</a> | float | 
| NG_unifiednoise3d_float | <a href="../documents/definitions/unifiednoise3d.html">unifiednoise3d</a> | float | 

</details>


<details><summary>0 implementations removed</summary>

| Name | Node Category | Node Type |
| --- | --- | --- |

</details>


<details><summary>0 implementations modified</summary>

| Name | Change |
| --- | --- |

</details>


### Higher Level Comparisons

As can be seen, the comparisons only provide low level details of count and value changes but not any higher level
semantic meaning. Some examples include:

- The change to inline source code was done for consistency and performance reasons but shows up as a a few hundred attribute changes. Some additional logic could be added to detect a `file` attribute to `sourcecode` attribute but
this could be "brittle" long-term if these attribute names ever changed. Additional information such as how to
perform this update would be useful. For example the script used to perform this can be found in 
<a href="./inlineImplementationFiles.py" target="_blank">inlineImplementationFiles.py</a> file included here.

- Changes for node graph implementations are very hard to understand as only textual comparison is performed
with possible new values and connections reflected when comparing the definitions.

It would probably be more useful to traverse the graphs via it's connections to give a better sense of whether there have been any topological changes (added / removed nodes, changes in connections). Better visual feedback could be given by drawing out the graphs as diagrams. This is beyond the scope of this notebook.

- Changes due to inheritance, whether added, removed or changed, are very hard determine though the change in the `inherit`
attribute or a version change is a first indicator. The utility inheritance functions added here would be useful if added to the core distribution.

- There is a very large number of changes between the previous and current patch release tested (at time of writing).
The same comparison could be performed between patches to see incremental changes.
